In [5]:
python ecosystem.py

SyntaxError: invalid syntax (117202154.py, line 1)

In [2]:
pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 55.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 0.5.4 claude

"""
ECO-sim v0.5.4 (~800 lines)
Major changes from v0.5.3:
1) Fixed predator death counting: now only counts actual prey deaths (line ~450)
2) Added HH:MM:SS time format for simulation time (line ~50)
3) Reorganized stats display into clear categories (line ~100)
4) Changed creature colors to reflect attributes:
   - Red = energy/strength
   - Green = speed 
   - Blue = fear level
5) Updated predator shape to proper cursor design (line ~670)
6) Darker day/night cycle colors (line ~40)
7) Added color and stats legends
8) Fixed mutation tracking and display
"""

import pygame
import numpy as np
import random
from collections import deque, Counter
from datetime import datetime
import math

# =================== CONFIG ====================
MAX_CREATURES = 1000

INITIAL_HERBIVORES = 180
INITIAL_PREDATORS  = 20

INITIAL_FOOD = 40

AGE_BASE = 700

ENERGY_GAIN_FROM_PREY = 60       
ENERGY_GAIN_FROM_FOOD = 15       

BASE_ENERGY_LOSS_PER_MOVE = 0.04
SIZE_ENERGY_LOSS_FACTOR   = 0.08 

REPRODUCE_ENERGY_COST = 50
ENV_REPRO_CHANCE_BASE = 0.3
ENV_REPRO_FOOD_FACTOR = 0.2
ENV_REPRO_POP_FACTOR  = 0.2

MUTATION_RATE = 0.1
BASE_DIET_FLIP_CHANCE = 0.005

FEAR_MUTATION_CHANCE = 0.2
SIZE_MUTATION_CHANCE = 0.2
LONGEVITY_MUTATION_CHANCE = 0.2

MAX_SPEED = 8.0
MIN_SPEED = 0.5

MIN_SIZE  = 0.5
MAX_SIZE  = 2.0   

IDLE_COST = 0.01  

FEAR_COST_BASE = 0.02
FEAR_THRESHOLD_HERB = 0.3

AMBUSH_BONUS = 1.1  
PRED_FEAR_THRESHOLD = 0.5   
SIZE_DOMINANCE_RATIO = 1.2  
HERB_SIZE_DOMINANCE_RATIO = 1.2

SIGHT_RANGE = 200.0
BLOCKING_RADIUS = 5.0

DAY_LENGTH = 600        
DAY_COLOR = (100, 180, 220)    # Darker aquamarine with slight green tint
NIGHT_COLOR = (0, 15, 45)      # Even darker navy

DAYTIME_FOOD_SPAWN_CHANCE = 0.02

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)
PREDATOR_GRAPH_COLOR = (150, 100, 200)

GRID_CELL_SIZE = 25   
COLLISION_PASSES = 2

def format_sim_time(frames):
    """Convert frame count to hours:minutes:seconds"""
    total_seconds = frames // 30
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def format_stats(creatures, foods, day_count, sim_frames, death_log, avg_frame_time, mutation_tracker):
    stats = []
    
    # Population Stats
    pred_count = sum(1 for x in creatures if x and x.diet==1 and not x.dead)
    herb_count = len(creatures) - pred_count
    ratio_str = "∞" if pred_count == 0 else f"{herb_count/pred_count:.1f}"
    
    stats.extend([
        "=== Population ===",
        f"Total: {len(creatures)}/{MAX_CREATURES}",
        f"Herbivores: {herb_count}",
        f"Predators: {pred_count} (ratio 1:{ratio_str})",
        f"Food: {len(foods)}",
        ""
    ])

    # Time Stats
    stats.extend([
        "=== Time ===",
        f"Sim Time: {format_sim_time(sim_frames)}",
        f"Day Count: {day_count}",
        f"FPS: {int(clock.get_fps())}",
        f"Frame Time: {avg_frame_time:.1f}ms",
        ""
    ])

    # Evolution Stats
    if creatures:
        live_creatures = [c for c in creatures if c and not c.dead]
        avg_speed = np.mean([c.speed for c in live_creatures])
        avg_size = np.mean([c.size_factor for c in live_creatures])
        avg_fear = np.mean([c.fear for c in live_creatures])
        
        stats.extend([
            "=== Evolution ===",
            f"Avg Speed: {avg_speed:.2f}",
            f"Avg Size: {avg_size:.2f}",
            f"Avg Fear: {avg_fear:.2f}",
            f"Deaths - Starved: {death_log['starved']}",
            f"Deaths - Age: {death_log['aged']}",
            f"Deaths - Hunted: {death_log['hunted']}",
            ""
        ])

    # Color Legend
    stats.extend([
        "=== Color Legend ===",
        "Red = Energy/Strength",
        "Green = Speed",
        "Blue = Fear Level",
        ""
    ])

    # Controls
    stats.extend([
        "=== Controls ===",
        "[SPACE] Pause/Resume"
    ])
    
    return stats

def write_log(message):
    if sim_frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {sim_frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    ABx = bx - ax
    ABy = by - ay
    APx = px - ax
    APy = py - ay
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        return distance(px, py, ax, ay)
    t = (APx*ABx + APy*ABy) / AB_len_sq
    t = max(0, min(1, t))
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

def calculate_creature_color(energy, speed, fear):
    """Calculate creature color based on attributes"""
    r = int(np.clip((energy / 100.0) * 255, 50, 255))
    g = int(np.clip((speed / MAX_SPEED) * 255, 50, 255))
    b = int(np.clip(fear * 255, 50, 255))
    return (r, g, b)

# =============== Collisions using Uniform Grid ===============
def collision_broad_phase_grid(creatures):
    grid_map = {}
    for i, c in enumerate(creatures):
        if c is None:
            continue
        cell_x = int(c.x // GRID_CELL_SIZE)
        cell_y = int(c.y // GRID_CELL_SIZE)
        key = (cell_x, cell_y)
        grid_map.setdefault(key, []).append(i)
    return grid_map

def collision_narrow_phase(creatures, indices):
    for i in range(len(indices)):
        for j in range(i+1, len(indices)):
            idxA = indices[i]
            idxB = indices[j]
            c1 = creatures[idxA]
            c2 = creatures[idxB]
            if c1 is None or c2 is None:
                continue
            r1 = c1.radius()
            r2 = c2.radius()
            dx = c2.x - c1.x
            dy = c2.y - c1.y
            dist = math.hypot(dx, dy)
            min_dist = r1 + r2
            if dist < min_dist and dist > 0:
                overlap = 0.5 * (min_dist - dist)
                nx = dx / dist
                ny = dy / dist
                c1.x -= overlap * nx
                c1.y -= overlap * ny
                c2.x += overlap * nx
                c2.y += overlap * ny

def resolve_collisions(creatures):
    neighbor_offsets = [
        (0,0),(1,0),(0,1),(1,1),
        (-1,0),(0,-1),(-1,-1),(1,-1),(-1,1)
    ]
    for _ in range(COLLISION_PASSES):
        grid_map = collision_broad_phase_grid(creatures)
        for cell_key, idx_list in grid_map.items():
            collision_narrow_phase(creatures, idx_list)
            for (nx, ny) in neighbor_offsets:
                neigh_key = (cell_key[0]+nx, cell_key[1]+ny)
                if neigh_key in grid_map:
                    combined = idx_list + grid_map[neigh_key]
                    collision_narrow_phase(creatures, combined)

# =====================================================================
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    v0.5.4 changes:
     - colors now reflect attributes: energy=red, speed=green, fear=blue
     - changed shape to cursor for predators
     - fixed hunting death counter
     - dynamic color updates based on current state
    """
    def __init__(self, x, y, father_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        self.heading = random.random() * 2.0 * math.pi
        self.dead = False

        if father_dna:
            speed = np.clip(father_dna[0] * random.uniform(0.9, 1.1), MIN_SPEED, MAX_SPEED)
            fear  = father_dna[1]
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            size_factor = father_dna[2]
            if random.random() < SIZE_MUTATION_CHANCE:
                size_factor *= random.uniform(0.9, 1.1)
                size_factor = min(max(MIN_SIZE, size_factor), MAX_SIZE)

            longevity = father_dna[3]
            if random.random() < LONGEVITY_MUTATION_CHANCE:
                longevity *= random.uniform(0.8, 1.2)
                longevity = min(max(0.5, longevity), 2.0)

            diet = father_dna[4]

            self.dna = [speed, fear, size_factor, longevity, diet]

            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], MIN_SPEED, MAX_SPEED)
        else:
            self.dna = [
                random.uniform(1.5, 3.0),
                random.uniform(0.0, 0.2),
                random.uniform(MIN_SIZE, 1.2),
                random.uniform(0.8, 1.2),
                0
            ]

        self.speed = self.dna[0]
        self.fear = self.dna[1]
        self.size_factor = self.dna[2]
        self.longevity = self.dna[3]
        self.diet = int(self.dna[4])
        
        # Color based on attributes
        self.update_color()

    def update_color(self):
        """Update creature color based on current attributes"""
        self.color = calculate_creature_color(self.energy, self.speed, self.fear)

    def _get_mutation_type(self, father_dna):
        if not father_dna:
            return "initial"
        changes = []
        if abs(self.dna[0] - father_dna[0]) > 1.0:
            changes.append("speed_shift")
        if abs(self.dna[1] - father_dna[1]) > 0.05:
            changes.append("fear_shift")
        if abs(self.dna[2] - father_dna[2]) > 0.3:
            changes.append("size_shift")
        if abs(self.dna[3] - father_dna[3]) > 0.3:
            changes.append("long_shift")
        if self.dna[4] != father_dna[4]:
            changes.append("diet_flip")
        if not changes:
            return "minor"
        return "+".join(changes)

    def max_age(self):
        return AGE_BASE * self.longevity

    def strength(self):
        return self.size_factor * (1.0 + 0.5*(self.energy/100.0))

    def max_energy_capacity(self):
        return 100 + (self.size_factor - 1.0)*50

    def radius(self):
        return 6 * self.size_factor

    def update(self, all_creatures, foods):
        if self.dead:
            return ("starved", None)
        self.age += 1
        self.energy -= IDLE_COST

        if self.diet == 0 and self.fear > FEAR_THRESHOLD_HERB:
            self.energy -= FEAR_COST_BASE * self.fear * self.size_factor
        elif self.diet == 1 and self.fear > PRED_FEAR_THRESHOLD:
            self.energy -= FEAR_COST_BASE * self.fear * (self.size_factor * 0.5)

        self.energy -= BASE_ENERGY_LOSS_PER_MOVE
        self.energy -= (self.size_factor * SIZE_ENERGY_LOSS_FACTOR)
        self.energy = max(self.energy, 0)
        
        # Update color based on new energy
        self.update_color()

        if self.diet == 0:
            if self.fear > FEAR_THRESHOLD_HERB:
                predator, distp = self.find_nearest(all_creatures, lambda c: c is not self and c.diet == 1 and not c.dead)
                if predator and distp < 120:
                    self.run_away(predator.x, predator.y)
                    self.wrap_screen()
                    return self.check_survival()

            food_t, distf = self.find_nearest(foods)
            if food_t and distf < SIGHT_RANGE:
                if not self.is_line_blocked(food_t.x, food_t.y, all_creatures):
                    self.move_toward(food_t.x, food_t.y)
                    if distance(self.x, self.y, food_t.x, food_t.y) < (6 + self.size_factor*4):
                        self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_FOOD)
                        foods.remove(food_t)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        else:
            ctarget, distc = self.find_nearest(all_creatures, lambda c: c is not self and not c.dead)
            if ctarget and distc < SIGHT_RANGE:
                if not self.is_line_blocked(ctarget.x, ctarget.y, all_creatures):
                    if ctarget.diet == 1:
                        if self.fear > PRED_FEAR_THRESHOLD and (ctarget.size_factor > self.size_factor*SIZE_DOMINANCE_RATIO):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                return self.predator_fight(ctarget)
                    else:
                        if (ctarget.size_factor>self.size_factor*HERB_SIZE_DOMINANCE_RATIO) and (self.fear> PRED_FEAR_THRESHOLD):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                gain = ENERGY_GAIN_FROM_PREY * ctarget.size_factor
                                self.energy = min(self.max_energy_capacity(), self.energy + gain)
                                ctarget.dead = True
                                return ("ate_creature", ctarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def predator_fight(self, other):
        if other.dead:
            return ("alive", None)
        if random.random() < 0.3:
            my_str = self.strength() * AMBUSH_BONUS
            their_str = other.strength()
        else:
            my_str = self.strength()
            their_str = other.strength()
        
        if my_str > their_str:
            gain = ENERGY_GAIN_FROM_PREY * other.size_factor
            self.energy = min(self.max_energy_capacity(), self.energy + gain)
            other.dead = True
            if other.diet == 0:
                return ("ate_creature", other)
            return ("pred_fight", other)
        elif their_str > my_str:
            self.dead = True
            if self.diet == 0:
                return ("ate_creature", self)
            return ("pred_fight", self)
        else:
            if random.random() < 0.5:
                other.dead = True
                if other.diet == 0:
                    return ("ate_creature", other)
                return ("pred_fight", other)
            else:
                self.dead = True
                if self.diet == 0:
                    return ("ate_creature", self)
                return ("pred_fight", self)

    def check_survival(self):
        if self.dead:
            return ("starved", None)
        if self.energy <= 0:
            self.dead = True
            return ("starved", None)
        if self.age > self.max_age():
            self.dead = True
            return ("aged", None)
        return ("alive", None)

    def reproduce(self, creatures, foods):
        if self.dead:
            return None
        if self.energy < 80:
            return None
        pop_size = len(creatures)
        food_count = len(foods)
        ratio_food_pop = food_count/pop_size if pop_size>0 else 1
        ratio_pop_cap = pop_size / MAX_CREATURES

        chance = ENV_REPRO_CHANCE_BASE
        chance += ENV_REPRO_FOOD_FACTOR*ratio_food_pop
        chance -= ENV_REPRO_POP_FACTOR*ratio_pop_cap
        chance = min(max(chance,0.0),1.0)

        if random.random() < chance:
            self.energy -= REPRODUCE_ENERGY_COST
            child_dna = list(self.dna)

            herb_count = sum(1 for c in creatures if c and c.diet==0 and not c.dead)
            pred_count = pop_size-herb_count
            flip_chance = BASE_DIET_FLIP_CHANCE

            if self.diet==0:
                if food_count < (herb_count*0.7):
                    flip_chance += 0.05
                if (self.size_factor>1.5 and ratio_food_pop<0.5):
                    flip_chance += 0.05
                if random.random()<flip_chance:
                    child_dna[4] = 1
            else:
                if pred_count>(herb_count*0.75):
                    flip_chance += 0.05
                if random.random()<flip_chance:
                    child_dna[4]=0

            offx = random.gauss(0, 10)
            offy = random.gauss(0, 10)
            baby = Creature(self.x+offx, self.y+offy, father_dna=child_dna)
            return baby
        return None

    def random_walk(self):
        self.heading += random.uniform(-0.2, 0.2)
        dx = math.cos(self.heading)*self.speed
        dy = math.sin(self.heading)*self.speed
        self.x+=dx
        self.y+=dy

    def run_away(self,ox,oy):
        dx=self.x-ox
        dy=self.y-oy
        distv=distance(self.x,self.y,ox,oy)
        if distv>1:
            self.x+=(dx/distv)*self.speed
            self.y+=(dy/distv)*self.speed
            # Update heading based on movement direction
            self.heading = math.atan2(dy, dx)
        else:
            self.random_walk()

    def move_toward(self,tx,ty):
        dx=tx-self.x
        dy=ty-self.y
        distv=distance(self.x,self.y,tx,ty)
        if distv>1:
            self.x+=(dx/distv)*self.speed
            self.y+=(dy/distv)*self.speed
            # Update heading based on movement direction
            self.heading = math.atan2(dy, dx)
        else:
            self.random_walk()

    def wrap_screen(self):
        self.x%=1200
        self.y%=800

    def find_nearest(self, target_list, condition=None):
        valids=[t for t in target_list if(t and(t is not self))]
        if condition:
            valids=[v for v in valids if condition(v)]
        if not valids:
            return(None,float('inf'))
        best_obj,best_dist=None,float('inf')
        for obj in valids:
            d=distance(self.x,self.y,obj.x,obj.y)
            if d<best_dist:
                best_dist=d
                best_obj=obj
        return(best_obj,best_dist)

    def is_line_blocked(self,tx,ty,creatures):
        targ_dist=distance(self.x,self.y,tx,ty)
        for c in creatures:
            if not c or c is self or c.dead:
                continue
            dist_c=distance(self.x,self.y,c.x,c.y)
            if dist_c>=targ_dist:
                continue
            d_line=line_distance_point(c.x,c.y,self.x,self.y,tx,ty)
            if d_line<BLOCKING_RADIUS:
                return True
        return False

    def draw(self, surface):
        if self.dead:
            return
        sz = int(8*self.size_factor)
        if self.diet == 0:
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), sz)
        else:
            # Draw cursor shape
            tip_x = self.x + math.cos(self.heading) * sz * 2.0
            tip_y = self.y + math.sin(self.heading) * sz * 2.0
            
            # Calculate perpendicular vector for cursor base
            perp_x = math.cos(self.heading + math.pi/2)
            perp_y = math.sin(self.heading + math.pi/2)
            
            # Calculate base position (tail of cursor)
            base_x = self.x - math.cos(self.heading) * sz
            base_y = self.y - math.sin(self.heading) * sz
            
            # Calculate the points for a cursor shape (like a mouse cursor)
            points = [
                (tip_x, tip_y),                                           # Front tip
                (base_x + perp_x * sz, base_y + perp_y * sz),           # Right base
                (base_x, base_y),                                        # Center base
                (base_x - perp_x * sz, base_y - perp_y * sz)            # Left base
            ]
            pygame.draw.polygon(surface, self.color, points)

# ==================== SETUP =========================
open(LOG_FILE,"w").close()

creatures=[]
for _ in range(INITIAL_HERBIVORES):
    c=Creature(random.randint(100,1100),random.randint(100,700))
    c.dna[4]=0
    c.diet=0
    creatures.append(c)
for _ in range(INITIAL_PREDATORS):
    c=Creature(random.randint(100,1100),random.randint(100,700))
    c.dna[4]=1
    c.diet=1
    creatures.append(c)

foods=[Food() for _ in range(INITIAL_FOOD)]

paused=False
frames=0
sim_frames=0
population_history=deque(maxlen=300)
pred_ratio_history=deque(maxlen=300)

death_log={'starved':0,'aged':0,'hunted':0}
mutation_tracker=Counter()

day_cycle_frame=0
day_count=0
fps_sample_timer=0.0
avg_frame_time=0.0

running=True

while running:
    dt=clock.tick(30)
    frames+=1

    fps_sample_timer+=dt
    avg_frame_time=0.9*avg_frame_time+0.1*dt

    for event in pygame.event.get():
        if event.type==pygame.QUIT:
            running=False
        elif event.type==pygame.KEYDOWN:
            if event.key==pygame.K_SPACE:
                paused=not paused

    if not paused:
        sim_frames+=1
        day_cycle_frame=(day_cycle_frame+1)%DAY_LENGTH
        if day_cycle_frame==0:
            day_count+=1

    alpha=day_cycle_frame/(DAY_LENGTH-1)if(DAY_LENGTH>1)else 1.0
    if alpha<0.5:
        sub_a=alpha/0.5
        bg_color=interpolate_color(NIGHT_COLOR,DAY_COLOR,sub_a)
        is_day=True
    else:
        sub_a=(alpha-0.5)/0.5
        bg_color=interpolate_color(DAY_COLOR,NIGHT_COLOR,sub_a)
        is_day=False

    screen.fill(bg_color)

    if not paused:
        new_creatures=[]
        dead=[]

        for c in creatures:
            if c and not c.dead:
                status,other=c.update(creatures,foods)
                if status=="starved":
                    death_log["starved"]+=1
                    write_log(f"Death: starved (speed={c.speed:.2f})")
                    dead.append(c)
                    food_chunks=1+int(c.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
                elif status=="aged":
                    death_log["aged"]+=1
                    write_log(f"Death: aged (speed={c.speed:.2f})")
                    dead.append(c)
                    food_chunks=1+int(c.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
                elif status=="ate_creature":
                    if other.diet == 0:  # Only count herbivore deaths
                        death_log["hunted"]+=1
                    write_log(f"Death: {'hunted' if other.diet == 0 else 'pred_fight'} (speed={other.speed:.2f})")
                    dead.append(other)
                    food_chunks=1+int(other.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
                elif status=="pred_fight":
                    write_log(f"Death: pred_fight (speed={other.speed:.2f})")
                    dead.append(other)
                    food_chunks=1+int(other.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
                else:
                    child=c.reproduce(creatures,foods)
                    if child and len(creatures)<MAX_CREATURES:
                        mut_type=child._get_mutation_type(c.dna)
                        if mut_type not in("initial","minor"):
                            mutation_tracker[mut_type]+=1
                        new_creatures.append(child)
                        write_log(
                            f"Birth: speed={child.speed:.2f}, diet={child.diet}, size={child.size_factor:.2f}, "
                            f"fear={child.fear:.2f}, longevity={child.longevity:.2f}, mutation={mut_type}"
                        )

        for dcreat in dead:
            if dcreat in creatures:
                idx=creatures.index(dcreat)
                creatures[idx]=None

        creatures=[cr for cr in creatures if cr is not None and not cr.dead]
        space_left=max(0,MAX_CREATURES-len(creatures))
        creatures+=new_creatures[:space_left]

        resolve_collisions(creatures)

        pop_size=len(creatures)
        pred_count=sum(1 for cr in creatures if cr.diet==1 and not cr.dead)
        population_history.append(pop_size)
        ratio=pred_count/pop_size if pop_size>0 else 0
        pred_ratio_history.append(ratio)

        if is_day:
            if random.random()<DAYTIME_FOOD_SPAWN_CHANCE:
                foods.append(Food())

    for f in foods:
        pygame.draw.circle(screen,FOOD_COLOR,(int(f.x),int(f.y)),4)
    for cr in creatures:
        if cr and not cr.dead:
            cr.draw(screen)

    stats = format_stats(creatures, foods, day_count, sim_frames, death_log, avg_frame_time, mutation_tracker)
    for i, text in enumerate(stats):
        txt = font.render(text, True, (255, 255, 255))
        screen.blit(txt, (10, 10 + i*25))

    mut_top3 = mutation_tracker.most_common(3)
    if mut_top3:
        mut_strs = [f"{mt}:{cnt}" for (mt, cnt) in mut_top3]
        mut_line = "=== Mutations === " + ", ".join(mut_strs)
        txtm = font.render(mut_line, True, (255, 255, 255))
        screen.blit(txtm, (10, 10 + len(stats)*25))

    graph_x, graph_y = 800, 70
    graph_w, graph_h = 380, 90
    pygame.draw.rect(screen, (50, 50, 50), (graph_x, graph_y, graph_w, graph_h))

    if len(population_history)>1:
        points_pop = []
        for i, val in enumerate(population_history):
            xx = graph_x + graph_w-(len(population_history)-i)*2
            yy = graph_y + graph_h-((val/MAX_CREATURES)*graph_h)
            points_pop.append((xx, yy))
        pygame.draw.lines(screen, GRAPH_COLOR, False, points_pop, 2)

    if len(pred_ratio_history)>1:
        points_pred = []
        for i, ratio_val in enumerate(pred_ratio_history):
            xx = graph_x + graph_w-(len(pred_ratio_history)-i)*2
            yy = graph_y + graph_h-(ratio_val*graph_h)
            points_pred.append((xx, yy))
        pygame.draw.lines(screen, PREDATOR_GRAPH_COLOR, False, points_pred, 2)

    pygame.display.flip()

pygame.quit()

In [ ]:
# 0.5.3.1  
# added cursor predator and changed day/night colors via claude

"""
emergent_sim_v0.5.3 (~700 lines)
Major changes from v0.5.2:
1) Added an idle cost + higher size-based cost.
2) Lowered ENERGY_GAIN_FROM_PREY/FOOD.
3) Narrowed size mutation range & smaller max size = 2.0.
4) Hunting overcount fix: once a predator kills a victim, we mark victim "dead" so it can't be double-counted.
5) Slower day/night => from 240 frames to 600 frames.
6) Smaller cell size for collision from 40 to 25.
"""

import pygame
import numpy as np
import random
from collections import deque, Counter
from datetime import datetime
import math

# =================== CONFIG ====================
MAX_CREATURES = 1000

INITIAL_HERBIVORES = 180
INITIAL_PREDATORS  = 20

INITIAL_FOOD = 40

AGE_BASE = 700

ENERGY_GAIN_FROM_PREY = 60       ### CHANGED (was 80)
ENERGY_GAIN_FROM_FOOD = 15       ### CHANGED (was 22)

BASE_ENERGY_LOSS_PER_MOVE = 0.04
SIZE_ENERGY_LOSS_FACTOR   = 0.08 ### CHANGED (was 0.04)

REPRODUCE_ENERGY_COST = 50
ENV_REPRO_CHANCE_BASE = 0.3
ENV_REPRO_FOOD_FACTOR = 0.2
ENV_REPRO_POP_FACTOR  = 0.2

MUTATION_RATE = 0.1
BASE_DIET_FLIP_CHANCE = 0.005

FEAR_MUTATION_CHANCE = 0.2
SIZE_MUTATION_CHANCE = 0.2
LONGEVITY_MUTATION_CHANCE = 0.2

MAX_SPEED = 8.0
MIN_SPEED = 0.5

MIN_SIZE  = 0.5
MAX_SIZE  = 2.0   ### CHANGED (was 2.5)

# We'll add an idle cost each frame:
IDLE_COST = 0.01  ### NEW

FEAR_COST_BASE = 0.02
FEAR_THRESHOLD_HERB = 0.3

AMBUSH_BONUS = 1.1  
PRED_FEAR_THRESHOLD = 0.5   
SIZE_DOMINANCE_RATIO = 1.2  
HERB_SIZE_DOMINANCE_RATIO = 1.2

SIGHT_RANGE = 200.0
BLOCKING_RADIUS = 5.0

DAY_LENGTH = 600        ### CHANGED (was 240 => slower day/night)
DAY_COLOR = (154, 224, 255)  # Aquamarine (changed from Light ocean blue)
NIGHT_COLOR = (0, 20, 60)    # Very dark navy blue (changed from black)

DAYTIME_FOOD_SPAWN_CHANCE = 0.02

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)
PREDATOR_GRAPH_COLOR = (150, 100, 200)

# Grid settings for collision broad-phase
GRID_CELL_SIZE = 25   ### CHANGED (was 40 => smaller => more accurate)
COLLISION_PASSES = 2

def write_log(message):
    if sim_frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {sim_frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    ABx = bx - ax
    ABy = by - ay
    APx = px - ax
    APy = py - ay
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        return distance(px, py, ax, ay)
    t = (APx*ABx + APy*ABy) / AB_len_sq
    t = max(0, min(1, t))
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

# =============== Collisions using Uniform Grid ===============
def collision_broad_phase_grid(creatures):
    grid_map = {}
    for i, c in enumerate(creatures):
        if c is None:
            continue
        cell_x = int(c.x // GRID_CELL_SIZE)
        cell_y = int(c.y // GRID_CELL_SIZE)
        key = (cell_x, cell_y)
        grid_map.setdefault(key, []).append(i)
    return grid_map

def collision_narrow_phase(creatures, indices):
    for i in range(len(indices)):
        for j in range(i+1, len(indices)):
            idxA = indices[i]
            idxB = indices[j]
            c1 = creatures[idxA]
            c2 = creatures[idxB]
            if c1 is None or c2 is None:
                continue
            r1 = c1.radius()
            r2 = c2.radius()
            dx = c2.x - c1.x
            dy = c2.y - c1.y
            dist = math.hypot(dx, dy)
            min_dist = r1 + r2
            if dist < min_dist and dist > 0:
                overlap = 0.5 * (min_dist - dist)
                nx = dx / dist
                ny = dy / dist
                c1.x -= overlap * nx
                c1.y -= overlap * ny
                c2.x += overlap * nx
                c2.y += overlap * ny

def resolve_collisions(creatures):
    neighbor_offsets = [
        (0,0),(1,0),(0,1),(1,1),
        (-1,0),(0,-1),(-1,-1),(1,-1),(-1,1)
    ]
    for _ in range(COLLISION_PASSES):
        grid_map = collision_broad_phase_grid(creatures)
        for cell_key, idx_list in grid_map.items():
            collision_narrow_phase(creatures, idx_list)
            # check neighbors
            for (nx, ny) in neighbor_offsets:
                neigh_key = (cell_key[0]+nx, cell_key[1]+ny)
                if neigh_key in grid_map:
                    combined = idx_list + grid_map[neigh_key]
                    collision_narrow_phase(creatures, combined)

# =====================================================================
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    v0.5.3 changes:
     - narrower size mutation range
     - added idle cost
     - smaller max_size
     - ensure each victim is removed from the list quickly
    """
    def __init__(self, x, y, father_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        self.heading = random.random() * 2.0 * math.pi
        self.dead = False   ### NEW: mark if creature is "dead" to avoid double kills

        if father_dna:
            speed = np.clip(father_dna[0] * random.uniform(0.9, 1.1), MIN_SPEED, MAX_SPEED)
            r     = np.clip(father_dna[1] + random.randint(-15, 15), 50, 255)
            g     = np.clip(father_dna[2] + random.randint(-15, 15), 50, 255)
            b     = np.clip(father_dna[3] + random.randint(-15, 15), 50, 255)

            diet  = father_dna[4]
            fear  = father_dna[5]
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            # narrower size mutation => random.uniform(0.9..1.1)
            size_factor = father_dna[6]
            if random.random() < SIZE_MUTATION_CHANCE:
                size_factor *= random.uniform(0.9, 1.1)  ### CHANGED: narrower than 0.8..1.2
                size_factor = min(max(MIN_SIZE, size_factor), MAX_SIZE)

            longevity = father_dna[7]
            if random.random() < LONGEVITY_MUTATION_CHANCE:
                longevity *= random.uniform(0.8, 1.2)
                longevity = min(max(0.5, longevity), 2.0)

            self.dna = [speed, r, g, b, diet, fear, size_factor, longevity]

            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], MIN_SPEED, MAX_SPEED)
        else:
            self.dna = [
                random.uniform(1.5, 3.0),
                random.randint(50, 255),
                random.randint(50, 255),
                random.randint(50, 255),
                0,
                random.uniform(0.0, 0.2),
                random.uniform(MIN_SIZE, 1.2),
                random.uniform(0.8, 1.2)
            ]

        self.speed       = self.dna[0]
        self.color       = (self.dna[1], self.dna[2], self.dna[3])
        self.diet        = int(self.dna[4])
        self.fear        = self.dna[5]
        self.size_factor = self.dna[6]
        self.longevity   = self.dna[7]

    def _get_mutation_type(self, father_dna):
        if not father_dna:
            return "initial"
        changes = []
        if abs(self.dna[0] - father_dna[0]) > 1.0:
            changes.append("speed_shift")
        color_diff = abs(self.dna[1] - father_dna[1]) + abs(self.dna[2] - father_dna[2]) + abs(self.dna[3] - father_dna[3])
        if color_diff > 45:
            changes.append("color_shift")
        if self.dna[4] != father_dna[4]:
            changes.append("diet_flip")
        if abs(self.dna[5] - father_dna[5]) > 0.05:
            changes.append("fear_shift")
        if abs(self.dna[6] - father_dna[6]) > 0.3:
            changes.append("size_shift")
        if abs(self.dna[7] - father_dna[7]) > 0.3:
            changes.append("long_shift")
        if not changes:
            return "minor"
        return "+".join(changes)

    def max_age(self):
        return AGE_BASE * self.longevity

    def strength(self):
        return self.size_factor * (1.0 + 0.5*(self.energy/100.0))

    def max_energy_capacity(self):
        return 100 + (self.size_factor - 1.0)*50

    def radius(self):
        return 6 * self.size_factor

    def update(self, all_creatures, foods):
        if self.dead:
            return ("starved", None)  # won't matter, just a fallback
        self.age += 1

        # Add an idle cost:
        self.energy -= IDLE_COST  ### NEW

        # Fear cost:
        if self.diet == 0 and self.fear > FEAR_THRESHOLD_HERB:
            self.energy -= FEAR_COST_BASE * self.fear * self.size_factor
        elif self.diet == 1 and self.fear > PRED_FEAR_THRESHOLD:
            self.energy -= FEAR_COST_BASE * self.fear * (self.size_factor * 0.5)

        # movement cost
        self.energy -= BASE_ENERGY_LOSS_PER_MOVE
        self.energy -= (self.size_factor * SIZE_ENERGY_LOSS_FACTOR)
        self.energy = max(self.energy, 0)

        # AI
        if self.diet == 0:
            # herb
            if self.fear > FEAR_THRESHOLD_HERB:
                # find nearest predator
                predator, distp = self.find_nearest(all_creatures, lambda c: c is not self and c.diet == 1 and not c.dead)
                if predator and distp < 120:
                    self.run_away(predator.x, predator.y)
                    self.wrap_screen()
                    return self.check_survival()

            # find nearest food
            food_t, distf = self.find_nearest(foods)
            if food_t and distf < SIGHT_RANGE:
                if not self.is_line_blocked(food_t.x, food_t.y, all_creatures):
                    self.move_toward(food_t.x, food_t.y)
                    if distance(self.x, self.y, food_t.x, food_t.y) < (6 + self.size_factor*4):
                        self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_FOOD)
                        foods.remove(food_t)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        else:
            # predator
            ctarget, distc = self.find_nearest(all_creatures, lambda c: c is not self and not c.dead)
            if ctarget and distc < SIGHT_RANGE:
                if not self.is_line_blocked(ctarget.x, ctarget.y, all_creatures):
                    if ctarget.diet == 1:
                        # predator vs predator
                        if self.fear > PRED_FEAR_THRESHOLD and (ctarget.size_factor > self.size_factor*SIZE_DOMINANCE_RATIO):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                return self.predator_fight(ctarget)
                    else:
                        # predator vs herb
                        if (ctarget.size_factor>self.size_factor*HERB_SIZE_DOMINANCE_RATIO) and (self.fear> PRED_FEAR_THRESHOLD):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                gain = ENERGY_GAIN_FROM_PREY * ctarget.size_factor
                                self.energy = min(self.max_energy_capacity(), self.energy + gain)
                                # Mark victim "dead"
                                ctarget.dead = True  ### NEW: avoid double kills
                                return ("ate_creature", ctarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def predator_fight(self, other):
        if other.dead:
            return ("alive", None)   # skip if victim is already dead
        if random.random() < 0.3:
            my_str = self.strength() * AMBUSH_BONUS
            their_str = other.strength()
        else:
            my_str = self.strength()
            their_str = other.strength()
        if my_str > their_str:
            gain = ENERGY_GAIN_FROM_PREY * other.size_factor
            self.energy = min(self.max_energy_capacity(), self.energy + gain)
            other.dead = True  ### NEW
            return ("ate_creature", other)
        elif their_str > my_str:
            self.dead = True
            return ("ate_creature", self)
        else:
            if random.random() < 0.5:
                other.dead = True
                return ("ate_creature", other)
            else:
                self.dead = True
                return ("ate_creature", self)

    def check_survival(self):
        if self.dead:
            return ("starved", None)  # doesn't matter what we call it
        if self.energy <= 0:
            self.dead = True
            return ("starved", None)
        if self.age > self.max_age():
            self.dead = True
            return ("aged", None)
        return ("alive", None)

    def reproduce(self, creatures, foods):
        if self.dead:
            return None
        if self.energy < 80:
            return None
        pop_size = len(creatures)
        food_count = len(foods)
        ratio_food_pop = food_count/pop_size if pop_size>0 else 1
        ratio_pop_cap  = pop_size / MAX_CREATURES

        chance = ENV_REPRO_CHANCE_BASE
        chance += ENV_REPRO_FOOD_FACTOR*ratio_food_pop
        chance -= ENV_REPRO_POP_FACTOR*ratio_pop_cap
        chance = min(max(chance,0.0),1.0)

        if random.random() < chance:
            self.energy -= REPRODUCE_ENERGY_COST
            child_dna = list(self.dna)

            herb_count = sum(1 for c in creatures if c and c.diet==0 and not c.dead)
            pred_count = pop_size-herb_count
            flip_chance = BASE_DIET_FLIP_CHANCE

            if self.diet==0:
                # father=herb
                if food_count < (herb_count*0.7):
                    flip_chance += 0.05
                if (self.size_factor>1.5 and ratio_food_pop<0.5):
                    flip_chance += 0.05
                if random.random()<flip_chance:
                    child_dna[4] = 1
            else:
                # father=pred
                if pred_count>(herb_count*0.75):
                    flip_chance += 0.05
                if random.random()<flip_chance:
                    child_dna[4]=0

            offx = random.gauss(0, 10)
            offy = random.gauss(0, 10)
            baby = Creature(self.x+offx, self.y+offy, father_dna=child_dna)
            return baby
        return None

    def random_walk(self):
        self.heading += random.uniform(-0.2, 0.2)
        dx = math.cos(self.heading)*self.speed
        dy = math.sin(self.heading)*self.speed
        self.x+=dx
        self.y+=dy

    def run_away(self,ox,oy):
        dx=self.x-ox
        dy=self.y-oy
        distv=distance(self.x,self.y,ox,oy)
        if distv>1:
            self.x+=(dx/distv)*self.speed
            self.y+=(dy/distv)*self.speed
        else:
            self.random_walk()

    def move_toward(self,tx,ty):
        dx=tx-self.x
        dy=ty-self.y
        distv=distance(self.x,self.y,tx,ty)
        if distv>1:
            self.x+=(dx/distv)*self.speed
            self.y+=(dy/distv)*self.speed
        else:
            self.random_walk()

    def wrap_screen(self):
        self.x%=1200
        self.y%=800

    def find_nearest(self, target_list, condition=None):
        valids=[t for t in target_list if(t and(t is not self))]
        if condition:
            valids=[v for v in valids if condition(v)]
        if not valids:
            return(None,float('inf'))
        best_obj,best_dist=None,float('inf')
        for obj in valids:
            d=distance(self.x,self.y,obj.x,obj.y)
            if d<best_dist:
                best_dist=d
                best_obj=obj
        return(best_obj,best_dist)

    def is_line_blocked(self,tx,ty,creatures):
        targ_dist=distance(self.x,self.y,tx,ty)
        for c in creatures:
            if not c or c is self or c.dead:
                continue
            dist_c=distance(self.x,self.y,c.x,c.y)
            if dist_c>=targ_dist:
                continue
            d_line=line_distance_point(c.x,c.y,self.x,self.y,tx,ty)
            if d_line<BLOCKING_RADIUS:
                return True
        return False

    def draw(self, surface):
        if self.dead:
            return
        sz = int(8*self.size_factor)
        if self.diet == 0:
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), sz)
        else:
            # Calculate points for cursor shape based on heading
            tip_x = self.x + math.cos(self.heading) * sz * 1.5
            tip_y = self.y + math.sin(self.heading) * sz * 1.5
            
            # Calculate base points perpendicular to heading
            base_x = self.x - math.cos(self.heading) * sz
            base_y = self.y - math.sin(self.heading) * sz
            
            perp_x = math.cos(self.heading + math.pi/2) * sz * 0.6
            perp_y = math.sin(self.heading + math.pi/2) * sz * 0.6
            
            points = [
                (tip_x, tip_y),  # Front tip
                (base_x + perp_x, base_y + perp_y),  # Right base
                (base_x - perp_x, base_y - perp_y)   # Left base
            ]
            pygame.draw.polygon(surface, self.color, points)

# ==================== SETUP =========================
open(LOG_FILE,"w").close()

creatures=[]
for _ in range(INITIAL_HERBIVORES):
    c=Creature(random.randint(100,1100),random.randint(100,700))
    c.dna[4]=0
    c.diet=0
    creatures.append(c)
for _ in range(INITIAL_PREDATORS):
    c=Creature(random.randint(100,1100),random.randint(100,700))
    c.dna[4]=1
    c.diet=1
    creatures.append(c)

foods=[Food() for _ in range(INITIAL_FOOD)]

paused=False
frames=0
sim_frames=0
population_history=deque(maxlen=300)
pred_ratio_history=deque(maxlen=300)

death_log={'starved':0,'aged':0,'hunted':0}
mutation_tracker=Counter()

day_cycle_frame=0
day_count=0
fps_sample_timer=0.0
avg_frame_time=0.0

running=True

while running:
    dt=clock.tick(30)
    frames+=1

    fps_sample_timer+=dt
    avg_frame_time=0.9*avg_frame_time+0.1*dt

    for event in pygame.event.get():
        if event.type==pygame.QUIT:
            running=False
        elif event.type==pygame.KEYDOWN:
            if event.key==pygame.K_SPACE:
                paused=not paused

    if not paused:
        sim_frames+=1
        day_cycle_frame=(day_cycle_frame+1)%DAY_LENGTH
        if day_cycle_frame==0:
            day_count+=1

    alpha=day_cycle_frame/(DAY_LENGTH-1)if(DAY_LENGTH>1)else 1.0
    if alpha<0.5:
        sub_a=alpha/0.5
        bg_color=interpolate_color(NIGHT_COLOR,DAY_COLOR,sub_a)
        is_day=True
    else:
        sub_a=(alpha-0.5)/0.5
        bg_color=interpolate_color(DAY_COLOR,NIGHT_COLOR,sub_a)
        is_day=False

    screen.fill(bg_color)

    if not paused:
        new_creatures=[]
        dead=[]

        for c in creatures:
            if c and not c.dead:
                status,other=c.update(creatures,foods)
                if status=="starved":
                    death_log["starved"]+=1
                    write_log(f"Death: starved (speed={c.speed:.2f})")
                    dead.append(c)
                    # multiple food from corpse
                    food_chunks=1+int(c.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
                elif status=="aged":
                    death_log["aged"]+=1
                    write_log(f"Death: aged (speed={c.speed:.2f})")
                    dead.append(c)
                    food_chunks=1+int(c.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
                elif status=="ate_creature":
                    death_log["hunted"]+=1
                    # if c= eater, other= meal
                    # if c= meal, other= eater
                    if other is c:
                        write_log(f"Death: hunted (speed={c.speed:.2f}) [lost predator fight]")
                        dead.append(c)
                        food_chunks=1+int(c.size_factor*2)
                        for _ in range(food_chunks):
                            if random.random()<0.3:
                                foods.append(Food())
                    else:
                        write_log(f"Death: hunted (speed={other.speed:.2f})")
                        dead.append(other)
                        food_chunks=1+int(other.size_factor*2)
                        for _ in range(food_chunks):
                            if random.random()<0.3:
                                foods.append(Food())
                # else "alive" => can reproduce
                else:
                    child=c.reproduce(creatures,foods)
                    if child and len(creatures)<MAX_CREATURES:
                        mut_type=child._get_mutation_type(c.dna)
                        if mut_type not in("initial","minor"):
                            mutation_tracker[mut_type]+=1
                        new_creatures.append(child)
                        write_log(
                            f"Birth: speed={child.speed:.2f}, diet={child.diet}, size={child.size_factor:.2f}, "
                            f"fear={child.fear:.2f}, longevity={child.longevity:.2f}, mutation={mut_type}"
                        )

        # remove dead
        for dcreat in dead:
            if dcreat in creatures:
                idx=creatures.index(dcreat)
                creatures[idx]=None   ### set to None, fully remove later

        # add newborns
        # remove None placeholders
        creatures=[cr for cr in creatures if cr is not None and not cr.dead]
        space_left=max(0,MAX_CREATURES-len(creatures))
        creatures+=new_creatures[:space_left]

        # collisions with uniform grid
        resolve_collisions(creatures)

        pop_size=len(creatures)
        pred_count=sum(1 for cr in creatures if cr.diet==1 and not cr.dead)
        population_history.append(pop_size)
        ratio=pred_count/pop_size if pop_size>0 else 0
        pred_ratio_history.append(ratio)

        if is_day:
            if random.random()<DAYTIME_FOOD_SPAWN_CHANCE:
                foods.append(Food())

    for f in foods:
        pygame.draw.circle(screen,FOOD_COLOR,(int(f.x),int(f.y)),4)
    for cr in creatures:
        if cr and not cr.dead:
            cr.draw(screen)

    pred_count=sum(1 for x in creatures if x and x.diet==1 and not x.dead)
    herb_count=len(creatures)-pred_count
    ratio_str="∞"
    if pred_count>0:
        ratio_val=herb_count/pred_count
        ratio_str=f"{ratio_val:.1f}"

    stats=[
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Predators: {pred_count}, ratio=1:{ratio_str}",
        f"Food: {len(foods)}",
        f"Day Count: {day_count}",
        f"Sim Time: {sim_frames}",
    ]
    if creatures:
        avg_speed=np.mean([c.speed for c in creatures if c and not c.dead])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']} aged:{death_log['aged']} hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    for i,text in enumerate(stats):
        txt=font.render(text,True,(255,255,255))
        screen.blit(txt,(10,10+i*25))

    mut_top3=mutation_tracker.most_common(3)
    if mut_top3:
        mut_strs=[f"{mt}:{cnt}"for(mt,cnt)in mut_top3]
        mut_line="Mutations: "+", ".join(mut_strs)
        txtm=font.render(mut_line,True,(255,255,255))
        screen.blit(txtm,(10,10+len(stats)*25))

    fps_txt=f"FPS: {int(clock.get_fps())}"
    ms_txt=f"FrameTime: {avg_frame_time:.1f}ms"
    fps_surf=font.render(fps_txt,True,(255,255,255))
    ms_surf=font.render(ms_txt,True,(255,255,255))
    screen.blit(fps_surf,(1050,10))
    screen.blit(ms_surf,(1050,35))

    graph_x,graph_y=800,70
    graph_w,graph_h=380,90
    pygame.draw.rect(screen,(50,50,50),(graph_x,graph_y,graph_w,graph_h))

    if len(population_history)>1:
        points_pop=[]
        for i,val in enumerate(population_history):
            xx=graph_x+graph_w-(len(population_history)-i)*2
            yy=graph_y+graph_h-((val/MAX_CREATURES)*graph_h)
            points_pop.append((xx,yy))
        pygame.draw.lines(screen,GRAPH_COLOR,False,points_pop,2)

    if len(pred_ratio_history)>1:
        points_pred=[]
        for i,ratio_val in enumerate(pred_ratio_history):
            xx=graph_x+graph_w-(len(pred_ratio_history)-i)*2
            yy=graph_y+graph_h-(ratio_val*graph_h)
            points_pred.append((xx,yy))
        pygame.draw.lines(screen,PREDATOR_GRAPH_COLOR,False,points_pred,2)

    pygame.display.flip()

pygame.quit()


In [ ]:
"""
emergent_sim_v0.5.3 (~700 lines)
Major changes from v0.5.2:
1) Added an idle cost + higher size-based cost.
2) Lowered ENERGY_GAIN_FROM_PREY/FOOD.
3) Narrowed size mutation range & smaller max size = 2.0.
4) Hunting overcount fix: once a predator kills a victim, we mark victim "dead" so it can't be double-counted.
5) Slower day/night => from 240 frames to 600 frames.
6) Smaller cell size for collision from 40 to 25.
"""

import pygame
import numpy as np
import random
from collections import deque, Counter
from datetime import datetime
import math

# =================== CONFIG ====================
MAX_CREATURES = 1000

INITIAL_HERBIVORES = 180
INITIAL_PREDATORS  = 20

INITIAL_FOOD = 40

AGE_BASE = 700

ENERGY_GAIN_FROM_PREY = 60       ### CHANGED (was 80)
ENERGY_GAIN_FROM_FOOD = 15       ### CHANGED (was 22)

BASE_ENERGY_LOSS_PER_MOVE = 0.04
SIZE_ENERGY_LOSS_FACTOR   = 0.08 ### CHANGED (was 0.04)

REPRODUCE_ENERGY_COST = 50
ENV_REPRO_CHANCE_BASE = 0.3
ENV_REPRO_FOOD_FACTOR = 0.2
ENV_REPRO_POP_FACTOR  = 0.2

MUTATION_RATE = 0.1
BASE_DIET_FLIP_CHANCE = 0.005

FEAR_MUTATION_CHANCE = 0.2
SIZE_MUTATION_CHANCE = 0.2
LONGEVITY_MUTATION_CHANCE = 0.2

MAX_SPEED = 8.0
MIN_SPEED = 0.5

MIN_SIZE  = 0.5
MAX_SIZE  = 2.0   ### CHANGED (was 2.5)

# We'll add an idle cost each frame:
IDLE_COST = 0.01  ### NEW

FEAR_COST_BASE = 0.02
FEAR_THRESHOLD_HERB = 0.3

AMBUSH_BONUS = 1.1  
PRED_FEAR_THRESHOLD = 0.5   
SIZE_DOMINANCE_RATIO = 1.2  
HERB_SIZE_DOMINANCE_RATIO = 1.2

SIGHT_RANGE = 200.0
BLOCKING_RADIUS = 5.0

DAY_LENGTH = 600        ### CHANGED (was 240 => slower day/night)
DAY_COLOR   = (170, 220, 255)
NIGHT_COLOR = (0, 0, 0)

DAYTIME_FOOD_SPAWN_CHANCE = 0.02

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)
PREDATOR_GRAPH_COLOR = (150, 100, 200)

# Grid settings for collision broad-phase
GRID_CELL_SIZE = 25   ### CHANGED (was 40 => smaller => more accurate)
COLLISION_PASSES = 2

def write_log(message):
    if sim_frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {sim_frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    ABx = bx - ax
    ABy = by - ay
    APx = px - ax
    APy = py - ay
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        return distance(px, py, ax, ay)
    t = (APx*ABx + APy*ABy) / AB_len_sq
    t = max(0, min(1, t))
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

# =============== Collisions using Uniform Grid ===============
def collision_broad_phase_grid(creatures):
    grid_map = {}
    for i, c in enumerate(creatures):
        if c is None:
            continue
        cell_x = int(c.x // GRID_CELL_SIZE)
        cell_y = int(c.y // GRID_CELL_SIZE)
        key = (cell_x, cell_y)
        grid_map.setdefault(key, []).append(i)
    return grid_map

def collision_narrow_phase(creatures, indices):
    for i in range(len(indices)):
        for j in range(i+1, len(indices)):
            idxA = indices[i]
            idxB = indices[j]
            c1 = creatures[idxA]
            c2 = creatures[idxB]
            if c1 is None or c2 is None:
                continue
            r1 = c1.radius()
            r2 = c2.radius()
            dx = c2.x - c1.x
            dy = c2.y - c1.y
            dist = math.hypot(dx, dy)
            min_dist = r1 + r2
            if dist < min_dist and dist > 0:
                overlap = 0.5 * (min_dist - dist)
                nx = dx / dist
                ny = dy / dist
                c1.x -= overlap * nx
                c1.y -= overlap * ny
                c2.x += overlap * nx
                c2.y += overlap * ny

def resolve_collisions(creatures):
    neighbor_offsets = [
        (0,0),(1,0),(0,1),(1,1),
        (-1,0),(0,-1),(-1,-1),(1,-1),(-1,1)
    ]
    for _ in range(COLLISION_PASSES):
        grid_map = collision_broad_phase_grid(creatures)
        for cell_key, idx_list in grid_map.items():
            collision_narrow_phase(creatures, idx_list)
            # check neighbors
            for (nx, ny) in neighbor_offsets:
                neigh_key = (cell_key[0]+nx, cell_key[1]+ny)
                if neigh_key in grid_map:
                    combined = idx_list + grid_map[neigh_key]
                    collision_narrow_phase(creatures, combined)

# =====================================================================
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    v0.5.3 changes:
     - narrower size mutation range
     - added idle cost
     - smaller max_size
     - ensure each victim is removed from the list quickly
    """
    def __init__(self, x, y, father_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        self.heading = random.random() * 2.0 * math.pi
        self.dead = False   ### NEW: mark if creature is "dead" to avoid double kills

        if father_dna:
            speed = np.clip(father_dna[0] * random.uniform(0.9, 1.1), MIN_SPEED, MAX_SPEED)
            r     = np.clip(father_dna[1] + random.randint(-15, 15), 50, 255)
            g     = np.clip(father_dna[2] + random.randint(-15, 15), 50, 255)
            b     = np.clip(father_dna[3] + random.randint(-15, 15), 50, 255)

            diet  = father_dna[4]
            fear  = father_dna[5]
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            # narrower size mutation => random.uniform(0.9..1.1)
            size_factor = father_dna[6]
            if random.random() < SIZE_MUTATION_CHANCE:
                size_factor *= random.uniform(0.9, 1.1)  ### CHANGED: narrower than 0.8..1.2
                size_factor = min(max(MIN_SIZE, size_factor), MAX_SIZE)

            longevity = father_dna[7]
            if random.random() < LONGEVITY_MUTATION_CHANCE:
                longevity *= random.uniform(0.8, 1.2)
                longevity = min(max(0.5, longevity), 2.0)

            self.dna = [speed, r, g, b, diet, fear, size_factor, longevity]

            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], MIN_SPEED, MAX_SPEED)
        else:
            self.dna = [
                random.uniform(1.5, 3.0),
                random.randint(50, 255),
                random.randint(50, 255),
                random.randint(50, 255),
                0,
                random.uniform(0.0, 0.2),
                random.uniform(MIN_SIZE, 1.2),
                random.uniform(0.8, 1.2)
            ]

        self.speed       = self.dna[0]
        self.color       = (self.dna[1], self.dna[2], self.dna[3])
        self.diet        = int(self.dna[4])
        self.fear        = self.dna[5]
        self.size_factor = self.dna[6]
        self.longevity   = self.dna[7]

    def _get_mutation_type(self, father_dna):
        if not father_dna:
            return "initial"
        changes = []
        if abs(self.dna[0] - father_dna[0]) > 1.0:
            changes.append("speed_shift")
        color_diff = abs(self.dna[1] - father_dna[1]) + abs(self.dna[2] - father_dna[2]) + abs(self.dna[3] - father_dna[3])
        if color_diff > 45:
            changes.append("color_shift")
        if self.dna[4] != father_dna[4]:
            changes.append("diet_flip")
        if abs(self.dna[5] - father_dna[5]) > 0.05:
            changes.append("fear_shift")
        if abs(self.dna[6] - father_dna[6]) > 0.3:
            changes.append("size_shift")
        if abs(self.dna[7] - father_dna[7]) > 0.3:
            changes.append("long_shift")
        if not changes:
            return "minor"
        return "+".join(changes)

    def max_age(self):
        return AGE_BASE * self.longevity

    def strength(self):
        return self.size_factor * (1.0 + 0.5*(self.energy/100.0))

    def max_energy_capacity(self):
        return 100 + (self.size_factor - 1.0)*50

    def radius(self):
        return 6 * self.size_factor

    def update(self, all_creatures, foods):
        if self.dead:
            return ("starved", None)  # won't matter, just a fallback
        self.age += 1

        # Add an idle cost:
        self.energy -= IDLE_COST  ### NEW

        # Fear cost:
        if self.diet == 0 and self.fear > FEAR_THRESHOLD_HERB:
            self.energy -= FEAR_COST_BASE * self.fear * self.size_factor
        elif self.diet == 1 and self.fear > PRED_FEAR_THRESHOLD:
            self.energy -= FEAR_COST_BASE * self.fear * (self.size_factor * 0.5)

        # movement cost
        self.energy -= BASE_ENERGY_LOSS_PER_MOVE
        self.energy -= (self.size_factor * SIZE_ENERGY_LOSS_FACTOR)
        self.energy = max(self.energy, 0)

        # AI
        if self.diet == 0:
            # herb
            if self.fear > FEAR_THRESHOLD_HERB:
                # find nearest predator
                predator, distp = self.find_nearest(all_creatures, lambda c: c is not self and c.diet == 1 and not c.dead)
                if predator and distp < 120:
                    self.run_away(predator.x, predator.y)
                    self.wrap_screen()
                    return self.check_survival()

            # find nearest food
            food_t, distf = self.find_nearest(foods)
            if food_t and distf < SIGHT_RANGE:
                if not self.is_line_blocked(food_t.x, food_t.y, all_creatures):
                    self.move_toward(food_t.x, food_t.y)
                    if distance(self.x, self.y, food_t.x, food_t.y) < (6 + self.size_factor*4):
                        self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_FOOD)
                        foods.remove(food_t)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        else:
            # predator
            ctarget, distc = self.find_nearest(all_creatures, lambda c: c is not self and not c.dead)
            if ctarget and distc < SIGHT_RANGE:
                if not self.is_line_blocked(ctarget.x, ctarget.y, all_creatures):
                    if ctarget.diet == 1:
                        # predator vs predator
                        if self.fear > PRED_FEAR_THRESHOLD and (ctarget.size_factor > self.size_factor*SIZE_DOMINANCE_RATIO):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                return self.predator_fight(ctarget)
                    else:
                        # predator vs herb
                        if (ctarget.size_factor>self.size_factor*HERB_SIZE_DOMINANCE_RATIO) and (self.fear> PRED_FEAR_THRESHOLD):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                gain = ENERGY_GAIN_FROM_PREY * ctarget.size_factor
                                self.energy = min(self.max_energy_capacity(), self.energy + gain)
                                # Mark victim "dead"
                                ctarget.dead = True  ### NEW: avoid double kills
                                return ("ate_creature", ctarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def predator_fight(self, other):
        if other.dead:
            return ("alive", None)   # skip if victim is already dead
        if random.random() < 0.3:
            my_str = self.strength() * AMBUSH_BONUS
            their_str = other.strength()
        else:
            my_str = self.strength()
            their_str = other.strength()
        if my_str > their_str:
            gain = ENERGY_GAIN_FROM_PREY * other.size_factor
            self.energy = min(self.max_energy_capacity(), self.energy + gain)
            other.dead = True  ### NEW
            return ("ate_creature", other)
        elif their_str > my_str:
            self.dead = True
            return ("ate_creature", self)
        else:
            if random.random() < 0.5:
                other.dead = True
                return ("ate_creature", other)
            else:
                self.dead = True
                return ("ate_creature", self)

    def check_survival(self):
        if self.dead:
            return ("starved", None)  # doesn't matter what we call it
        if self.energy <= 0:
            self.dead = True
            return ("starved", None)
        if self.age > self.max_age():
            self.dead = True
            return ("aged", None)
        return ("alive", None)

    def reproduce(self, creatures, foods):
        if self.dead:
            return None
        if self.energy < 80:
            return None
        pop_size = len(creatures)
        food_count = len(foods)
        ratio_food_pop = food_count/pop_size if pop_size>0 else 1
        ratio_pop_cap  = pop_size / MAX_CREATURES

        chance = ENV_REPRO_CHANCE_BASE
        chance += ENV_REPRO_FOOD_FACTOR*ratio_food_pop
        chance -= ENV_REPRO_POP_FACTOR*ratio_pop_cap
        chance = min(max(chance,0.0),1.0)

        if random.random() < chance:
            self.energy -= REPRODUCE_ENERGY_COST
            child_dna = list(self.dna)

            herb_count = sum(1 for c in creatures if c and c.diet==0 and not c.dead)
            pred_count = pop_size-herb_count
            flip_chance = BASE_DIET_FLIP_CHANCE

            if self.diet==0:
                # father=herb
                if food_count < (herb_count*0.7):
                    flip_chance += 0.05
                if (self.size_factor>1.5 and ratio_food_pop<0.5):
                    flip_chance += 0.05
                if random.random()<flip_chance:
                    child_dna[4] = 1
            else:
                # father=pred
                if pred_count>(herb_count*0.75):
                    flip_chance += 0.05
                if random.random()<flip_chance:
                    child_dna[4]=0

            offx = random.gauss(0, 10)
            offy = random.gauss(0, 10)
            baby = Creature(self.x+offx, self.y+offy, father_dna=child_dna)
            return baby
        return None

    def random_walk(self):
        self.heading += random.uniform(-0.2, 0.2)
        dx = math.cos(self.heading)*self.speed
        dy = math.sin(self.heading)*self.speed
        self.x+=dx
        self.y+=dy

    def run_away(self,ox,oy):
        dx=self.x-ox
        dy=self.y-oy
        distv=distance(self.x,self.y,ox,oy)
        if distv>1:
            self.x+=(dx/distv)*self.speed
            self.y+=(dy/distv)*self.speed
        else:
            self.random_walk()

    def move_toward(self,tx,ty):
        dx=tx-self.x
        dy=ty-self.y
        distv=distance(self.x,self.y,tx,ty)
        if distv>1:
            self.x+=(dx/distv)*self.speed
            self.y+=(dy/distv)*self.speed
        else:
            self.random_walk()

    def wrap_screen(self):
        self.x%=1200
        self.y%=800

    def find_nearest(self, target_list, condition=None):
        valids=[t for t in target_list if(t and(t is not self))]
        if condition:
            valids=[v for v in valids if condition(v)]
        if not valids:
            return(None,float('inf'))
        best_obj,best_dist=None,float('inf')
        for obj in valids:
            d=distance(self.x,self.y,obj.x,obj.y)
            if d<best_dist:
                best_dist=d
                best_obj=obj
        return(best_obj,best_dist)

    def is_line_blocked(self,tx,ty,creatures):
        targ_dist=distance(self.x,self.y,tx,ty)
        for c in creatures:
            if not c or c is self or c.dead:
                continue
            dist_c=distance(self.x,self.y,c.x,c.y)
            if dist_c>=targ_dist:
                continue
            d_line=line_distance_point(c.x,c.y,self.x,self.y,tx,ty)
            if d_line<BLOCKING_RADIUS:
                return True
        return False

    def draw(self, surface):
        if self.dead:
            return
        sz = int(8*self.size_factor)
        if self.diet==0:
            pygame.draw.circle(surface,self.color,(int(self.x),int(self.y)),sz)
        else:
            half_w=sz
            points=[
                (self.x,self.y-sz),
                (self.x-half_w,self.y+sz),
                (self.x+half_w,self.y+sz)
            ]
            pygame.draw.polygon(surface,self.color,points)

# ==================== SETUP =========================
open(LOG_FILE,"w").close()

creatures=[]
for _ in range(INITIAL_HERBIVORES):
    c=Creature(random.randint(100,1100),random.randint(100,700))
    c.dna[4]=0
    c.diet=0
    creatures.append(c)
for _ in range(INITIAL_PREDATORS):
    c=Creature(random.randint(100,1100),random.randint(100,700))
    c.dna[4]=1
    c.diet=1
    creatures.append(c)

foods=[Food() for _ in range(INITIAL_FOOD)]

paused=False
frames=0
sim_frames=0
population_history=deque(maxlen=300)
pred_ratio_history=deque(maxlen=300)

death_log={'starved':0,'aged':0,'hunted':0}
mutation_tracker=Counter()

day_cycle_frame=0
day_count=0
fps_sample_timer=0.0
avg_frame_time=0.0

running=True

while running:
    dt=clock.tick(30)
    frames+=1

    fps_sample_timer+=dt
    avg_frame_time=0.9*avg_frame_time+0.1*dt

    for event in pygame.event.get():
        if event.type==pygame.QUIT:
            running=False
        elif event.type==pygame.KEYDOWN:
            if event.key==pygame.K_SPACE:
                paused=not paused

    if not paused:
        sim_frames+=1
        day_cycle_frame=(day_cycle_frame+1)%DAY_LENGTH
        if day_cycle_frame==0:
            day_count+=1

    alpha=day_cycle_frame/(DAY_LENGTH-1)if(DAY_LENGTH>1)else 1.0
    if alpha<0.5:
        sub_a=alpha/0.5
        bg_color=interpolate_color(NIGHT_COLOR,DAY_COLOR,sub_a)
        is_day=True
    else:
        sub_a=(alpha-0.5)/0.5
        bg_color=interpolate_color(DAY_COLOR,NIGHT_COLOR,sub_a)
        is_day=False

    screen.fill(bg_color)

    if not paused:
        new_creatures=[]
        dead=[]

        for c in creatures:
            if c and not c.dead:
                status,other=c.update(creatures,foods)
                if status=="starved":
                    death_log["starved"]+=1
                    write_log(f"Death: starved (speed={c.speed:.2f})")
                    dead.append(c)
                    # multiple food from corpse
                    food_chunks=1+int(c.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
                elif status=="aged":
                    death_log["aged"]+=1
                    write_log(f"Death: aged (speed={c.speed:.2f})")
                    dead.append(c)
                    food_chunks=1+int(c.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
                elif status=="ate_creature":
                    death_log["hunted"]+=1
                    # if c= eater, other= meal
                    # if c= meal, other= eater
                    if other is c:
                        write_log(f"Death: hunted (speed={c.speed:.2f}) [lost predator fight]")
                        dead.append(c)
                        food_chunks=1+int(c.size_factor*2)
                        for _ in range(food_chunks):
                            if random.random()<0.3:
                                foods.append(Food())
                    else:
                        write_log(f"Death: hunted (speed={other.speed:.2f})")
                        dead.append(other)
                        food_chunks=1+int(other.size_factor*2)
                        for _ in range(food_chunks):
                            if random.random()<0.3:
                                foods.append(Food())
                # else "alive" => can reproduce
                else:
                    child=c.reproduce(creatures,foods)
                    if child and len(creatures)<MAX_CREATURES:
                        mut_type=child._get_mutation_type(c.dna)
                        if mut_type not in("initial","minor"):
                            mutation_tracker[mut_type]+=1
                        new_creatures.append(child)
                        write_log(
                            f"Birth: speed={child.speed:.2f}, diet={child.diet}, size={child.size_factor:.2f}, "
                            f"fear={child.fear:.2f}, longevity={child.longevity:.2f}, mutation={mut_type}"
                        )

        # remove dead
        for dcreat in dead:
            if dcreat in creatures:
                idx=creatures.index(dcreat)
                creatures[idx]=None   ### set to None, fully remove later

        # add newborns
        # remove None placeholders
        creatures=[cr for cr in creatures if cr is not None and not cr.dead]
        space_left=max(0,MAX_CREATURES-len(creatures))
        creatures+=new_creatures[:space_left]

        # collisions with uniform grid
        resolve_collisions(creatures)

        pop_size=len(creatures)
        pred_count=sum(1 for cr in creatures if cr.diet==1 and not cr.dead)
        population_history.append(pop_size)
        ratio=pred_count/pop_size if pop_size>0 else 0
        pred_ratio_history.append(ratio)

        if is_day:
            if random.random()<DAYTIME_FOOD_SPAWN_CHANCE:
                foods.append(Food())

    for f in foods:
        pygame.draw.circle(screen,FOOD_COLOR,(int(f.x),int(f.y)),4)
    for cr in creatures:
        if cr and not cr.dead:
            cr.draw(screen)

    pred_count=sum(1 for x in creatures if x and x.diet==1 and not x.dead)
    herb_count=len(creatures)-pred_count
    ratio_str="∞"
    if pred_count>0:
        ratio_val=herb_count/pred_count
        ratio_str=f"{ratio_val:.1f}"

    stats=[
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Predators: {pred_count}, ratio=1:{ratio_str}",
        f"Food: {len(foods)}",
        f"Day Count: {day_count}",
        f"Sim Time: {sim_frames}",
    ]
    if creatures:
        avg_speed=np.mean([c.speed for c in creatures if c and not c.dead])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']} aged:{death_log['aged']} hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    for i,text in enumerate(stats):
        txt=font.render(text,True,(255,255,255))
        screen.blit(txt,(10,10+i*25))

    mut_top3=mutation_tracker.most_common(3)
    if mut_top3:
        mut_strs=[f"{mt}:{cnt}"for(mt,cnt)in mut_top3]
        mut_line="Mutations: "+", ".join(mut_strs)
        txtm=font.render(mut_line,True,(255,255,255))
        screen.blit(txtm,(10,10+len(stats)*25))

    fps_txt=f"FPS: {int(clock.get_fps())}"
    ms_txt=f"FrameTime: {avg_frame_time:.1f}ms"
    fps_surf=font.render(fps_txt,True,(255,255,255))
    ms_surf=font.render(ms_txt,True,(255,255,255))
    screen.blit(fps_surf,(1050,10))
    screen.blit(ms_surf,(1050,35))

    graph_x,graph_y=800,70
    graph_w,graph_h=380,90
    pygame.draw.rect(screen,(50,50,50),(graph_x,graph_y,graph_w,graph_h))

    if len(population_history)>1:
        points_pop=[]
        for i,val in enumerate(population_history):
            xx=graph_x+graph_w-(len(population_history)-i)*2
            yy=graph_y+graph_h-((val/MAX_CREATURES)*graph_h)
            points_pop.append((xx,yy))
        pygame.draw.lines(screen,GRAPH_COLOR,False,points_pop,2)

    if len(pred_ratio_history)>1:
        points_pred=[]
        for i,ratio_val in enumerate(pred_ratio_history):
            xx=graph_x+graph_w-(len(pred_ratio_history)-i)*2
            yy=graph_y+graph_h-(ratio_val*graph_h)
            points_pred.append((xx,yy))
        pygame.draw.lines(screen,PREDATOR_GRAPH_COLOR,False,points_pred,2)

    pygame.display.flip()

pygame.quit()


In [ ]:
"""
emergent_sim_v0.5.2 -- (~700 lines total)
Summary of changes from v0.5.1:
 - MAX_CREATURES = 1000 (down from 2000)
 - DAYTIME_FOOD_SPAWN_CHANCE = 0.02 (down from 0.05)
 - BASE_DIET_FLIP_CHANCE = 0.005 (down from 0.02)
 - Use a uniform grid to accelerate collision detection (instead of O(N^2)).
   => Significantly better performance for large populations.
 - Perform multiple collision "passes" each frame so that repeated overlaps
   in the same frame get resolved more thoroughly.
"""

import pygame
import numpy as np
import random
from collections import deque, Counter
from datetime import datetime
import math

# =================== CONFIG ====================
MAX_CREATURES = 1000      # (1) changed from 2000
INITIAL_HERBIVORES = 180
INITIAL_PREDATORS  = 20

INITIAL_FOOD = 40
AGE_BASE = 700

ENERGY_GAIN_FROM_PREY = 80  
ENERGY_GAIN_FROM_FOOD = 22

BASE_ENERGY_LOSS_PER_MOVE = 0.04
SIZE_ENERGY_LOSS_FACTOR   = 0.04

REPRODUCE_ENERGY_COST = 50
ENV_REPRO_CHANCE_BASE = 0.3
ENV_REPRO_FOOD_FACTOR = 0.2
ENV_REPRO_POP_FACTOR  = 0.2

MUTATION_RATE = 0.1
BASE_DIET_FLIP_CHANCE = 0.005   # (3) changed from 0.02

FEAR_MUTATION_CHANCE = 0.2
SIZE_MUTATION_CHANCE = 0.2
LONGEVITY_MUTATION_CHANCE = 0.2

MAX_SPEED = 8.0
MIN_SPEED = 0.5
MIN_SIZE  = 0.5
MAX_SIZE  = 2.5

FEAR_COST_BASE = 0.02
FEAR_THRESHOLD_HERB = 0.3

AMBUSH_BONUS = 1.1  
PRED_FEAR_THRESHOLD = 0.5   
SIZE_DOMINANCE_RATIO = 1.2  
HERB_SIZE_DOMINANCE_RATIO = 1.2

SIGHT_RANGE = 200.0
BLOCKING_RADIUS = 5.0

DAY_LENGTH = 240
DAY_COLOR   = (170, 220, 255)
NIGHT_COLOR = (0, 0, 0)

DAYTIME_FOOD_SPAWN_CHANCE = 0.02  # (2) changed from 0.05

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Colors
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)
PREDATOR_GRAPH_COLOR = (150, 100, 200)

# Grid settings for collision broad-phase
GRID_CELL_SIZE = 40   # reduce if you see many collisions in the same cell
COLLISION_PASSES = 2  # do multiple passes to reduce shape overlap

def write_log(message):
    if sim_frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {sim_frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    ABx = bx - ax
    ABy = by - ay
    APx = px - ax
    APy = py - ay
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        return distance(px, py, ax, ay)
    t = (APx*ABx + APy*ABy) / AB_len_sq
    t = max(0, min(1, t))
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

def collision_broad_phase_grid(creatures):
    """
    Build a uniform grid of size GRID_CELL_SIZE.
    Return a dict: grid[cell] = list of creature indices
    Then we only do narrow-phase collisions inside each cell (and maybe neighbors).
    """
    grid_map = {}
    cell_size = GRID_CELL_SIZE
    for i, c in enumerate(creatures):
        cell_x = int(c.x // cell_size)
        cell_y = int(c.y // cell_size)
        key = (cell_x, cell_y)
        grid_map.setdefault(key, []).append(i)
    return grid_map

def collision_narrow_phase(creatures, indices):
    """
    Among these creature indices, check collisions pairwise.
    """
    for i in range(len(indices)):
        for j in range(i+1, len(indices)):
            c1 = creatures[indices[i]]
            c2 = creatures[indices[j]]
            r1 = c1.radius()
            r2 = c2.radius()
            dx = c2.x - c1.x
            dy = c2.y - c1.y
            dist = math.hypot(dx, dy)
            min_dist = r1 + r2
            if dist < min_dist and dist > 0:
                overlap = 0.5 * (min_dist - dist)
                nx = dx / dist
                ny = dy / dist
                c1.x -= overlap * nx
                c1.y -= overlap * ny
                c2.x += overlap * nx
                c2.y += overlap * ny

def resolve_collisions(creatures):
    """
    We'll do multiple passes. On each pass:
      1) Build grid
      2) For each cell, do narrow-phase collisions
      3) Possibly also check immediate neighbor cells if you like
         (for completeness in borderline collisions)
    """
    cell_size = GRID_CELL_SIZE

    for _ in range(COLLISION_PASSES):
        grid_map = collision_broad_phase_grid(creatures)
        # Optionally, we can also check neighbor cells to handle border collisions
        neighbor_offsets = [
            (0, 0), (1, 0), (-1, 0), (0, 1), (0, -1),
            (1, 1), (1, -1), (-1, 1), (-1, -1)
        ]
        for cell_key, idx_list in grid_map.items():
            # handle collisions within this cell
            collision_narrow_phase(creatures, idx_list)
            # Also check collisions with neighbor cells if you want more thorough coverage
            for nx, ny in neighbor_offsets:
                neigh_key = (cell_key[0] + nx, cell_key[1] + ny)
                if neigh_key in grid_map:
                    # combine cell idx_list with neighbor's idx_list
                    combined_list = idx_list + grid_map[neigh_key]
                    collision_narrow_phase(creatures, combined_list)

# ---------------------------------------------------------------------
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    dna = [speed, r, g, b, diet, fear, size_factor, longevity]
    We have:
      - dynamic diet flip logic
      - bigger corpse => multiple food
      - collisions via uniform grid
    """
    def __init__(self, x, y, father_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        self.heading = random.random() * 2.0 * math.pi

        if father_dna:
            speed = np.clip(father_dna[0] * random.uniform(0.9, 1.1), MIN_SPEED, MAX_SPEED)
            r     = np.clip(father_dna[1] + random.randint(-15, 15), 50, 255)
            g     = np.clip(father_dna[2] + random.randint(-15, 15), 50, 255)
            b     = np.clip(father_dna[3] + random.randint(-15, 15), 50, 255)

            diet  = father_dna[4]
            fear  = father_dna[5]
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            size_factor = father_dna[6]
            if random.random() < SIZE_MUTATION_CHANCE:
                size_factor *= random.uniform(0.8, 1.2)
                size_factor = min(max(MIN_SIZE, size_factor), MAX_SIZE)

            longevity = father_dna[7]
            if random.random() < LONGEVITY_MUTATION_CHANCE:
                longevity *= random.uniform(0.8, 1.2)
                longevity = min(max(0.5, longevity), 2.0)

            self.dna = [speed, r, g, b, diet, fear, size_factor, longevity]

            # big random mutation in speed?
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], MIN_SPEED, MAX_SPEED)
        else:
            # brand-new
            self.dna = [
                random.uniform(1.5, 3.0),
                random.randint(50, 255),
                random.randint(50, 255),
                random.randint(50, 255),
                0,
                random.uniform(0.0, 0.2),
                random.uniform(MIN_SIZE, 1.2),
                random.uniform(0.8, 1.2)
            ]

        self.speed       = self.dna[0]
        self.color       = (self.dna[1], self.dna[2], self.dna[3])
        self.diet        = int(self.dna[4])
        self.fear        = self.dna[5]
        self.size_factor = self.dna[6]
        self.longevity   = self.dna[7]

    def _get_mutation_type(self, father_dna):
        if not father_dna:
            return "initial"
        changes = []
        if abs(self.dna[0] - father_dna[0]) > 1.0:
            changes.append("speed_shift")
        c_diff = abs(self.dna[1] - father_dna[1]) + abs(self.dna[2] - father_dna[2]) + abs(self.dna[3] - father_dna[3])
        if c_diff > 45:
            changes.append("color_shift")
        if self.dna[4] != father_dna[4]:
            changes.append("diet_flip")
        if abs(self.dna[5] - father_dna[5]) > 0.05:
            changes.append("fear_shift")
        if abs(self.dna[6] - father_dna[6]) > 0.3:
            changes.append("size_shift")
        if abs(self.dna[7] - father_dna[7]) > 0.3:
            changes.append("long_shift")
        if not changes:
            return "minor"
        return "+".join(changes)

    def max_age(self):
        return AGE_BASE * self.longevity

    def strength(self):
        return self.size_factor * (1.0 + 0.5*(self.energy/100.0))

    def max_energy_capacity(self):
        return 100 + (self.size_factor - 1.0)*50

    def radius(self):
        return 6 * self.size_factor

    def update(self, all_creatures, foods):
        self.age += 1

        # fear cost
        if self.diet == 0 and self.fear > FEAR_THRESHOLD_HERB:
            self.energy -= FEAR_COST_BASE * self.fear * self.size_factor
        elif self.diet == 1 and self.fear > PRED_FEAR_THRESHOLD:
            self.energy -= FEAR_COST_BASE * self.fear * (self.size_factor * 0.5)

        self.energy -= BASE_ENERGY_LOSS_PER_MOVE
        self.energy -= (self.size_factor * SIZE_ENERGY_LOSS_FACTOR)
        self.energy = max(self.energy, 0)

        if self.diet == 0:
            # herb
            if self.fear > FEAR_THRESHOLD_HERB:
                predator, distp = self.find_nearest(all_creatures, lambda c: c is not self and c.diet == 1)
                if predator and distp < 120:
                    self.run_away(predator.x, predator.y)
                    self.wrap_screen()
                    return self.check_survival()

            food_t, distf = self.find_nearest(foods)
            if food_t and distf < SIGHT_RANGE:
                if not self.is_line_blocked(food_t.x, food_t.y, all_creatures):
                    self.move_toward(food_t.x, food_t.y)
                    if distance(self.x, self.y, food_t.x, food_t.y) < (6 + self.size_factor*4):
                        self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_FOOD)
                        foods.remove(food_t)
                else:
                    self.random_walk()
            else:
                self.random_walk()
        else:
            # predator
            ctarget, distc = self.find_nearest(all_creatures, lambda c: c is not self)
            if ctarget and distc < SIGHT_RANGE:
                if not self.is_line_blocked(ctarget.x, ctarget.y, all_creatures):
                    if ctarget.diet == 1:
                        # predator vs predator
                        if self.fear > PRED_FEAR_THRESHOLD and (ctarget.size_factor > self.size_factor*SIZE_DOMINANCE_RATIO):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                return self.predator_fight(ctarget)
                    else:
                        # predator vs herb
                        if (ctarget.size_factor>self.size_factor*HERB_SIZE_DOMINANCE_RATIO) and (self.fear> PRED_FEAR_THRESHOLD):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                gain = ENERGY_GAIN_FROM_PREY * ctarget.size_factor
                                self.energy = min(self.max_energy_capacity(), self.energy + gain)
                                return ("ate_creature", ctarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def predator_fight(self, other):
        if random.random() < 0.3:
            my_str = self.strength() * AMBUSH_BONUS
            their_str = other.strength()
        else:
            my_str = self.strength()
            their_str = other.strength()
        if my_str > their_str:
            gain = ENERGY_GAIN_FROM_PREY * other.size_factor
            self.energy = min(self.max_energy_capacity(), self.energy + gain)
            return ("ate_creature", other)
        elif their_str > my_str:
            return ("ate_creature", self)
        else:
            if random.random() < 0.5:
                return ("ate_creature", other)
            else:
                return ("ate_creature", self)

    def check_survival(self):
        if self.energy <= 0:
            return ("starved", None)
        if self.age > self.max_age():
            return ("aged", None)
        return ("alive", None)

    def reproduce(self, creatures, foods):
        if self.energy < 80:
            return None

        pop_size = len(creatures)
        food_count = len(foods)
        ratio_food_pop = food_count / pop_size if pop_size>0 else 1
        ratio_pop_cap  = pop_size / MAX_CREATURES

        chance = ENV_REPRO_CHANCE_BASE
        chance += ENV_REPRO_FOOD_FACTOR * ratio_food_pop
        chance -= ENV_REPRO_POP_FACTOR  * ratio_pop_cap
        chance = min(max(chance, 0.0), 1.0)

        if random.random() < chance:
            self.energy -= REPRODUCE_ENERGY_COST
            child_dna = list(self.dna)

            herb_count = sum(1 for c in creatures if c.diet == 0)
            pred_count = pop_size - herb_count
            flip_chance = BASE_DIET_FLIP_CHANCE

            if self.diet == 0:
                # father=herb
                if food_count < (herb_count * 0.7):
                    flip_chance += 0.05
                if (self.size_factor>1.5 and ratio_food_pop<0.5):
                    flip_chance += 0.05
                if random.random() < flip_chance:
                    child_dna[4] = 1
            else:
                # father=pred
                if pred_count > (herb_count*0.75):
                    flip_chance += 0.05
                if random.random() < flip_chance:
                    child_dna[4] = 0

            offx = random.gauss(0, 10)
            offy = random.gauss(0, 10)
            baby = Creature(self.x + offx, self.y + offy, father_dna=child_dna)
            return baby
        return None

    def random_walk(self):
        self.heading += random.uniform(-0.2, 0.2)
        dx = math.cos(self.heading) * self.speed
        dy = math.sin(self.heading) * self.speed
        self.x += dx
        self.y += dy

    def run_away(self, ox, oy):
        dx = self.x - ox
        dy = self.y - oy
        distv = distance(self.x, self.y, ox, oy)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def move_toward(self, tx, ty):
        dx = tx - self.x
        dy = ty - self.y
        distv = distance(self.x, self.y, tx, ty)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def wrap_screen(self):
        self.x %= 1200
        self.y %= 800

    def find_nearest(self, target_list, condition=None):
        valids = target_list
        if condition:
            valids = [t for t in target_list if condition(t)]
        if not valids:
            return (None, float('inf'))
        best_obj, best_dist = None, float('inf')
        for obj in valids:
            d = distance(self.x, self.y, obj.x, obj.y)
            if d < best_dist:
                best_dist = d
                best_obj = obj
        return (best_obj, best_dist)

    def is_line_blocked(self, tx, ty, creatures):
        targ_dist = distance(self.x, self.y, tx, ty)
        for c in creatures:
            if c is self:
                continue
            dist_c = distance(self.x, self.y, c.x, c.y)
            if dist_c >= targ_dist:
                continue
            d_line = line_distance_point(c.x, c.y, self.x, self.y, tx, ty)
            if d_line < BLOCKING_RADIUS:
                return True
        return False

    def draw(self, surface):
        sz = int(8 * self.size_factor)
        if self.diet == 0:
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), sz)
        else:
            half_w = sz
            points = [
                (self.x, self.y - sz),
                (self.x - half_w, self.y + sz),
                (self.x + half_w, self.y + sz)
            ]
            pygame.draw.polygon(surface, self.color, points)

# ============== SETUP ==============================
open(LOG_FILE, "w").close()

creatures = []
for _ in range(INITIAL_HERBIVORES):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 0
    c.diet   = 0
    creatures.append(c)
for _ in range(INITIAL_PREDATORS):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 1
    c.diet   = 1
    creatures.append(c)

foods = [Food() for _ in range(INITIAL_FOOD)]

paused = False
frames = 0
sim_frames = 0
population_history = deque(maxlen=300)
pred_ratio_history = deque(maxlen=300)

death_log = {'starved': 0, 'aged': 0, 'hunted': 0}
mutation_tracker = Counter()

day_cycle_frame = 0
day_count = 0

fps_sample_timer = 0.0
avg_frame_time = 0.0

running = True

while running:
    dt = clock.tick(30)
    frames += 1

    fps_sample_timer += dt
    avg_frame_time = 0.9 * avg_frame_time + 0.1 * dt

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused

    if not paused:
        sim_frames += 1
        day_cycle_frame = (day_cycle_frame + 1) % DAY_LENGTH
        if day_cycle_frame == 0:
            day_count += 1

    alpha = day_cycle_frame / (DAY_LENGTH - 1) if (DAY_LENGTH>1) else 1.0
    if alpha < 0.5:
        sub_a = alpha / 0.5
        bg_color = interpolate_color(NIGHT_COLOR, DAY_COLOR, sub_a)
        is_day = True
    else:
        sub_a = (alpha - 0.5) / 0.5
        bg_color = interpolate_color(DAY_COLOR, NIGHT_COLOR, sub_a)
        is_day = False

    screen.fill(bg_color)

    if not paused:
        new_creatures = []
        dead = []

        for c in creatures:
            status, other = c.update(creatures, foods)
            if status == "starved":
                death_log["starved"] += 1
                write_log(f"Death: starved (speed={c.speed:.2f})")
                dead.append(c)
                # spawn multiple food from corpse
                food_chunks = 1 + int(c.size_factor*2)
                for _ in range(food_chunks):
                    if random.random() < 0.3:
                        foods.append(Food())

            elif status == "aged":
                death_log["aged"] += 1
                write_log(f"Death: aged (speed={c.speed:.2f})")
                dead.append(c)
                food_chunks = 1 + int(c.size_factor*2)
                for _ in range(food_chunks):
                    if random.random() < 0.3:
                        foods.append(Food())

            elif status == "ate_creature":
                death_log["hunted"] += 1
                if other is c:
                    write_log(f"Death: hunted (speed={c.speed:.2f}) [lost predator fight]")
                    dead.append(c)
                    food_chunks = 1 + int(c.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random() < 0.3:
                            foods.append(Food())
                else:
                    write_log(f"Death: hunted (speed={other.speed:.2f})")
                    dead.append(other)
                    food_chunks = 1 + int(other.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random() < 0.3:
                            foods.append(Food())

            else:
                child = c.reproduce(creatures, foods)
                if child and len(creatures) < MAX_CREATURES:
                    mut_type = child._get_mutation_type(c.dna)
                    if mut_type not in ("initial", "minor"):
                        mutation_tracker[mut_type] += 1
                    new_creatures.append(child)
                    write_log(
                        f"Birth: speed={child.speed:.2f}, diet={child.diet}, size={child.size_factor:.2f}, "
                        f"fear={child.fear:.2f}, longevity={child.longevity:.2f}, mutation={mut_type}"
                    )

        creatures = [cr for cr in creatures if cr not in dead]
        space_left = max(0, MAX_CREATURES - len(creatures))
        creatures += new_creatures[:space_left]

        # collisions with uniform grid
        resolve_collisions(creatures)

        pop_size = len(creatures)
        pred_count = sum(1 for cr in creatures if cr.diet == 1)
        population_history.append(pop_size)
        ratio = pred_count / pop_size if pop_size > 0 else 0
        pred_ratio_history.append(ratio)

        if is_day:
            if random.random() < DAYTIME_FOOD_SPAWN_CHANCE:
                foods.append(Food())

    for f in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(f.x), int(f.y)), 4)
    for cr in creatures:
        cr.draw(screen)

    pred_count = sum(1 for x in creatures if x.diet == 1)
    herb_count = len(creatures) - pred_count
    ratio_str = "∞"
    if pred_count > 0:
        ratio_val = herb_count / pred_count
        ratio_str = f"{ratio_val:.1f}"

    stats = [
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Predators: {pred_count}, ratio=1:{ratio_str}",
        f"Food: {len(foods)}",
        f"Day Count: {day_count}",
        f"Sim Time: {sim_frames}",
    ]
    if creatures:
        avg_speed = np.mean([c.speed for c in creatures])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']}  aged:{death_log['aged']}  hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    for i, text in enumerate(stats):
        txt = font.render(text, True, (255,255,255))
        screen.blit(txt, (10, 10 + i * 25))

    mut_top3 = mutation_tracker.most_common(3)
    if mut_top3:
        mut_strs = [f"{mt}:{cnt}" for (mt, cnt) in mut_top3]
        mut_line = "Mutations: " + ", ".join(mut_strs)
        txtm = font.render(mut_line, True, (255,255,255))
        screen.blit(txtm, (10, 10 + len(stats) * 25))

    fps_txt = f"FPS: {int(clock.get_fps())}"
    ms_txt = f"FrameTime: {avg_frame_time:.1f}ms"
    fps_surf = font.render(fps_txt, True, (255,255,255))
    ms_surf = font.render(ms_txt, True, (255,255,255))
    screen.blit(fps_surf, (1050, 10))
    screen.blit(ms_surf, (1050, 35))

    graph_x, graph_y = 800, 70
    graph_w, graph_h = 380, 90
    pygame.draw.rect(screen, (50, 50, 50), (graph_x, graph_y, graph_w, graph_h))

    if len(population_history) > 1:
        points_pop = []
        for i, val in enumerate(population_history):
            xx = graph_x + graph_w - (len(population_history) - i)*2
            yy = graph_y + graph_h - (val / MAX_CREATURES) * graph_h
            points_pop.append((xx, yy))
        pygame.draw.lines(screen, GRAPH_COLOR, False, points_pop, 2)

    if len(pred_ratio_history) > 1:
        points_pred = []
        for i, ratio_val in enumerate(pred_ratio_history):
            xx = graph_x + graph_w - (len(pred_ratio_history) - i)*2
            yy = graph_y + graph_h - ratio_val * graph_h
            points_pred.append((xx, yy))
        pygame.draw.lines(screen, PREDATOR_GRAPH_COLOR, False, points_pred, 2)

    pygame.display.flip()

pygame.quit()


In [ ]:
"""
emergent_sim_v0.5.1 -- (~698 lines total)
Summary of changes from v0.5.0:
 - Collision resolution (to prevent overlapping creatures)
 - Bigger corpses => multiple food pieces
 - Dynamic diet flipping logic (father-based with environment scaling)
"""

import pygame
import numpy as np
import random
from collections import deque, Counter
from datetime import datetime
import math

# =================== CONFIG ====================
MAX_CREATURES = 2000

# Start ratio
INITIAL_HERBIVORES = 180
INITIAL_PREDATORS  = 20

INITIAL_FOOD = 40

AGE_BASE = 700  # base lifespan, scaled by longevity gene

# If predators die off too quickly, raise ENERGY_GAIN_FROM_PREY.
ENERGY_GAIN_FROM_PREY = 80  
ENERGY_GAIN_FROM_FOOD = 22

# Movement energy costs
BASE_ENERGY_LOSS_PER_MOVE = 0.04   # slightly lower than before
SIZE_ENERGY_LOSS_FACTOR   = 0.04

REPRODUCE_ENERGY_COST = 50
ENV_REPRO_CHANCE_BASE = 0.3
ENV_REPRO_FOOD_FACTOR = 0.2
ENV_REPRO_POP_FACTOR  = 0.2

MUTATION_RATE = 0.1
# Reintroduce a dynamic flipping chance in reproduction:
BASE_DIET_FLIP_CHANCE = 0.02   # 2% default

# Fear, size, longevity, etc.
FEAR_MUTATION_CHANCE = 0.2
SIZE_MUTATION_CHANCE = 0.2
LONGEVITY_MUTATION_CHANCE = 0.2

MAX_SPEED = 8.0
MIN_SPEED = 0.5
MIN_SIZE  = 0.5
MAX_SIZE  = 2.5

FEAR_COST_BASE = 0.02
FEAR_THRESHOLD_HERB = 0.3

AMBUSH_BONUS = 1.1  
PRED_FEAR_THRESHOLD = 0.5   
SIZE_DOMINANCE_RATIO = 1.2  
HERB_SIZE_DOMINANCE_RATIO = 1.2

SIGHT_RANGE = 200.0
BLOCKING_RADIUS = 5.0

DAY_LENGTH = 240
DAY_COLOR   = (170, 220, 255)  # pale blue
NIGHT_COLOR = (0, 0, 0)

DAYTIME_FOOD_SPAWN_CHANCE = 0.05

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Colors
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)
PREDATOR_GRAPH_COLOR = (150, 100, 200)

# ---------------------------------------------------------------------
def write_log(message):
    """Write to the log file every LOG_INTERVAL 'sim_frames'."""
    if sim_frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {sim_frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    """Returns shortest distance from (px,py) to line segment A->B."""
    ABx = bx - ax
    ABy = by - ay
    APx = px - ax
    APy = py - ay
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        return distance(px, py, ax, ay)
    t = (APx*ABx + APy*ABy) / AB_len_sq
    t = max(0, min(1, t))
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    """Linear interpolate between two RGB tuples, alpha in [0..1]."""
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

def resolve_collisions(creatures):
    """
    Simple O(N^2) collision check. If two creatures overlap, push them apart.
    For real performance, use spatial partitioning or a physics engine.
    """
    for i in range(len(creatures)):
        for j in range(i+1, len(creatures)):
            c1 = creatures[i]
            c2 = creatures[j]
            # approximate radius
            r1 = 6 * c1.size_factor
            r2 = 6 * c2.size_factor
            dx = c2.x - c1.x
            dy = c2.y - c1.y
            dist = math.hypot(dx, dy)
            min_dist = r1 + r2
            if dist < min_dist and dist > 0:
                # They overlap => push them apart half each
                overlap = 0.5*(min_dist - dist)
                nx = dx / dist
                ny = dy / dist
                c1.x -= overlap * nx
                c1.y -= overlap * ny
                c2.x += overlap * nx
                c2.y += overlap * ny

# ---------------------------------------------------------------------
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    dna = [speed, r, g, b, diet, fear, size_factor, longevity]

    A dynamic chance to flip diet in child based on environment:
      - If father is herbivore, but environment is "food-limited" + many herbivores,
        we raise the chance the child is predator.
      - If father is predator, but environment is "herb-limited" + abundant plants,
        we raise chance child flips to herb.

    Also includes:
      - Collision resolution
      - Multiple food pieces from large corpses
      - Gains from prey scaled by prey's size
    """
    def __init__(self, x, y, father_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        self.heading = random.random() * 2.0 * math.pi

        if father_dna:
            speed = np.clip(father_dna[0] * random.uniform(0.9, 1.1), MIN_SPEED, MAX_SPEED)
            r     = np.clip(father_dna[1] + random.randint(-15, 15), 50, 255)
            g     = np.clip(father_dna[2] + random.randint(-15, 15), 50, 255)
            b     = np.clip(father_dna[3] + random.randint(-15, 15), 50, 255)

            diet  = father_dna[4]
            fear  = father_dna[5]
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            size_factor = father_dna[6]
            if random.random() < SIZE_MUTATION_CHANCE:
                size_factor *= random.uniform(0.8, 1.2)
                size_factor = min(max(MIN_SIZE, size_factor), MAX_SIZE)

            longevity = father_dna[7]
            if random.random() < LONGEVITY_MUTATION_CHANCE:
                longevity *= random.uniform(0.8, 1.2)
                longevity = min(max(0.5, longevity), 2.0)

            self.dna = [speed, r, g, b, diet, fear, size_factor, longevity]

            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], MIN_SPEED, MAX_SPEED)
        else:
            # brand new
            self.dna = [
                random.uniform(1.5, 3.0),
                random.randint(50, 255),
                random.randint(50, 255),
                random.randint(50, 255),
                0,  # default diet = herb
                random.uniform(0.0, 0.2),
                random.uniform(MIN_SIZE, 1.2),
                random.uniform(0.8, 1.2)
            ]

        self.speed       = self.dna[0]
        self.color       = (self.dna[1], self.dna[2], self.dna[3])
        self.diet        = int(self.dna[4])
        self.fear        = self.dna[5]
        self.size_factor = self.dna[6]
        self.longevity   = self.dna[7]

    def _get_mutation_type(self, father_dna):
        if not father_dna:
            return "initial"
        changes = []
        if abs(self.dna[0] - father_dna[0]) > 1.0:
            changes.append("speed_shift")
        c_diff = abs(self.dna[1] - father_dna[1]) + abs(self.dna[2] - father_dna[2]) + abs(self.dna[3] - father_dna[3])
        if c_diff > 45:
            changes.append("color_shift")
        if self.dna[4] != father_dna[4]:
            changes.append("diet_flip")
        if abs(self.dna[5] - father_dna[5]) > 0.05:
            changes.append("fear_shift")
        if abs(self.dna[6] - father_dna[6]) > 0.3:
            changes.append("size_shift")
        if abs(self.dna[7] - father_dna[7]) > 0.3:
            changes.append("long_shift")
        if not changes:
            return "minor"
        return "+".join(changes)

    def max_age(self):
        return AGE_BASE * self.longevity

    def strength(self):
        return self.size_factor * (1.0 + 0.5*(self.energy/100.0))

    def max_energy_capacity(self):
        return 100 + (self.size_factor - 1.0)*50

    def radius(self):
        """Approximate collision radius for collision resolution."""
        return 6 * self.size_factor

    def update(self, all_creatures, foods):
        self.age += 1

        # fear cost
        if self.diet == 0 and self.fear > FEAR_THRESHOLD_HERB:
            self.energy -= FEAR_COST_BASE * self.fear * self.size_factor
        elif self.diet == 1 and self.fear > PRED_FEAR_THRESHOLD:
            self.energy -= FEAR_COST_BASE * self.fear * (self.size_factor * 0.5)

        # size-based drain
        self.energy -= BASE_ENERGY_LOSS_PER_MOVE
        self.energy -= (self.size_factor * SIZE_ENERGY_LOSS_FACTOR)
        self.energy = max(self.energy, 0)

        # basic AI
        if self.diet == 0:
            # herb
            if self.fear > FEAR_THRESHOLD_HERB:
                predator, distp = self.find_nearest(all_creatures, lambda c: c is not self and c.diet == 1)
                if predator and distp < 120:
                    self.run_away(predator.x, predator.y)
                    self.wrap_screen()
                    return self.check_survival()

            # find nearest food
            food_t, distf = self.find_nearest(foods)
            if food_t and distf < SIGHT_RANGE:
                if not self.is_line_blocked(food_t.x, food_t.y, all_creatures):
                    self.move_toward(food_t.x, food_t.y)
                    if distance(self.x, self.y, food_t.x, food_t.y) < (6 + self.size_factor*4):
                        # eat
                        self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_FOOD)
                        foods.remove(food_t)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        else:
            # predator
            target, distv = self.find_nearest(all_creatures, lambda c: c is not self)
            if target and distv < SIGHT_RANGE:
                if not self.is_line_blocked(target.x, target.y, all_creatures):
                    if target.diet == 1:
                        # predator vs predator
                        if self.fear > PRED_FEAR_THRESHOLD and \
                           (target.size_factor > self.size_factor * SIZE_DOMINANCE_RATIO):
                            self.run_away(target.x, target.y)
                        else:
                            self.move_toward(target.x, target.y)
                            if distance(self.x, self.y, target.x, target.y) < (10 + self.size_factor*4 + target.size_factor*4):
                                return self.predator_fight(target)
                    else:
                        # predator vs herb
                        if (target.size_factor > self.size_factor * HERB_SIZE_DOMINANCE_RATIO) \
                           and (self.fear > PRED_FEAR_THRESHOLD):
                            self.run_away(target.x, target.y)
                        else:
                            self.move_toward(target.x, target.y)
                            if distance(self.x, self.y, target.x, target.y) < (10 + self.size_factor*4 + target.size_factor*4):
                                # eat, scaled by prey size
                                gain = ENERGY_GAIN_FROM_PREY * target.size_factor
                                self.energy = min(self.max_energy_capacity(), self.energy + gain)
                                return ("ate_creature", target)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def predator_fight(self, other):
        # 30% chance to get ambush bonus
        if random.random() < 0.3:
            my_str = self.strength() * AMBUSH_BONUS
            their_str = other.strength()
        else:
            my_str = self.strength()
            their_str = other.strength()
        if my_str > their_str:
            # I win
            gain = ENERGY_GAIN_FROM_PREY * other.size_factor
            self.energy = min(self.max_energy_capacity(), self.energy + gain)
            return ("ate_creature", other)
        elif their_str > my_str:
            # I lose
            return ("ate_creature", self)
        else:
            # tie
            if random.random() < 0.5:
                return ("ate_creature", other)
            else:
                return ("ate_creature", self)

    def check_survival(self):
        if self.energy <= 0:
            return ("starved", None)
        if self.age > self.max_age():
            return ("aged", None)
        return ("alive", None)

    def reproduce(self, creatures, foods):
        """Dynamic diet flipping logic + environment-based chance to reproduce."""
        if self.energy < 80:
            return None

        pop_size = len(creatures)
        food_count = len(foods)
        ratio_food_pop = food_count / pop_size if pop_size>0 else 1
        ratio_pop_cap  = pop_size / MAX_CREATURES

        chance = ENV_REPRO_CHANCE_BASE
        chance += ENV_REPRO_FOOD_FACTOR * ratio_food_pop
        chance -= ENV_REPRO_POP_FACTOR  * ratio_pop_cap
        chance = min(max(chance, 0.0), 1.0)

        if random.random() < chance:
            self.energy -= REPRODUCE_ENERGY_COST
            # father_dna = self.dna
            # decide if child flips diet
            child_dna = list(self.dna)  # copy

            # We'll scale the flip chance based on environment
            # e.g., if there's a big surplus of herbivores vs food => easier to become predator
            # or if there's a big surplus of food => easier to remain herb
            # This is just an example formula:
            herb_count = sum(1 for c in creatures if c.diet == 0)
            pred_count = pop_size - herb_count

            # if father is herb
            if self.diet == 0:
                # base chance for flipping
                flip_chance = BASE_DIET_FLIP_CHANCE
                # if big ratio of herbivores to actual food => more flipping to predator
                if food_count < (herb_count * 0.7):
                    # we have ~30% deficit in food
                    flip_chance += 0.05
                # if father is large + not enough food => bigger chance
                if (self.size_factor>1.5 and ratio_food_pop <0.5):
                    flip_chance += 0.05

                if random.random() < flip_chance:
                    child_dna[4] = 1  # become predator
            else:
                # father is predator
                flip_chance = BASE_DIET_FLIP_CHANCE
                # if plenty of food vs. few herbivores => maybe child flips to herb
                # or if predator count is too high vs. total => become herb
                if pred_count > herb_count*0.75:
                    flip_chance += 0.05
                if random.random() < flip_chance:
                    child_dna[4] = 0  # become herb

            # spawn child near father
            offx = random.gauss(0, 10)
            offy = random.gauss(0, 10)
            baby = Creature(self.x + offx, self.y + offy, father_dna=child_dna)
            return baby
        return None

    # movement helpers
    def random_walk(self):
        self.heading += random.uniform(-0.2, 0.2)
        dx = math.cos(self.heading) * self.speed
        dy = math.sin(self.heading) * self.speed
        self.x += dx
        self.y += dy

    def run_away(self, ox, oy):
        dx = self.x - ox
        dy = self.y - oy
        distv = distance(self.x, self.y, ox, oy)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def move_toward(self, tx, ty):
        dx = tx - self.x
        dy = ty - self.y
        distv = distance(self.x, self.y, tx, ty)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def wrap_screen(self):
        self.x %= 1200
        self.y %= 800

    def find_nearest(self, target_list, condition=None):
        valids = target_list
        if condition:
            valids = [t for t in target_list if condition(t)]
        if not valids:
            return (None, float('inf'))
        best_obj, best_dist = None, float('inf')
        for obj in valids:
            d = distance(self.x, self.y, obj.x, obj.y)
            if d < best_dist:
                best_dist = d
                best_obj = obj
        return (best_obj, best_dist)

    def is_line_blocked(self, tx, ty, creatures):
        targ_dist = distance(self.x, self.y, tx, ty)
        for c in creatures:
            if c is self:
                continue
            dist_c = distance(self.x, self.y, c.x, c.y)
            if dist_c >= targ_dist:
                continue
            d_line = line_distance_point(c.x, c.y, self.x, self.y, tx, ty)
            if d_line < BLOCKING_RADIUS:
                return True
        return False

    def draw(self, surface):
        sz = int(8 * self.size_factor)
        if self.diet == 0:
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), sz)
        else:
            half_w = sz
            points = [
                (self.x, self.y - sz),
                (self.x - half_w, self.y + sz),
                (self.x + half_w, self.y + sz)
            ]
            pygame.draw.polygon(surface, self.color, points)

# ====================================================================
# ============== SETUP ==============================================
open(LOG_FILE, "w").close()

creatures = []
for _ in range(INITIAL_HERBIVORES):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 0
    c.diet   = 0
    creatures.append(c)
for _ in range(INITIAL_PREDATORS):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 1
    c.diet   = 1
    creatures.append(c)

foods = [Food() for _ in range(INITIAL_FOOD)]

paused = False
frames = 0
sim_frames = 0
population_history = deque(maxlen=300)
pred_ratio_history = deque(maxlen=300)

death_log = {'starved': 0, 'aged': 0, 'hunted': 0}
mutation_tracker = Counter()

day_cycle_frame = 0
day_count = 0

fps_sample_timer = 0.0
avg_frame_time = 0.0

running = True

# ====================================================================
while running:
    dt = clock.tick(30)
    frames += 1
    fps_sample_timer += dt
    avg_frame_time = 0.9 * avg_frame_time + 0.1 * dt

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused

    if not paused:
        sim_frames += 1
        day_cycle_frame = (day_cycle_frame + 1) % DAY_LENGTH
        if day_cycle_frame == 0:
            day_count += 1

    alpha = day_cycle_frame / (DAY_LENGTH - 1) if (DAY_LENGTH>1) else 1.0
    if alpha < 0.5:
        sub_a = alpha / 0.5
        bg_color = interpolate_color(NIGHT_COLOR, DAY_COLOR, sub_a)
        is_day = True
    else:
        sub_a = (alpha - 0.5) / 0.5
        bg_color = interpolate_color(DAY_COLOR, NIGHT_COLOR, sub_a)
        is_day = False

    screen.fill(bg_color)

    if not paused:
        new_creatures = []
        dead = []

        for c in creatures:
            status, other = c.update(creatures, foods)
            if status == "starved":
                death_log["starved"] += 1
                write_log(f"Death: starved (speed={c.speed:.2f})")
                dead.append(c)
                # spawn multiple food based on c's size
                food_chunks = 1 + int(c.size_factor*2)
                for _ in range(food_chunks):
                    if random.random()<0.3:  # or 100% if you prefer
                        foods.append(Food())

            elif status == "aged":
                death_log["aged"] += 1
                write_log(f"Death: aged (speed={c.speed:.2f})")
                dead.append(c)
                # spawn multiple food from corpse
                food_chunks = 1 + int(c.size_factor*2)
                for _ in range(food_chunks):
                    if random.random()<0.3:
                        foods.append(Food())

            elif status == "ate_creature":
                # predator kill
                death_log["hunted"] += 1
                if other is c:
                    # c lost the fight
                    write_log(f"Death: hunted (speed={c.speed:.2f}) [lost predator fight]")
                    dead.append(c)
                    # bigger corpse => more food
                    food_chunks = 1 + int(c.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
                else:
                    write_log(f"Death: hunted (speed={other.speed:.2f})")
                    dead.append(other)
                    # bigger corpse => more food
                    food_chunks = 1 + int(other.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
            else:
                # alive => possible reproduction
                child = c.reproduce(creatures, foods)
                if child and len(creatures) < MAX_CREATURES:
                    mut_type = child._get_mutation_type(c.dna)
                    if mut_type not in ("initial", "minor"):
                        mutation_tracker[mut_type] += 1
                    new_creatures.append(child)
                    write_log(
                        f"Birth: speed={child.speed:.2f}, diet={child.diet}, size={child.size_factor:.2f}, "
                        f"fear={child.fear:.2f}, longevity={child.longevity:.2f}, mutation={mut_type}"
                    )

        # remove dead
        creatures = [cr for cr in creatures if cr not in dead]
        # add newborns
        space_left = max(0, MAX_CREATURES - len(creatures))
        creatures += new_creatures[:space_left]

        # collisions
        resolve_collisions(creatures)

        pop_size = len(creatures)
        pred_count = sum(1 for cr in creatures if cr.diet == 1)
        population_history.append(pop_size)
        ratio = pred_count / pop_size if pop_size>0 else 0
        pred_ratio_history.append(ratio)

        if is_day:
            if random.random() < DAYTIME_FOOD_SPAWN_CHANCE:
                foods.append(Food())

    for f in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(f.x), int(f.y)), 4)
    for cr in creatures:
        cr.draw(screen)

    pred_count = sum(1 for x in creatures if x.diet == 1)
    herb_count = len(creatures) - pred_count
    ratio_str = "∞"
    if pred_count > 0:
        ratio_val = herb_count / pred_count
        ratio_str = f"{ratio_val:.1f}"

    stats = [
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Predators: {pred_count}, ratio=1:{ratio_str}",
        f"Food: {len(foods)}",
        f"Day Count: {day_count}",
        f"Sim Time: {sim_frames}",
    ]
    if creatures:
        avg_speed = np.mean([c.speed for c in creatures])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']}  aged:{death_log['aged']}  hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    for i, text in enumerate(stats):
        txt = font.render(text, True, (255,255,255))
        screen.blit(txt, (10, 10 + i*25))

    mut_top3 = mutation_tracker.most_common(3)
    if mut_top3:
        mut_strs = [f"{mt}:{cnt}" for (mt, cnt) in mut_top3]
        mut_line = "Mutations: " + ", ".join(mut_strs)
        txtm = font.render(mut_line, True, (255,255,255))
        screen.blit(txtm, (10, 10 + len(stats)*25))

    fps_txt = f"FPS: {int(clock.get_fps())}"
    ms_txt = f"FrameTime: {avg_frame_time:.1f}ms"
    fps_surf = font.render(fps_txt, True, (255,255,255))
    ms_surf = font.render(ms_txt, True, (255,255,255))
    screen.blit(fps_surf, (1050, 10))
    screen.blit(ms_surf, (1050, 35))

    graph_x, graph_y = 800, 70
    graph_w, graph_h = 380, 90
    pygame.draw.rect(screen, (50,50,50), (graph_x, graph_y, graph_w, graph_h))

    if len(population_history) > 1:
        points_pop = []
        for i, val in enumerate(population_history):
            xx = graph_x + graph_w - (len(population_history) - i)*2
            yy = graph_y + graph_h - (val / MAX_CREATURES) * graph_h
            points_pop.append((xx, yy))
        pygame.draw.lines(screen, GRAPH_COLOR, False, points_pop, 2)

    if len(pred_ratio_history) > 1:
        points_pred = []
        for i, ratio_val in enumerate(pred_ratio_history):
            xx = graph_x + graph_w - (len(pred_ratio_history) - i)*2
            yy = graph_y + graph_h - ratio_val * graph_h
            points_pred.append((xx, yy))
        pygame.draw.lines(screen, PREDATOR_GRAPH_COLOR, False, points_pred, 2)

    pygame.display.flip()

pygame.quit()


In [ ]:
# emergent_sim_v0.5.0

import pygame
import numpy as np
import random
from collections import deque, Counter
from datetime import datetime
import math

# =================== CONFIG ====================
MAX_CREATURES = 2000

# Start ratio
INITIAL_HERBIVORES = 180
INITIAL_PREDATORS  = 20

INITIAL_FOOD = 40

AGE_BASE = 700  # base lifespan, scaled by longevity gene

# If predators die off too quickly, raise ENERGY_GAIN_FROM_PREY.
ENERGY_GAIN_FROM_PREY = 80  
ENERGY_GAIN_FROM_FOOD = 22

# Movement energy costs
BASE_ENERGY_LOSS_PER_MOVE = 0.04   # slightly lower than before
SIZE_ENERGY_LOSS_FACTOR   = 0.04

REPRODUCE_ENERGY_COST = 50
ENV_REPRO_CHANCE_BASE = 0.3
ENV_REPRO_FOOD_FACTOR = 0.2
ENV_REPRO_POP_FACTOR  = 0.2

MUTATION_RATE = 0.1
# Reintroduce a dynamic flipping chance in reproduction:
BASE_DIET_FLIP_CHANCE = 0.02   # 2% default

# Fear, size, longevity, etc.
FEAR_MUTATION_CHANCE = 0.2
SIZE_MUTATION_CHANCE = 0.2
LONGEVITY_MUTATION_CHANCE = 0.2

MAX_SPEED = 8.0
MIN_SPEED = 0.5
MIN_SIZE  = 0.5
MAX_SIZE  = 2.5

FEAR_COST_BASE = 0.02
FEAR_THRESHOLD_HERB = 0.3

AMBUSH_BONUS = 1.1  
PRED_FEAR_THRESHOLD = 0.5   
SIZE_DOMINANCE_RATIO = 1.2  
HERB_SIZE_DOMINANCE_RATIO = 1.2

SIGHT_RANGE = 200.0
BLOCKING_RADIUS = 5.0

DAY_LENGTH = 240
DAY_COLOR   = (170, 220, 255)  # pale blue
NIGHT_COLOR = (0, 0, 0)

DAYTIME_FOOD_SPAWN_CHANCE = 0.05

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Colors
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)
PREDATOR_GRAPH_COLOR = (150, 100, 200)

# ---------------------------------------------------------------------
def write_log(message):
    """Write to the log file every LOG_INTERVAL 'sim_frames'."""
    if sim_frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {sim_frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    """Returns shortest distance from (px,py) to line segment A->B."""
    ABx = bx - ax
    ABy = by - ay
    APx = px - ax
    APy = py - ay
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        return distance(px, py, ax, ay)
    t = (APx*ABx + APy*ABy) / AB_len_sq
    t = max(0, min(1, t))
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    """Linear interpolate between two RGB tuples, alpha in [0..1]."""
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

def resolve_collisions(creatures):
    """
    Simple O(N^2) collision check. If two creatures overlap, push them apart.
    For real performance, use spatial partitioning or a physics engine.
    """
    for i in range(len(creatures)):
        for j in range(i+1, len(creatures)):
            c1 = creatures[i]
            c2 = creatures[j]
            # approximate radius
            r1 = 6 * c1.size_factor
            r2 = 6 * c2.size_factor
            dx = c2.x - c1.x
            dy = c2.y - c1.y
            dist = math.hypot(dx, dy)
            min_dist = r1 + r2
            if dist < min_dist and dist > 0:
                # They overlap => push them apart half each
                overlap = 0.5*(min_dist - dist)
                nx = dx / dist
                ny = dy / dist
                c1.x -= overlap * nx
                c1.y -= overlap * ny
                c2.x += overlap * nx
                c2.y += overlap * ny

# ---------------------------------------------------------------------
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    dna = [speed, r, g, b, diet, fear, size_factor, longevity]

    A dynamic chance to flip diet in child based on environment:
      - If father is herbivore, but environment is "food-limited" + many herbivores,
        we raise the chance the child is predator.
      - If father is predator, but environment is "herb-limited" + abundant plants,
        we raise chance child flips to herb.
    """
    def __init__(self, x, y, father_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        self.heading = random.random() * 2.0 * math.pi

        if father_dna:
            speed = np.clip(father_dna[0] * random.uniform(0.9, 1.1), MIN_SPEED, MAX_SPEED)
            r     = np.clip(father_dna[1] + random.randint(-15, 15), 50, 255)
            g     = np.clip(father_dna[2] + random.randint(-15, 15), 50, 255)
            b     = np.clip(father_dna[3] + random.randint(-15, 15), 50, 255)

            # diet flipping logic handled in reproduce() => father_dna[4] is "base"
            diet  = father_dna[4]

            fear  = father_dna[5]
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            size_factor = father_dna[6]
            if random.random() < SIZE_MUTATION_CHANCE:
                size_factor *= random.uniform(0.8, 1.2)
                size_factor = min(max(MIN_SIZE, size_factor), MAX_SIZE)

            longevity = father_dna[7]
            if random.random() < LONGEVITY_MUTATION_CHANCE:
                longevity *= random.uniform(0.8, 1.2)
                longevity = min(max(0.5, longevity), 2.0)

            self.dna = [speed, r, g, b, diet, fear, size_factor, longevity]

            # Big random mutation in speed?
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], MIN_SPEED, MAX_SPEED)
        else:
            # brand new
            self.dna = [
                random.uniform(1.5, 3.0),
                random.randint(50, 255),
                random.randint(50, 255),
                random.randint(50, 255),
                0,  # default diet = herb
                random.uniform(0.0, 0.2),
                random.uniform(MIN_SIZE, 1.2),
                random.uniform(0.8, 1.2)
            ]

        self.speed       = self.dna[0]
        self.color       = (self.dna[1], self.dna[2], self.dna[3])
        self.diet        = int(self.dna[4])
        self.fear        = self.dna[5]
        self.size_factor = self.dna[6]
        self.longevity   = self.dna[7]

    def _get_mutation_type(self, father_dna):
        if not father_dna:
            return "initial"
        changes = []
        if abs(self.dna[0] - father_dna[0]) > 1.0:
            changes.append("speed_shift")
        c_diff = abs(self.dna[1] - father_dna[1]) + abs(self.dna[2] - father_dna[2]) + abs(self.dna[3] - father_dna[3])
        if c_diff > 45:
            changes.append("color_shift")
        if self.dna[4] != father_dna[4]:
            changes.append("diet_flip")
        if abs(self.dna[5] - father_dna[5]) > 0.05:
            changes.append("fear_shift")
        if abs(self.dna[6] - father_dna[6]) > 0.3:
            changes.append("size_shift")
        if abs(self.dna[7] - father_dna[7]) > 0.3:
            changes.append("long_shift")
        if not changes:
            return "minor"
        return "+".join(changes)

    def max_age(self):
        return AGE_BASE * self.longevity

    def strength(self):
        return self.size_factor * (1.0 + 0.5*(self.energy/100.0))

    def max_energy_capacity(self):
        return 100 + (self.size_factor - 1.0)*50

    def radius(self):
        """Approximate collision radius for collision resolution."""
        return 6 * self.size_factor

    def update(self, all_creatures, foods):
        self.age += 1

        # fear cost
        if self.diet == 0 and self.fear > FEAR_THRESHOLD_HERB:
            self.energy -= FEAR_COST_BASE * self.fear * self.size_factor
        elif self.diet == 1 and self.fear > PRED_FEAR_THRESHOLD:
            self.energy -= FEAR_COST_BASE * self.fear * (self.size_factor * 0.5)

        # size-based drain
        self.energy -= BASE_ENERGY_LOSS_PER_MOVE
        self.energy -= (self.size_factor * SIZE_ENERGY_LOSS_FACTOR)
        self.energy = max(self.energy, 0)

        # basic AI
        if self.diet == 0:
            # herb
            if self.fear > FEAR_THRESHOLD_HERB:
                predator, distp = self.find_nearest(all_creatures, lambda c: c is not self and c.diet == 1)
                if predator and distp < 120:
                    self.run_away(predator.x, predator.y)
                    self.wrap_screen()
                    return self.check_survival()

            # find nearest food
            food_t, distf = self.find_nearest(foods)
            if food_t and distf < SIGHT_RANGE:
                if not self.is_line_blocked(food_t.x, food_t.y, all_creatures):
                    self.move_toward(food_t.x, food_t.y)
                    if distance(self.x, self.y, food_t.x, food_t.y) < (6 + self.size_factor*4):
                        # eat
                        self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_FOOD)
                        foods.remove(food_t)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        else:
            # predator
            target, distv = self.find_nearest(all_creatures, lambda c: c is not self)
            if target and distv < SIGHT_RANGE:
                if not self.is_line_blocked(target.x, target.y, all_creatures):
                    if target.diet == 1:
                        # predator vs predator
                        if self.fear > PRED_FEAR_THRESHOLD and \
                           (target.size_factor > self.size_factor * SIZE_DOMINANCE_RATIO):
                            self.run_away(target.x, target.y)
                        else:
                            self.move_toward(target.x, target.y)
                            if distance(self.x, self.y, target.x, target.y) < (10 + self.size_factor*4 + target.size_factor*4):
                                return self.predator_fight(target)
                    else:
                        # predator vs herb
                        if (target.size_factor > self.size_factor * HERB_SIZE_DOMINANCE_RATIO) \
                           and (self.fear > PRED_FEAR_THRESHOLD):
                            self.run_away(target.x, target.y)
                        else:
                            self.move_toward(target.x, target.y)
                            if distance(self.x, self.y, target.x, target.y) < (10 + self.size_factor*4 + target.size_factor*4):
                                # eat, scaled by prey size
                                gain = ENERGY_GAIN_FROM_PREY * target.size_factor
                                self.energy = min(self.max_energy_capacity(), self.energy + gain)
                                return ("ate_creature", target)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def predator_fight(self, other):
        # 30% chance to get ambush bonus
        if random.random() < 0.3:
            my_str = self.strength() * AMBUSH_BONUS
            their_str = other.strength()
        else:
            my_str = self.strength()
            their_str = other.strength()
        if my_str > their_str:
            # I win
            gain = ENERGY_GAIN_FROM_PREY * other.size_factor
            self.energy = min(self.max_energy_capacity(), self.energy + gain)
            return ("ate_creature", other)
        elif their_str > my_str:
            # I lose
            return ("ate_creature", self)
        else:
            # tie
            if random.random() < 0.5:
                return ("ate_creature", other)
            else:
                return ("ate_creature", self)

    def check_survival(self):
        if self.energy <= 0:
            return ("starved", None)
        if self.age > self.max_age():
            return ("aged", None)
        return ("alive", None)

    def reproduce(self, creatures, foods):
        """Dynamic diet flipping logic + environment-based chance to reproduce."""
        if self.energy < 80:
            return None

        pop_size = len(creatures)
        food_count = len(foods)
        ratio_food_pop = food_count / pop_size if pop_size>0 else 1
        ratio_pop_cap  = pop_size / MAX_CREATURES

        chance = ENV_REPRO_CHANCE_BASE
        chance += ENV_REPRO_FOOD_FACTOR * ratio_food_pop
        chance -= ENV_REPRO_POP_FACTOR  * ratio_pop_cap
        chance = min(max(chance, 0.0), 1.0)

        if random.random() < chance:
            self.energy -= REPRODUCE_ENERGY_COST
            # father_dna = self.dna
            # decide if child flips diet
            child_dna = list(self.dna)  # copy

            # We'll scale the flip chance based on environment
            # e.g., if there's a big surplus of herbivores vs food => easier to become predator
            # or if there's a big surplus of food => easier to remain herb
            # This is just an example formula:
            herb_count = sum(1 for c in creatures if c.diet == 0)
            pred_count = pop_size - herb_count

            # if father is herb
            if self.diet == 0:
                # base chance for flipping
                flip_chance = BASE_DIET_FLIP_CHANCE
                # if big ratio of herbivores to actual food => more flipping to predator
                if food_count < (herb_count * 0.7):
                    # we have ~30% deficit in food
                    flip_chance += 0.05
                # if father is large + not enough food => bigger chance
                if (self.size_factor>1.5 and ratio_food_pop <0.5):
                    flip_chance += 0.05

                if random.random() < flip_chance:
                    child_dna[4] = 1  # become predator
            else:
                # father is predator
                flip_chance = BASE_DIET_FLIP_CHANCE
                # if plenty of food vs. few herbivores => maybe child flips to herb
                # or if predator count is too high vs. total => become herb
                if pred_count > herb_count*0.75:
                    flip_chance += 0.05
                if random.random() < flip_chance:
                    child_dna[4] = 0  # become herb

            # spawn child near father
            offx = random.gauss(0, 10)
            offy = random.gauss(0, 10)
            baby = Creature(self.x + offx, self.y + offy, father_dna=child_dna)
            return baby
        return None

    # movement helpers
    def random_walk(self):
        self.heading += random.uniform(-0.2, 0.2)
        dx = math.cos(self.heading) * self.speed
        dy = math.sin(self.heading) * self.speed
        self.x += dx
        self.y += dy

    def run_away(self, ox, oy):
        dx = self.x - ox
        dy = self.y - oy
        distv = distance(self.x, self.y, ox, oy)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def move_toward(self, tx, ty):
        dx = tx - self.x
        dy = ty - self.y
        distv = distance(self.x, self.y, tx, ty)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def wrap_screen(self):
        self.x %= 1200
        self.y %= 800

    def find_nearest(self, target_list, condition=None):
        valids = target_list
        if condition:
            valids = [t for t in target_list if condition(t)]
        if not valids:
            return (None, float('inf'))
        best_obj, best_dist = None, float('inf')
        for obj in valids:
            d = distance(self.x, self.y, obj.x, obj.y)
            if d < best_dist:
                best_dist = d
                best_obj = obj
        return (best_obj, best_dist)

    def is_line_blocked(self, tx, ty, creatures):
        targ_dist = distance(self.x, self.y, tx, ty)
        for c in creatures:
            if c is self:
                continue
            dist_c = distance(self.x, self.y, c.x, c.y)
            if dist_c >= targ_dist:
                continue
            d_line = line_distance_point(c.x, c.y, self.x, self.y, tx, ty)
            if d_line < BLOCKING_RADIUS:
                return True
        return False

    def draw(self, surface):
        sz = int(8 * self.size_factor)
        if self.diet == 0:
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), sz)
        else:
            half_w = sz
            points = [
                (self.x, self.y - sz),
                (self.x - half_w, self.y + sz),
                (self.x + half_w, self.y + sz)
            ]
            pygame.draw.polygon(surface, self.color, points)

# ====================================================================
# ============== SETUP ==============================================
open(LOG_FILE, "w").close()

creatures = []
for _ in range(INITIAL_HERBIVORES):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 0
    c.diet   = 0
    creatures.append(c)
for _ in range(INITIAL_PREDATORS):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 1
    c.diet   = 1
    creatures.append(c)

foods = [Food() for _ in range(INITIAL_FOOD)]

paused = False
frames = 0
sim_frames = 0
population_history = deque(maxlen=300)
pred_ratio_history = deque(maxlen=300)

death_log = {'starved': 0, 'aged': 0, 'hunted': 0}
mutation_tracker = Counter()

day_cycle_frame = 0
day_count = 0

fps_sample_timer = 0.0
avg_frame_time = 0.0

running = True

# ====================================================================
while running:
    dt = clock.tick(30)
    frames += 1

    fps_sample_timer += dt
    avg_frame_time = 0.9 * avg_frame_time + 0.1 * dt

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused

    # day-night cycle only advances if not paused
    if not paused:
        sim_frames += 1
        day_cycle_frame = (day_cycle_frame + 1) % DAY_LENGTH
        if day_cycle_frame == 0:
            day_count += 1

    alpha = day_cycle_frame / (DAY_LENGTH - 1) if (DAY_LENGTH>1) else 1.0
    if alpha < 0.5:
        sub_a = alpha / 0.5
        bg_color = interpolate_color(NIGHT_COLOR, DAY_COLOR, sub_a)
        is_day = True
    else:
        sub_a = (alpha - 0.5) / 0.5
        bg_color = interpolate_color(DAY_COLOR, NIGHT_COLOR, sub_a)
        is_day = False

    screen.fill(bg_color)

    if not paused:
        new_creatures = []
        dead = []

        # ============= MAIN UPDATE =============
        for c in creatures:
            status, other = c.update(creatures, foods)
            if status == "starved":
                death_log["starved"] += 1
                write_log(f"Death: starved (speed={c.speed:.2f})")
                dead.append(c)
                # spawn multiple food based on c's size
                food_chunks = 1 + int(c.size_factor*2)
                for _ in range(food_chunks):
                    if random.random()<0.3:  # or 100% if you prefer
                        foods.append(Food())

            elif status == "aged":
                death_log["aged"] += 1
                write_log(f"Death: aged (speed={c.speed:.2f})")
                dead.append(c)
                # spawn multiple food from corpse
                food_chunks = 1 + int(c.size_factor*2)
                for _ in range(food_chunks):
                    if random.random()<0.3:
                        foods.append(Food())

            elif status == "ate_creature":
                # predator kill
                death_log["hunted"] += 1
                if other is c:
                    # c lost the fight
                    write_log(f"Death: hunted (speed={c.speed:.2f}) [lost predator fight]")
                    dead.append(c)
                    # bigger corpse => more food
                    food_chunks = 1 + int(c.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
                else:
                    write_log(f"Death: hunted (speed={other.speed:.2f})")
                    dead.append(other)
                    # bigger corpse => more food
                    food_chunks = 1 + int(other.size_factor*2)
                    for _ in range(food_chunks):
                        if random.random()<0.3:
                            foods.append(Food())
            else:
                # alive => possible reproduction
                child = c.reproduce(creatures, foods)
                if child and len(creatures) < MAX_CREATURES:
                    mut_type = child._get_mutation_type(c.dna)
                    if mut_type not in ("initial", "minor"):
                        mutation_tracker[mut_type] += 1
                    new_creatures.append(child)
                    write_log(
                        f"Birth: speed={child.speed:.2f}, diet={child.diet}, size={child.size_factor:.2f}, "
                        f"fear={child.fear:.2f}, longevity={child.longevity:.2f}, mutation={mut_type}"
                    )

        # remove dead
        creatures = [cr for cr in creatures if cr not in dead]
        # add newborns
        space_left = max(0, MAX_CREATURES - len(creatures))
        creatures += new_creatures[:space_left]

        # ============= COLLISION RESOLUTION =============
        resolve_collisions(creatures)

        # track population
        pop_size = len(creatures)
        pred_count = sum(1 for cr in creatures if cr.diet == 1)
        population_history.append(pop_size)
        ratio = pred_count / pop_size if pop_size>0 else 0
        pred_ratio_history.append(ratio)

        # day-based extra food
        if is_day:
            if random.random() < DAYTIME_FOOD_SPAWN_CHANCE:
                foods.append(Food())

    # ============= DRAWING =============
    for f in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(f.x), int(f.y)), 4)
    for cr in creatures:
        cr.draw(screen)

    # ============= HUD =============
    pred_count = sum(1 for x in creatures if x.diet == 1)
    herb_count = len(creatures) - pred_count
    ratio_str = "∞"
    if pred_count > 0:
        ratio_val = herb_count / pred_count
        ratio_str = f"{ratio_val:.1f}"

    stats = [
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Predators: {pred_count}, ratio=1:{ratio_str}",
        f"Food: {len(foods)}",
        f"Day Count: {day_count}",
        f"Sim Time: {sim_frames}",
    ]
    if creatures:
        avg_speed = np.mean([c.speed for c in creatures])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']}  aged:{death_log['aged']}  hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    for i, text in enumerate(stats):
        txt = font.render(text, True, (255,255,255))
        screen.blit(txt, (10, 10 + i*25))

    # mutation tracker
    mut_top3 = mutation_tracker.most_common(3)
    if mut_top3:
        mut_strs = [f"{mt}:{cnt}" for (mt, cnt) in mut_top3]
        mut_line = "Mutations: " + ", ".join(mut_strs)
        txtm = font.render(mut_line, True, (255,255,255))
        screen.blit(txtm, (10, 10 + len(stats)*25))

    # FPS 
    fps_txt = f"FPS: {int(clock.get_fps())}"
    ms_txt = f"FrameTime: {avg_frame_time:.1f}ms"
    fps_surf = font.render(fps_txt, True, (255,255,255))
    ms_surf = font.render(ms_txt, True, (255,255,255))
    screen.blit(fps_surf, (1050, 10))
    screen.blit(ms_surf, (1050, 35))

    # Population graph
    graph_x, graph_y = 800, 70
    graph_w, graph_h = 380, 90
    pygame.draw.rect(screen, (50,50,50), (graph_x, graph_y, graph_w, graph_h))

    if len(population_history) > 1:
        points_pop = []
        for i, val in enumerate(population_history):
            xx = graph_x + graph_w - (len(population_history) - i)*2
            yy = graph_y + graph_h - (val / MAX_CREATURES) * graph_h
            points_pop.append((xx, yy))
        pygame.draw.lines(screen, GRAPH_COLOR, False, points_pop, 2)

    if len(pred_ratio_history) > 1:
        points_pred = []
        for i, ratio_val in enumerate(pred_ratio_history):
            xx = graph_x + graph_w - (len(pred_ratio_history) - i)*2
            yy = graph_y + graph_h - ratio_val * graph_h
            points_pred.append((xx, yy))
        pygame.draw.lines(screen, PREDATOR_GRAPH_COLOR, False, points_pred, 2)

    pygame.display.flip()

pygame.quit()


In [ ]:
# 0.4.6 predators died too fast
# 0.4.6
import pygame
import numpy as np
import random
from collections import deque, Counter
from datetime import datetime

# =================== CONFIG ====================
MAX_CREATURES = 2000

# Roughly 10% predators:
INITIAL_HERBIVORES = 180
INITIAL_PREDATORS  = 20

INITIAL_FOOD = 40

AGE_BASE = 700  # base lifespan, scaled by longevity gene

# If predators die off too quickly, raise this (e.g., 70):
ENERGY_GAIN_FROM_PREY = 70  
ENERGY_GAIN_FROM_FOOD = 22

BASE_ENERGY_LOSS_PER_MOVE = 0.05
SIZE_ENERGY_LOSS_FACTOR   = 0.04

REPRODUCE_ENERGY_COST = 50
ENV_REPRO_CHANCE_BASE = 0.3
ENV_REPRO_FOOD_FACTOR = 0.2
ENV_REPRO_POP_FACTOR  = 0.2

MUTATION_RATE = 0.1
DIET_FLIP_CHANCE = 0.0   # No crossing from predator to herb
FEAR_MUTATION_CHANCE = 0.2
SIZE_MUTATION_CHANCE = 0.2
LONGEVITY_MUTATION_CHANCE = 0.2

MAX_SPEED = 8.0
MIN_SPEED = 0.5
MIN_SIZE  = 0.5
MAX_SIZE  = 2.5

FEAR_COST_BASE = 0.02
FEAR_THRESHOLD_HERB = 0.3

AMBUSH_BONUS = 1.1  
PRED_FEAR_THRESHOLD = 0.5   
SIZE_DOMINANCE_RATIO = 1.2  
HERB_SIZE_DOMINANCE_RATIO = 1.2

SIGHT_RANGE = 200.0
BLOCKING_RADIUS = 5.0

# Faster day/night cycle
DAY_LENGTH = 240
# Brighter pale blue for daytime:
DAY_COLOR   = (170, 220, 255)
NIGHT_COLOR = (0, 0, 0)

DAYTIME_FOOD_SPAWN_CHANCE = 0.05

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Colors
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)
PREDATOR_GRAPH_COLOR = (150, 100, 200)

# ============== HELPERS ============================
def write_log(message):
    """Write to the log file every LOG_INTERVAL 'sim_frames'."""
    if sim_frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {sim_frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return np.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    ABx = bx - ax
    ABy = by - ay
    APx = px - ax
    APy = py - ay
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        return distance(px, py, ax, ay)
    t = (APx*ABx + APy*ABy) / AB_len_sq
    t = max(0, min(1, t))
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    """Linear interpolate between two RGB tuples, alpha in [0..1]."""
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

# ============== CLASSES ============================
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    dna = [speed, r, g, b, diet, fear, size_factor, longevity]
      - If father is predator, child is also predator (since diet_flip=0).
      - Reproduction "carries father's traits."
    """
    def __init__(self, x, y, father_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        self.heading = random.random() * 2.0 * np.pi

        if father_dna:
            speed = np.clip(father_dna[0] * random.uniform(0.9, 1.1), MIN_SPEED, MAX_SPEED)
            r     = np.clip(father_dna[1] + random.randint(-15, 15), 50, 255)
            g     = np.clip(father_dna[2] + random.randint(-15, 15), 50, 255)
            b     = np.clip(father_dna[3] + random.randint(-15, 15), 50, 255)

            diet  = father_dna[4]  # no flipping
            fear  = father_dna[5]
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            size_factor = father_dna[6]
            if random.random() < SIZE_MUTATION_CHANCE:
                size_factor *= random.uniform(0.8, 1.2)
                size_factor = min(max(MIN_SIZE, size_factor), MAX_SIZE)

            longevity = father_dna[7]
            if random.random() < LONGEVITY_MUTATION_CHANCE:
                longevity *= random.uniform(0.8, 1.2)
                longevity = min(max(0.5, longevity), 2.0)

            self.dna = [speed, r, g, b, diet, fear, size_factor, longevity]

            # Big speed shift chance
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], MIN_SPEED, MAX_SPEED)
        else:
            # brand-new random
            self.dna = [
                random.uniform(1.5, 3.0),
                random.randint(50, 255),
                random.randint(50, 255),
                random.randint(50, 255),
                0,  # default diet=0
                random.uniform(0.0, 0.2),
                random.uniform(MIN_SIZE, 1.2),
                random.uniform(0.8, 1.2)
            ]

        self.speed       = self.dna[0]
        self.color       = (self.dna[1], self.dna[2], self.dna[3])
        self.diet        = int(self.dna[4])
        self.fear        = self.dna[5]
        self.size_factor = self.dna[6]
        self.longevity   = self.dna[7]

    def _get_mutation_type(self, father_dna):
        if not father_dna:
            return "initial"
        changes = []
        if abs(self.dna[0] - father_dna[0]) > 1.0:
            changes.append("speed_shift")
        c_diff = abs(self.dna[1] - father_dna[1]) + \
                 abs(self.dna[2] - father_dna[2]) + \
                 abs(self.dna[3] - father_dna[3])
        if c_diff > 45:
            changes.append("color_shift")
        # no diet_flip (because DIET_FLIP_CHANCE=0)
        if abs(self.dna[5] - father_dna[5]) > 0.05:
            changes.append("fear_shift")
        if abs(self.dna[6] - father_dna[6]) > 0.3:
            changes.append("size_shift")
        if abs(self.dna[7] - father_dna[7]) > 0.3:
            changes.append("long_shift")

        if not changes:
            return "minor"
        return "+".join(changes)

    def max_age(self):
        return AGE_BASE * self.longevity

    def strength(self):
        return self.size_factor * (1.0 + 0.5*(self.energy/100.0))

    def max_energy_capacity(self):
        return 100 + (self.size_factor - 1.0)*50

    def update(self, all_creatures, foods):
        self.age += 1

        # Fear cost
        if self.diet == 0 and self.fear > FEAR_THRESHOLD_HERB:
            self.energy -= FEAR_COST_BASE * self.fear * self.size_factor
        elif self.diet == 1 and self.fear > PRED_FEAR_THRESHOLD:
            self.energy -= FEAR_COST_BASE * self.fear * (self.size_factor * 0.5)

        # Size-based energy drain
        self.energy -= BASE_ENERGY_LOSS_PER_MOVE
        self.energy -= (self.size_factor * SIZE_ENERGY_LOSS_FACTOR)
        self.energy = max(self.energy, 0)

        if self.diet == 0:
            # ========== HERBIVORE ==========
            if self.fear > FEAR_THRESHOLD_HERB:
                predator, distp = self.find_nearest(all_creatures, lambda c: c is not self and c.diet == 1)
                if predator and distp < 120:
                    self.run_away(predator.x, predator.y)
                    self.wrap_screen()
                    return self.check_survival()

            # look for food
            ftarget, distf = self.find_nearest(foods)
            if ftarget and distf < SIGHT_RANGE:
                if not self.is_line_blocked(ftarget.x, ftarget.y, all_creatures):
                    self.move_toward(ftarget.x, ftarget.y)
                    if distance(self.x, self.y, ftarget.x, ftarget.y) < (6 + self.size_factor*4):
                        self.energy = min(self.max_energy_capacity(),
                                          self.energy + ENERGY_GAIN_FROM_FOOD)
                        foods.remove(ftarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        else:
            # ========== PREDATOR ==========
            ctarget, distc = self.find_nearest(all_creatures, lambda c: c is not self)
            if ctarget and distc < SIGHT_RANGE:
                if not self.is_line_blocked(ctarget.x, ctarget.y, all_creatures):
                    if ctarget.diet == 1:
                        # predator vs predator
                        if self.fear > PRED_FEAR_THRESHOLD and \
                           (ctarget.size_factor > self.size_factor * SIZE_DOMINANCE_RATIO):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                return self.predator_fight(ctarget)
                    else:
                        # predator vs herb
                        if (ctarget.size_factor > self.size_factor * HERB_SIZE_DOMINANCE_RATIO) \
                           and (self.fear > PRED_FEAR_THRESHOLD):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                self.energy = min(self.max_energy_capacity(),
                                                  self.energy + ENERGY_GAIN_FROM_PREY)
                                return ("ate_creature", ctarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def predator_fight(self, other):
        if random.random() < 0.3:
            my_str = self.strength() * AMBUSH_BONUS
            their_str = other.strength()
        else:
            my_str = self.strength()
            their_str = other.strength()

        if my_str > their_str:
            self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_PREY)
            return ("ate_creature", other)
        elif their_str > my_str:
            return ("ate_creature", self)
        else:
            if random.random() < 0.5:
                return ("ate_creature", other)
            else:
                return ("ate_creature", self)

    def check_survival(self):
        if self.energy <= 0:
            return ("starved", None)
        if self.age > self.max_age():
            return ("aged", None)
        return ("alive", None)

    def reproduce(self, creatures, foods):
        """If father has enough energy, pass father DNA to child."""
        if self.energy < 80:
            return None
        pop_size = len(creatures)
        food_count = len(foods)
        ratio_food_pop = food_count / pop_size if pop_size>0 else 1
        ratio_pop_cap  = pop_size / MAX_CREATURES

        chance = ENV_REPRO_CHANCE_BASE
        chance += ENV_REPRO_FOOD_FACTOR * ratio_food_pop
        chance -= ENV_REPRO_POP_FACTOR  * ratio_pop_cap
        chance = max(0.0, min(1.0, chance))

        if random.random() < chance:
            self.energy -= REPRODUCE_ENERGY_COST
            # spawn child near father
            offx = random.gauss(0, 10)
            offy = random.gauss(0, 10)
            child = Creature(self.x + offx, self.y + offy, self.dna)
            return child
        return None

    def random_walk(self):
        self.heading += random.uniform(-0.2, 0.2)
        dx = np.cos(self.heading) * self.speed
        dy = np.sin(self.heading) * self.speed
        self.x += dx
        self.y += dy

    def run_away(self, ox, oy):
        dx = self.x - ox
        dy = self.y - oy
        distv = distance(self.x, self.y, ox, oy)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def move_toward(self, tx, ty):
        dx = tx - self.x
        dy = ty - self.y
        distv = distance(self.x, self.y, tx, ty)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def wrap_screen(self):
        """Toroidal wrapping (left <-> right, top <-> bottom)."""
        self.x %= 1200
        self.y %= 800

    def find_nearest(self, target_list, condition=None):
        valids = target_list
        if condition:
            valids = [t for t in target_list if condition(t)]
        if not valids:
            return (None, float('inf'))
        best_obj, best_dist = None, float('inf')
        for obj in valids:
            d = distance(self.x, self.y, obj.x, obj.y)
            if d < best_dist:
                best_dist = d
                best_obj = obj
        return (best_obj, best_dist)

    def is_line_blocked(self, tx, ty, creatures):
        targ_dist = distance(self.x, self.y, tx, ty)
        for c in creatures:
            if c is self:
                continue
            dist_c = distance(self.x, self.y, c.x, c.y)
            if dist_c >= targ_dist:
                continue
            d_line = line_distance_point(c.x, c.y, self.x, self.y, tx, ty)
            if d_line < BLOCKING_RADIUS:
                return True
        return False

    def draw(self, surface):
        sz = int(8 * self.size_factor)
        if self.diet == 0:
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), sz)
        else:
            half_w = sz
            points = [
                (self.x, self.y - sz),
                (self.x - half_w, self.y + sz),
                (self.x + half_w, self.y + sz)
            ]
            pygame.draw.polygon(surface, self.color, points)

# ============== SETUP ==============================
open(LOG_FILE, "w").close()

creatures = []
for _ in range(INITIAL_HERBIVORES):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 0
    c.diet   = 0
    creatures.append(c)
for _ in range(INITIAL_PREDATORS):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 1
    c.diet   = 1
    creatures.append(c)

foods = [Food() for _ in range(INITIAL_FOOD)]

paused = False
frames = 0     # rendered frames
sim_frames = 0 # only increments when not paused
population_history = deque(maxlen=300)
pred_ratio_history = deque(maxlen=300)

death_log = {'starved': 0, 'aged': 0, 'hunted': 0}
mutation_tracker = Counter()

day_cycle_frame = 0
day_count = 0

fps_sample_timer = 0.0
avg_frame_time = 0.0

running = True

while running:
    dt = clock.tick(30)
    frames += 1

    fps_sample_timer += dt
    avg_frame_time = 0.9 * avg_frame_time + 0.1 * dt

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused

    # day-night cycle only advances if not paused
    if not paused:
        sim_frames += 1

        day_cycle_frame = (day_cycle_frame + 1) % DAY_LENGTH
        if day_cycle_frame == 0:
            day_count += 1

    # compute alpha
    alpha = day_cycle_frame / (DAY_LENGTH - 1) if (DAY_LENGTH > 1) else 1.0
    if alpha < 0.5:
        sub_a = alpha / 0.5
        bg_color = interpolate_color(NIGHT_COLOR, DAY_COLOR, sub_a)
        is_day = True
    else:
        sub_a = (alpha - 0.5) / 0.5
        bg_color = interpolate_color(DAY_COLOR, NIGHT_COLOR, sub_a)
        is_day = False

    screen.fill(bg_color)

    if not paused:
        new_creatures = []
        dead = []

        for c in creatures:
            status, other = c.update(creatures, foods)
            if status == "starved":
                death_log["starved"] += 1
                write_log(f"Death: starved (speed={c.speed:.2f})")
                dead.append(c)
                if random.random() < 0.3:
                    foods.append(Food())

            elif status == "aged":
                death_log["aged"] += 1
                write_log(f"Death: aged (speed={c.speed:.2f})")
                dead.append(c)
                if random.random() < 0.3:
                    foods.append(Food())

            elif status == "ate_creature":
                # predator kill
                death_log["hunted"] += 1
                if other is c:
                    # c lost the fight
                    write_log(f"Death: hunted (speed={c.speed:.2f}) [lost predator fight]")
                    dead.append(c)
                else:
                    write_log(f"Death: hunted (speed={other.speed:.2f})")
                    dead.append(other)

            else:
                # alive => reproduction
                baby = c.reproduce(creatures, foods)
                if baby and len(creatures) < MAX_CREATURES:
                    mut_type = baby._get_mutation_type(c.dna)
                    if mut_type not in ("initial", "minor"):
                        mutation_tracker[mut_type] += 1
                    new_creatures.append(baby)
                    write_log(
                        f"Birth: speed={baby.speed:.2f}, diet={baby.diet}, size={baby.size_factor:.2f}, "
                        f"fear={baby.fear:.2f}, longevity={baby.longevity:.2f}, mutation={mut_type}"
                    )

        creatures = [cr for cr in creatures if cr not in dead]
        space_left = max(0, MAX_CREATURES - len(creatures))
        creatures += new_creatures[:space_left]

        # track population and ratio
        pop_size = len(creatures)
        pred_count = sum(1 for cr in creatures if cr.diet == 1)
        population_history.append(pop_size)
        ratio = pred_count / pop_size if pop_size>0 else 0
        pred_ratio_history.append(ratio)

        # day-based food spawn
        if is_day:
            if random.random() < DAYTIME_FOOD_SPAWN_CHANCE:
                foods.append(Food())

    # drawing
    for f in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(f.x), int(f.y)), 4)
    for cr in creatures:
        cr.draw(screen)

    # HUD (top-left)
    pred_count = sum(1 for x in creatures if x.diet == 1)
    herb_count = len(creatures) - pred_count
    ratio_str = "∞"
    if pred_count > 0:
        ratio_val = herb_count / pred_count
        ratio_str = f"{ratio_val:.1f}"

    stats = [
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Predators: {pred_count}, ratio=1:{ratio_str}",
        f"Food: {len(foods)}",
        f"Day Count: {day_count}",
        f"Sim Time: {sim_frames}",
    ]
    if creatures:
        avg_speed = np.mean([c.speed for c in creatures])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']}  aged:{death_log['aged']}  hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    for i, text in enumerate(stats):
        txt = font.render(text, True, (255, 255, 255))
        screen.blit(txt, (10, 10 + i*25))

    # mutation tracker (top-left, below stats)
    mut_top3 = mutation_tracker.most_common(3)
    if mut_top3:
        mut_strs = [f"{mt}:{cnt}" for (mt, cnt) in mut_top3]
        mut_line = "Mutations: " + ", ".join(mut_strs)
        txtm = font.render(mut_line, True, (255, 255, 255))
        screen.blit(txtm, (10, 10 + len(stats)*25))

    # FPS (top-right)
    fps_txt = f"FPS: {int(clock.get_fps())}"
    ms_txt = f"FrameTime: {avg_frame_time:.1f}ms"
    fps_surf = font.render(fps_txt, True, (255,255,255))
    ms_surf = font.render(ms_txt, True, (255,255,255))
    screen.blit(fps_surf, (1050, 10))
    screen.blit(ms_surf, (1050, 35))

    # Population graph
    graph_x, graph_y = 800, 70
    graph_w, graph_h = 380, 90
    pygame.draw.rect(screen, (50,50,50), (graph_x, graph_y, graph_w, graph_h))

    if len(population_history) > 1:
        # total pop
        points_pop = []
        for i, val in enumerate(population_history):
            xx = graph_x + graph_w - (len(population_history) - i)*2
            yy = graph_y + graph_h - (val / MAX_CREATURES) * graph_h
            points_pop.append((xx, yy))
        pygame.draw.lines(screen, GRAPH_COLOR, False, points_pop, 2)

    if len(pred_ratio_history) > 1:
        # predator ratio line => ratio in [0..1]
        points_pred = []
        for i, ratio_val in enumerate(pred_ratio_history):
            xx = graph_x + graph_w - (len(pred_ratio_history) - i)*2
            yy = graph_y + graph_h - ratio_val * graph_h
            points_pred.append((xx, yy))
        pygame.draw.lines(screen, PREDATOR_GRAPH_COLOR, False, points_pred, 2)

    pygame.display.flip()

pygame.quit()


In [ ]:
# 0.4.5
import pygame
import numpy as np
import random
from collections import deque, Counter
from datetime import datetime

# =================== CONFIG ====================
MAX_CREATURES = 2000

# Start ratio
INITIAL_HERBIVORES = 200
INITIAL_PREDATORS  = 5

INITIAL_FOOD = 40

AGE_BASE = 700    # base lifespan, scaled by longevity gene

ENERGY_GAIN_FROM_FOOD = 22
ENERGY_GAIN_FROM_PREY = 50

BASE_ENERGY_LOSS_PER_MOVE = 0.05
SIZE_ENERGY_LOSS_FACTOR   = 0.04

REPRODUCE_ENERGY_COST = 50
ENV_REPRO_CHANCE_BASE = 0.3
ENV_REPRO_FOOD_FACTOR = 0.2
ENV_REPRO_POP_FACTOR  = 0.2

MUTATION_RATE = 0.1
# Set diet flip chance to 0.0 => no crossing from predator to herbivore (and vice versa)
DIET_FLIP_CHANCE = 0.0
FEAR_MUTATION_CHANCE = 0.2
SIZE_MUTATION_CHANCE = 0.2
LONGEVITY_MUTATION_CHANCE = 0.2

MAX_SPEED = 8.0
MIN_SPEED = 0.5
MIN_SIZE  = 0.5
MAX_SIZE  = 2.5

FEAR_COST_BASE = 0.02
FEAR_THRESHOLD_HERB = 0.3

AMBUSH_BONUS = 1.1  
PRED_FEAR_THRESHOLD = 0.5   
SIZE_DOMINANCE_RATIO = 1.2  
HERB_SIZE_DOMINANCE_RATIO = 1.2

SIGHT_RANGE = 200.0
BLOCKING_RADIUS = 5.0

# Faster day/night cycle
DAY_LENGTH = 240   # was 600, now 4x faster
DAY_COLOR   = (0, 0, 50)
NIGHT_COLOR = (0, 0, 0)

DAYTIME_FOOD_SPAWN_CHANCE = 0.05

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Colors
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)
# We'll show ratio line in a different color
PREDATOR_GRAPH_COLOR = (150, 100, 200)

# ============== HELPERS ============================
def write_log(message):
    """Write to the log file every LOG_INTERVAL 'sim_frames'."""
    if sim_frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {sim_frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return np.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    ABx = bx - ax
    ABy = by - ay
    APx = px - ax
    APy = py - ay
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        return distance(px, py, ax, ay)
    t = (APx*ABx + APy*ABy) / AB_len_sq
    t = max(0, min(1, t))
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    """Linear interpolate between two RGB tuples, alpha in [0..1]."""
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

# ============== CLASSES ============================
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    dna = [speed, r, g, b, diet, fear, size_factor, longevity]
      - diet: 0=herb, 1=pred
      - fear: 0..1
      - size_factor: [0.5..2.5]
      - longevity: [0.5..2.0]

    Children spawn with a small offset from parent's position,
    so they don't appear exactly on top.
    """
    def __init__(self, x, y, parent_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        self.heading = random.random() * 2.0 * np.pi

        if parent_dna:
            speed = np.clip(parent_dna[0] * random.uniform(0.9, 1.1), MIN_SPEED, MAX_SPEED)
            r     = np.clip(parent_dna[1] + random.randint(-15, 15), 50, 255)
            g     = np.clip(parent_dna[2] + random.randint(-15, 15), 50, 255)
            b     = np.clip(parent_dna[3] + random.randint(-15, 15), 50, 255)

            diet  = parent_dna[4]  # no flipping
            fear  = parent_dna[5]
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            size_factor = parent_dna[6]
            if random.random() < SIZE_MUTATION_CHANCE:
                size_factor *= random.uniform(0.8, 1.2)
                size_factor = min(max(MIN_SIZE, size_factor), MAX_SIZE)

            longevity = parent_dna[7]
            if random.random() < LONGEVITY_MUTATION_CHANCE:
                longevity *= random.uniform(0.8, 1.2)
                longevity = min(max(0.5, longevity), 2.0)

            self.dna = [speed, r, g, b, diet, fear, size_factor, longevity]

            # Big speed shift chance
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], MIN_SPEED, MAX_SPEED)
        else:
            # brand-new
            self.dna = [
                random.uniform(1.5, 3.0),
                random.randint(50, 255),
                random.randint(50, 255),
                random.randint(50, 255),
                0,  # default diet=0
                random.uniform(0.0, 0.2),
                random.uniform(MIN_SIZE, 1.2),
                random.uniform(0.8, 1.2)
            ]

        self.speed       = self.dna[0]
        self.color       = (self.dna[1], self.dna[2], self.dna[3])
        self.diet        = int(self.dna[4])
        self.fear        = self.dna[5]
        self.size_factor = self.dna[6]
        self.longevity   = self.dna[7]

    def _get_mutation_type(self, parent_dna):
        if not parent_dna:
            return "initial"
        changes = []
        if abs(self.dna[0] - parent_dna[0]) > 1.0:
            changes.append("speed_shift")
        c_diff = abs(self.dna[1] - parent_dna[1]) + abs(self.dna[2] - parent_dna[2]) + abs(self.dna[3] - parent_dna[3])
        if c_diff > 45:
            changes.append("color_shift")
        # since no flipping, no diet_flip
        if abs(self.dna[5] - parent_dna[5]) > 0.05:
            changes.append("fear_shift")
        if abs(self.dna[6] - parent_dna[6]) > 0.3:
            changes.append("size_shift")
        if abs(self.dna[7] - parent_dna[7]) > 0.3:
            changes.append("long_shift")

        if not changes:
            return "minor"
        return "+".join(changes)

    def max_age(self):
        return AGE_BASE * self.longevity

    def strength(self):
        return self.size_factor * (1.0 + 0.5*(self.energy/100.0))

    def max_energy_capacity(self):
        return 100 + (self.size_factor - 1.0)*50

    def update(self, all_creatures, foods):
        self.age += 1

        # Fear cost
        if self.diet == 0 and self.fear > FEAR_THRESHOLD_HERB:
            self.energy -= FEAR_COST_BASE * self.fear * self.size_factor
        elif self.diet == 1 and self.fear > PRED_FEAR_THRESHOLD:
            self.energy -= FEAR_COST_BASE * self.fear * (self.size_factor * 0.5)

        # Size-based energy drain
        self.energy -= BASE_ENERGY_LOSS_PER_MOVE
        self.energy -= (self.size_factor * SIZE_ENERGY_LOSS_FACTOR)
        self.energy = max(self.energy, 0)

        if self.diet == 0:
            # ========== HERBIVORE ==========
            if self.fear > FEAR_THRESHOLD_HERB:
                predator, distp = self.find_nearest(all_creatures, lambda c: c is not self and c.diet == 1)
                if predator and distp < 120:
                    self.run_away(predator.x, predator.y)
                    self.wrap_screen()
                    return self.check_survival()

            # look for food
            ftarget, distf = self.find_nearest(foods)
            if ftarget and distf < SIGHT_RANGE:
                if not self.is_line_blocked(ftarget.x, ftarget.y, all_creatures):
                    self.move_toward(ftarget.x, ftarget.y)
                    if distance(self.x, self.y, ftarget.x, ftarget.y) < (6 + self.size_factor*4):
                        self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_FOOD)
                        foods.remove(ftarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        else:
            # ========== PREDATOR ==========
            ctarget, distc = self.find_nearest(all_creatures, lambda c: c is not self)
            if ctarget and distc < SIGHT_RANGE:
                if not self.is_line_blocked(ctarget.x, ctarget.y, all_creatures):
                    if ctarget.diet == 1:
                        # predator vs. predator
                        if self.fear > PRED_FEAR_THRESHOLD and (ctarget.size_factor > self.size_factor * SIZE_DOMINANCE_RATIO):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                return self.predator_fight(ctarget)
                    else:
                        # predator vs. herb
                        if (ctarget.size_factor > self.size_factor * HERB_SIZE_DOMINANCE_RATIO) and (self.fear > PRED_FEAR_THRESHOLD):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_PREY)
                                return ("ate_creature", ctarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def predator_fight(self, other):
        if random.random() < 0.3:
            my_str = self.strength() * AMBUSH_BONUS
            their_str = other.strength()
        else:
            my_str = self.strength()
            their_str = other.strength()

        if my_str > their_str:
            self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_PREY)
            return ("ate_creature", other)
        elif their_str > my_str:
            return ("ate_creature", self)
        else:
            if random.random() < 0.5:
                return ("ate_creature", other)
            else:
                return ("ate_creature", self)

    def check_survival(self):
        if self.energy <= 0:
            return ("starved", None)
        if self.age > self.max_age():
            return ("aged", None)
        return ("alive", None)

    def reproduce(self, creatures, foods):
        if self.energy < 80:
            return None
        pop_size = len(creatures)
        food_count = len(foods)
        ratio_food_pop = food_count / pop_size if pop_size>0 else 1
        ratio_pop_cap  = pop_size / MAX_CREATURES

        chance = ENV_REPRO_CHANCE_BASE
        chance += ENV_REPRO_FOOD_FACTOR * ratio_food_pop
        chance -= ENV_REPRO_POP_FACTOR  * ratio_pop_cap
        chance = max(0.0, min(1.0, chance))

        if random.random() < chance:
            self.energy -= REPRODUCE_ENERGY_COST
            # Spawn child with slight offset
            offx = random.gauss(0, 10)
            offy = random.gauss(0, 10)
            child = Creature(self.x + offx, self.y + offy, self.dna)
            return child
        return None

    def random_walk(self):
        self.heading += random.uniform(-0.2, 0.2)
        dx = np.cos(self.heading) * self.speed
        dy = np.sin(self.heading) * self.speed
        self.x += dx
        self.y += dy

    def run_away(self, ox, oy):
        dx = self.x - ox
        dy = self.y - oy
        distv = distance(self.x, self.y, ox, oy)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def move_toward(self, tx, ty):
        dx = tx - self.x
        dy = ty - self.y
        distv = distance(self.x, self.y, tx, ty)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def wrap_screen(self):
        self.x %= 1200
        self.y %= 800

    def find_nearest(self, target_list, condition=None):
        valids = target_list
        if condition:
            valids = [t for t in target_list if condition(t)]
        if not valids:
            return (None, float('inf'))
        best_obj, best_dist = None, float('inf')
        for obj in valids:
            d = distance(self.x, self.y, obj.x, obj.y)
            if d < best_dist:
                best_dist = d
                best_obj = obj
        return (best_obj, best_dist)

    def is_line_blocked(self, tx, ty, creatures):
        targ_dist = distance(self.x, self.y, tx, ty)
        for c in creatures:
            if c is self:
                continue
            dist_c = distance(self.x, self.y, c.x, c.y)
            if dist_c >= targ_dist:
                continue
            d_line = line_distance_point(c.x, c.y, self.x, self.y, tx, ty)
            if d_line < BLOCKING_RADIUS:
                return True
        return False

    def draw(self, surface):
        sz = int(8 * self.size_factor)
        if self.diet == 0:
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), sz)
        else:
            half_w = sz
            points = [
                (self.x, self.y - sz),
                (self.x - half_w, self.y + sz),
                (self.x + half_w, self.y + sz)
            ]
            pygame.draw.polygon(surface, self.color, points)

# ============== SETUP ==============================
open(LOG_FILE, "w").close()

creatures = []
for _ in range(INITIAL_HERBIVORES):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 0
    c.diet   = 0
    creatures.append(c)
for _ in range(INITIAL_PREDATORS):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 1
    c.diet   = 1
    creatures.append(c)

foods = [Food() for _ in range(INITIAL_FOOD)]

paused = False
frames = 0     # rendered frames
sim_frames = 0 # only increments when not paused
population_history = deque(maxlen=300)
pred_ratio_history = deque(maxlen=300)

death_log = {'starved': 0, 'aged': 0, 'hunted': 0}
mutation_tracker = Counter()

day_cycle_frame = 0
day_count = 0

fps_sample_timer = 0.0
avg_frame_time = 0.0

running = True

while running:
    dt = clock.tick(30)
    frames += 1

    fps_sample_timer += dt
    avg_frame_time = 0.9 * avg_frame_time + 0.1 * dt

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused

    # day-night cycle only advances if not paused
    if not paused:
        sim_frames += 1

        day_cycle_frame = (day_cycle_frame + 1) % DAY_LENGTH
        if day_cycle_frame == 0:
            day_count += 1

    # compute alpha
    alpha = day_cycle_frame / (DAY_LENGTH - 1) if (DAY_LENGTH > 1) else 1.0
    if alpha < 0.5:
        sub_a = alpha / 0.5
        bg_color = interpolate_color(NIGHT_COLOR, DAY_COLOR, sub_a)
        is_day = True
    else:
        sub_a = (alpha - 0.5) / 0.5
        bg_color = interpolate_color(DAY_COLOR, NIGHT_COLOR, sub_a)
        is_day = False

    screen.fill(bg_color)

    if not paused:
        new_creatures = []
        dead = []

        for c in creatures:
            status, other = c.update(creatures, foods)
            if status == "starved":
                death_log["starved"] += 1
                write_log(f"Death: starved (speed={c.speed:.2f})")
                dead.append(c)
                if random.random() < 0.3:
                    foods.append(Food())

            elif status == "aged":
                death_log["aged"] += 1
                write_log(f"Death: aged (speed={c.speed:.2f})")
                dead.append(c)
                if random.random() < 0.3:
                    foods.append(Food())

            elif status == "ate_creature":
                # predator kill
                death_log["hunted"] += 1
                if other is c:
                    # c lost the fight
                    write_log(f"Death: hunted (speed={c.speed:.2f}) [lost predator fight]")
                    dead.append(c)
                else:
                    write_log(f"Death: hunted (speed={other.speed:.2f})")
                    dead.append(other)

            else:
                # alive => reproduction
                baby = c.reproduce(creatures, foods)
                if baby and len(creatures) < MAX_CREATURES:
                    mut_type = baby._get_mutation_type(c.dna)
                    if mut_type not in ("initial", "minor"):
                        mutation_tracker[mut_type] += 1
                    new_creatures.append(baby)
                    write_log(
                        f"Birth: speed={baby.speed:.2f}, diet={baby.diet}, size={baby.size_factor:.2f}, "
                        f"fear={baby.fear:.2f}, longevity={baby.longevity:.2f}, mutation={mut_type}"
                    )

        creatures = [cr for cr in creatures if cr not in dead]
        space_left = max(0, MAX_CREATURES - len(creatures))
        creatures += new_creatures[:space_left]

        # track population and ratio
        pop_size = len(creatures)
        pred_count = sum(1 for cr in creatures if cr.diet == 1)
        population_history.append(pop_size)
        ratio = pred_count / pop_size if pop_size>0 else 0
        pred_ratio_history.append(ratio)

        # day-based food spawn
        if is_day:
            if random.random() < DAYTIME_FOOD_SPAWN_CHANCE:
                foods.append(Food())

    # drawing
    for f in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(f.x), int(f.y)), 4)
    for cr in creatures:
        cr.draw(screen)

    # HUD (top-left)
    pred_count = sum(1 for x in creatures if x.diet == 1)
    herb_count = len(creatures) - pred_count
    ratio_str = "∞"
    if pred_count > 0:
        ratio_val = herb_count / pred_count
        ratio_str = f"{ratio_val:.1f}"

    stats = [
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Predators: {pred_count}, ratio=1:{ratio_str}",
        f"Food: {len(foods)}",
        f"Day Count: {day_count}",
        f"Sim Time: {sim_frames}",
    ]
    if creatures:
        avg_speed = np.mean([c.speed for c in creatures])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']}  aged:{death_log['aged']}  hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    for i, text in enumerate(stats):
        txt = font.render(text, True, (255, 255, 255))
        screen.blit(txt, (10, 10 + i*25))

    # mutation tracker (top-left, below stats)
    mut_top3 = mutation_tracker.most_common(3)
    if mut_top3:
        mut_strs = [f"{mt}:{cnt}" for (mt, cnt) in mut_top3]
        mut_line = "Mutations: " + ", ".join(mut_strs)
        txtm = font.render(mut_line, True, (255, 255, 255))
        screen.blit(txtm, (10, 10 + len(stats)*25))

    # FPS (top-right)
    fps_txt = f"FPS: {int(clock.get_fps())}"
    ms_txt = f"FrameTime: {avg_frame_time:.1f}ms"
    fps_surf = font.render(fps_txt, True, (255,255,255))
    ms_surf = font.render(ms_txt, True, (255,255,255))
    screen.blit(fps_surf, (1050, 10))
    screen.blit(ms_surf, (1050, 35))

    # Population graph
    graph_x, graph_y = 800, 70
    graph_w, graph_h = 380, 90
    pygame.draw.rect(screen, (50,50,50), (graph_x, graph_y, graph_w, graph_h))

    if len(population_history) > 1:
        # total pop
        points_pop = []
        for i, val in enumerate(population_history):
            xx = graph_x + graph_w - (len(population_history) - i)*2
            yy = graph_y + graph_h - (val / MAX_CREATURES) * graph_h
            points_pop.append((xx, yy))
        pygame.draw.lines(screen, GRAPH_COLOR, False, points_pop, 2)

    if len(pred_ratio_history) > 1:
        # predator ratio line => ratio in [0..1]
        points_pred = []
        for i, ratio_val in enumerate(pred_ratio_history):
            xx = graph_x + graph_w - (len(pred_ratio_history) - i)*2
            yy = graph_y + graph_h - ratio_val * graph_h
            points_pred.append((xx, yy))
        pygame.draw.lines(screen, PREDATOR_GRAPH_COLOR, False, points_pred, 2)

    pygame.display.flip()

pygame.quit()


In [ ]:
# 0.4.4 faster food, fear scale with size, predators avoid large herbivores, size dependent energy loss, pvp fight or flight
import pygame
import numpy as np
import random
from collections import deque, Counter
from datetime import datetime

# =================== CONFIG ====================
MAX_CREATURES = 2000

# Start ratio
INITIAL_HERBIVORES = 200
INITIAL_PREDATORS  = 1

INITIAL_FOOD = 40

AGE_BASE = 700    # base lifespan, scaled by longevity gene

ENERGY_GAIN_FROM_FOOD = 22
ENERGY_GAIN_FROM_PREY = 50

# We'll split the per-frame energy cost into a base + a size-based portion:
BASE_ENERGY_LOSS_PER_MOVE = 0.05    # lower base cost
SIZE_ENERGY_LOSS_FACTOR   = 0.04    # each unit of size_factor adds this

REPRODUCE_ENERGY_COST = 50
ENV_REPRO_CHANCE_BASE = 0.3   # baseline chance to attempt reproduction
ENV_REPRO_FOOD_FACTOR = 0.2   # bonus factor if there's lots of food
ENV_REPRO_POP_FACTOR  = 0.2   # penalty factor if population is high

MUTATION_RATE = 0.1
DIET_FLIP_CHANCE = 0.01
FEAR_MUTATION_CHANCE = 0.2
SIZE_MUTATION_CHANCE = 0.2
LONGEVITY_MUTATION_CHANCE = 0.2

MAX_SPEED = 8.0
MIN_SPEED = 0.5
MIN_SIZE  = 0.5
MAX_SIZE  = 2.5

# Fear
FEAR_COST_BASE = 0.02  # We'll multiply by fear * size_factor
FEAR_THRESHOLD_HERB = 0.3

# Predator <-> Predator
AMBUSH_BONUS = 1.1  # if the other doesn't notice in time
PRED_FEAR_THRESHOLD = 0.5   # if predator's fear > 0.5, it avoids bigger predators
SIZE_DOMINANCE_RATIO = 1.2  # if target's size is 20% bigger, the fearful predator flees

# Predator <-> Herbivore
HERB_SIZE_DOMINANCE_RATIO = 1.2  # if the herb is 20% bigger, a fearful predator won't attack

SIGHT_RANGE = 200.0
BLOCKING_RADIUS = 5.0

DAY_LENGTH = 600  # frames for one full day-night cycle
DAY_COLOR   = (0, 0, 50)
NIGHT_COLOR = (0, 0, 0)

# Faster daytime food spawn
DAYTIME_FOOD_SPAWN_CHANCE = 0.05  # 5% each "frame" of day

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# ============== COLORS / GRAPHICS =================
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)
PREDATOR_GRAPH_COLOR = (150, 100, 200)  # second color for predator line

# ============== HELPERS ============================
def write_log(message):
    """Write to the log file every LOG_INTERVAL 'sim_frames'."""
    if sim_frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {sim_frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return np.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    """
    Returns the shortest distance from point (px,py) to line segment A->B.
    Used for naive line-of-sight blocking.
    """
    ABx = bx - ax
    ABy = by - ay
    APx = px - ax
    APy = py - ay
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        return distance(px, py, ax, ay)
    t = (APx*ABx + APy*ABy) / AB_len_sq
    t = max(0, min(1, t))
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    """Linear interpolate between two RGB tuples, alpha in [0..1]."""
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

# ============== CLASSES ============================
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    dna = [speed, r, g, b, diet, fear, size_factor, longevity]
      - diet: 0=herbivore, 1=predator
      - fear: 0..1
      - size_factor: [0.5..2.5]
      - longevity: [0.5..2.0], multiplies base lifespan (AGE_BASE)
    """
    def __init__(self, x, y, parent_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        # We'll store heading for correlated random walk
        self.heading = random.random() * 2.0 * np.pi

        if parent_dna:
            speed = np.clip(parent_dna[0] * random.uniform(0.9, 1.1), MIN_SPEED, MAX_SPEED)
            r     = np.clip(parent_dna[1] + random.randint(-15, 15), 50, 255)
            g     = np.clip(parent_dna[2] + random.randint(-15, 15), 50, 255)
            b     = np.clip(parent_dna[3] + random.randint(-15, 15), 50, 255)

            diet  = parent_dna[4]
            if random.random() < DIET_FLIP_CHANCE:
                diet = 1 - diet

            fear  = parent_dna[5]
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            size_factor = parent_dna[6]
            if random.random() < SIZE_MUTATION_CHANCE:
                size_factor *= random.uniform(0.8, 1.2)
                size_factor = min(max(MIN_SIZE, size_factor), MAX_SIZE)

            longevity = parent_dna[7]
            if random.random() < LONGEVITY_MUTATION_CHANCE:
                longevity *= random.uniform(0.8, 1.2)
                longevity = min(max(0.5, longevity), 2.0)

            self.dna = [speed, r, g, b, diet, fear, size_factor, longevity]

            # Extra random big speed shift
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], MIN_SPEED, MAX_SPEED)

        else:
            # brand-new
            self.dna = [
                random.uniform(1.5, 3.0),        # speed
                random.randint(50, 255),        # color R
                random.randint(50, 255),        # color G
                random.randint(50, 255),        # color B
                0,                              # diet=0 by default
                random.uniform(0.0, 0.2),       # fear
                random.uniform(MIN_SIZE, 1.2),  # size
                random.uniform(0.8, 1.2)        # longevity
            ]

        self.speed       = self.dna[0]
        self.color       = (self.dna[1], self.dna[2], self.dna[3])
        self.diet        = int(self.dna[4])
        self.fear        = self.dna[5]
        self.size_factor = self.dna[6]
        self.longevity   = self.dna[7]

    def _get_mutation_type(self, parent_dna):
        if not parent_dna:
            return "initial"
        changes = []
        # speed shift
        if abs(self.dna[0] - parent_dna[0]) > 1.0:
            changes.append("speed_shift")
        # color shift
        c_diff = abs(self.dna[1] - parent_dna[1]) + abs(self.dna[2] - parent_dna[2]) + abs(self.dna[3] - parent_dna[3])
        if c_diff > 45:
            changes.append("color_shift")
        # diet flip
        if self.dna[4] != parent_dna[4]:
            changes.append("diet_flip")
        # fear shift
        if abs(self.dna[5] - parent_dna[5]) > 0.05:
            changes.append("fear_shift")
        # size shift
        if abs(self.dna[6] - parent_dna[6]) > 0.3:
            changes.append("size_shift")
        # longevity shift
        if abs(self.dna[7] - parent_dna[7]) > 0.3:
            changes.append("long_shift")

        if not changes:
            return "minor"
        return "+".join(changes)

    def max_age(self):
        return AGE_BASE * self.longevity

    def strength(self):
        """Used in predator fights. Larger + more energy => stronger."""
        return self.size_factor * (1.0 + 0.5*(self.energy/100.0))

    def max_energy_capacity(self):
        return 100 + (self.size_factor - 1.0)*50

    def update(self, all_creatures, foods):
        self.age += 1

        # Fear cost if herbivore or possibly for predator as well (if you prefer)
        # We'll say herbivores pay the fear cost if fear > FEAR_THRESHOLD_HERB
        # but let's also let predators pay a small fear cost if fear>0.5 (they're timid).
        if self.diet == 0 and self.fear > FEAR_THRESHOLD_HERB:
            self.energy -= FEAR_COST_BASE * self.fear * self.size_factor
        elif self.diet == 1 and self.fear > PRED_FEAR_THRESHOLD:
            # timid predator
            self.energy -= FEAR_COST_BASE * self.fear * (self.size_factor * 0.5)

        # Size-based movement cost
        self.energy -= BASE_ENERGY_LOSS_PER_MOVE
        self.energy -= (self.size_factor * SIZE_ENERGY_LOSS_FACTOR)

        self.energy = max(self.energy, 0)

        if self.diet == 0:
            # ========== HERBIVORE ==========
            # Possibly flee if fear is high enough
            if self.fear > FEAR_THRESHOLD_HERB:
                predator, distp = self.find_nearest(all_creatures, 
                    lambda c: c is not self and c.diet == 1)
                if predator and distp < 120:
                    # run away
                    self.run_away(predator.x, predator.y)
                    self.wrap_screen()
                    return self.check_survival()

            # Check for food
            ftarget, distf = self.find_nearest(foods)
            if ftarget and distf < SIGHT_RANGE:
                if not self.is_line_blocked(ftarget.x, ftarget.y, all_creatures):
                    self.move_toward(ftarget.x, ftarget.y)
                    if distance(self.x, self.y, ftarget.x, ftarget.y) < (6 + self.size_factor*4):
                        self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_FOOD)
                        foods.remove(ftarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        else:
            # ========== PREDATOR ==========
            # find any creature
            ctarget, distc = self.find_nearest(all_creatures, lambda c: c is not self)
            if ctarget and distc < SIGHT_RANGE:
                if not self.is_line_blocked(ctarget.x, ctarget.y, all_creatures):
                    # If ctarget is a predator, do predator fight/avoid logic
                    if ctarget.diet == 1:
                        # If I'm timid (fear>0.5) and the other is bigger by ratio => flee
                        if self.fear > PRED_FEAR_THRESHOLD and (ctarget.size_factor > self.size_factor * SIZE_DOMINANCE_RATIO):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                return self.predator_fight(ctarget)
                    else:
                        # ctarget is herbivore
                        # If herb is significantly bigger & I'm timid => skip
                        if (ctarget.size_factor > self.size_factor * HERB_SIZE_DOMINANCE_RATIO) and (self.fear > PRED_FEAR_THRESHOLD):
                            self.run_away(ctarget.x, ctarget.y)
                        else:
                            self.move_toward(ctarget.x, ctarget.y)
                            if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                                # eat
                                self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_PREY)
                                return ("ate_creature", ctarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def predator_fight(self, other):
        # 30% chance other is unaware
        if random.random() < 0.3:
            my_str = self.strength() * AMBUSH_BONUS
            their_str = other.strength()
        else:
            my_str = self.strength()
            their_str = other.strength()

        if my_str > their_str:
            self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_PREY)
            return ("ate_creature", other)
        elif their_str > my_str:
            return ("ate_creature", self)
        else:
            # tie
            if random.random() < 0.5:
                return ("ate_creature", other)
            else:
                return ("ate_creature", self)

    def check_survival(self):
        if self.energy <= 0:
            return ("starved", None)
        if self.age > self.max_age():
            return ("aged", None)
        return ("alive", None)

    def reproduce(self, creatures, foods):
        if self.energy < 80:
            return None
        pop_size = len(creatures)
        food_count = len(foods)
        ratio_food_pop = food_count / pop_size if pop_size>0 else 1
        ratio_pop_cap  = pop_size / MAX_CREATURES

        chance = ENV_REPRO_CHANCE_BASE
        chance += ENV_REPRO_FOOD_FACTOR * ratio_food_pop
        chance -= ENV_REPRO_POP_FACTOR  * ratio_pop_cap
        chance = max(0.0, min(1.0, chance))

        if random.random() < chance:
            self.energy -= REPRODUCE_ENERGY_COST
            return Creature(self.x, self.y, self.dna)
        return None

    # Movement & line-of-sight
    def random_walk(self):
        self.heading += random.uniform(-0.2, 0.2)
        dx = np.cos(self.heading) * self.speed
        dy = np.sin(self.heading) * self.speed
        self.x += dx
        self.y += dy

    def run_away(self, ox, oy):
        dx = self.x - ox
        dy = self.y - oy
        distv = distance(self.x, self.y, ox, oy)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def move_toward(self, tx, ty):
        dx = tx - self.x
        dy = ty - self.y
        distv = distance(self.x, self.y, tx, ty)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def wrap_screen(self):
        """Toroidal wrapping—off one edge reappears on the opposite."""
        self.x %= 1200
        self.y %= 800

    def find_nearest(self, target_list, condition=None):
        valids = target_list
        if condition:
            valids = [t for t in target_list if condition(t)]
        if not valids:
            return (None, float('inf'))
        best_obj, best_dist = None, float('inf')
        for obj in valids:
            d = distance(self.x, self.y, obj.x, obj.y)
            if d < best_dist:
                best_dist = d
                best_obj = obj
        return (best_obj, best_dist)

    def is_line_blocked(self, tx, ty, creatures):
        targ_dist = distance(self.x, self.y, tx, ty)
        for c in creatures:
            if c is self:
                continue
            dist_c = distance(self.x, self.y, c.x, c.y)
            if dist_c >= targ_dist:
                continue
            d_line = line_distance_point(c.x, c.y, self.x, self.y, tx, ty)
            if d_line < BLOCKING_RADIUS:
                return True
        return False

    def draw(self, surface):
        sz = int(8 * self.size_factor)
        if self.diet == 0:  # herb
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), sz)
        else:  # predator
            half_w = sz
            points = [
                (self.x, self.y - sz),
                (self.x - half_w, self.y + sz),
                (self.x + half_w, self.y + sz)
            ]
            pygame.draw.polygon(surface, self.color, points)

# ============== SETUP ==============================

open(LOG_FILE, "w").close()

# Create initial population
creatures = []
for _ in range(INITIAL_HERBIVORES):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 0
    c.diet = 0
    creatures.append(c)
for _ in range(INITIAL_PREDATORS):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 1
    c.diet = 1
    creatures.append(c)

foods = [Food() for _ in range(INITIAL_FOOD)]

paused = False
# We'll distinguish "rendered frames" vs. "sim_frames".
# sim_frames increments only when not paused.
frames = 0     # rendered frames
sim_frames = 0 # "world" frames
population_history = deque(maxlen=300)
predator_history   = deque(maxlen=300)
death_log = {'starved': 0, 'aged': 0, 'hunted': 0}
mutation_tracker = Counter()

day_cycle_frame = 0
day_count = 0

fps_sample_timer = 0.0
avg_frame_time = 0.0

running = True

# ============== MAIN LOOP ==========================
while running:
    dt = clock.tick(30)
    frames += 1   # always increments

    fps_sample_timer += dt
    avg_frame_time = 0.9 * avg_frame_time + 0.1 * dt

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused

    if not paused:
        # Advance "simulation frames"
        sim_frames += 1

        # Day/night cycle
        day_cycle_frame = (day_cycle_frame + 1) % DAY_LENGTH
        if day_cycle_frame == 0:
            day_count += 1
        alpha = day_cycle_frame / (DAY_LENGTH - 1)
        is_day = (alpha < 0.5)
    else:
        # If paused, do NOT advance day_cycle_frame or sim_frames
        alpha = day_cycle_frame / (DAY_LENGTH - 1)
        is_day = (alpha < 0.5)

    # Background color
    if alpha < 0.5:
        sub_a = alpha / 0.5
        bg_color = interpolate_color(NIGHT_COLOR, DAY_COLOR, sub_a)
    else:
        sub_a = (alpha - 0.5) / 0.5
        bg_color = interpolate_color(DAY_COLOR, NIGHT_COLOR, sub_a)
    screen.fill(bg_color)

    # ========== SIM UPDATE IF NOT PAUSED ==========
    if not paused:
        new_creatures = []
        dead = []

        # Update creatures
        for c in creatures:
            status, other = c.update(creatures, foods)
            if status == "starved":
                death_log["starved"] += 1
                write_log(f"Death: starved (speed={c.speed:.2f})")
                dead.append(c)
                if random.random() < 0.3:
                    foods.append(Food())

            elif status == "aged":
                death_log["aged"] += 1
                write_log(f"Death: aged (speed={c.speed:.2f})")
                dead.append(c)
                if random.random() < 0.3:
                    foods.append(Food())

            elif status == "ate_creature":
                death_log["hunted"] += 1
                if other is c:
                    # c lost the fight
                    write_log(f"Death: hunted (speed={c.speed:.2f}) [lost predator fight]")
                    dead.append(c)
                else:
                    write_log(f"Death: hunted (speed={other.speed:.2f})")
                    dead.append(other)

            else:
                # alive => attempt reproduction
                child = c.reproduce(creatures, foods)
                if child and len(creatures) < MAX_CREATURES:
                    mut_type = child._get_mutation_type(c.dna)
                    if mut_type not in ("initial", "minor"):
                        mutation_tracker[mut_type] += 1
                    new_creatures.append(child)
                    write_log(
                        f"Birth: speed={child.speed:.2f}, diet={child.diet}, size={child.size_factor:.2f}, "
                        f"fear={child.fear:.2f}, longevity={child.longevity:.2f}, mutation={mut_type}"
                    )

        # remove dead
        creatures = [cr for cr in creatures if cr not in dead]
        # add newborns
        space_left = max(0, MAX_CREATURES - len(creatures))
        creatures += new_creatures[:space_left]

        # track population
        predator_count = sum(1 for x in creatures if x.diet == 1)
        population_history.append(len(creatures))
        predator_history.append(predator_count)

        # Day-based faster food spawn
        if is_day:
            # e.g. 5% chance each sim frame
            if random.random() < DAYTIME_FOOD_SPAWN_CHANCE:
                foods.append(Food())

    # ============ DRAW ALL =============
    for f in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(f.x), int(f.y)), 4)
    for cr in creatures:
        cr.draw(screen)

    # ============ HUD (top-left) ===========
    predator_count = sum(1 for x in creatures if x.diet == 1)
    herb_count     = len(creatures) - predator_count
    ratio_str = "∞"
    if predator_count > 0:
        ratio_val = herb_count / predator_count
        ratio_str = f"{ratio_val:.1f}"

    stats = [
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Predators: {predator_count}, ratio=1:{ratio_str}",
        f"Food: {len(foods)}",
        f"Day Count: {day_count}",
        f"Sim Time: {sim_frames} frames",  # stops incrementing when paused
    ]
    if creatures:
        avg_speed = np.mean([c.speed for c in creatures])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']}  aged:{death_log['aged']}  hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    for i, text in enumerate(stats):
        txt = font.render(text, True, (255, 255, 255))
        screen.blit(txt, (10, 10 + i*25))

    # =========== MUTATION TRACKER ===========
    mut_top3 = mutation_tracker.most_common(3)
    if mut_top3:
        mut_strs = [f"{mt}:{cnt}" for (mt, cnt) in mut_top3]
        mut_line = "Mutations: " + ", ".join(mut_strs)
        txtm = font.render(mut_line, True, (255, 255, 255))
        screen.blit(txtm, (10, 10 + len(stats)*25))

    # =========== FPS (top-right) ===========
    fps_txt = f"FPS: {int(clock.get_fps())}"
    ms_txt = f"FrameTime: {avg_frame_time:.1f}ms"
    fps_surf = font.render(fps_txt, True, (255,255,255))
    ms_surf = font.render(ms_txt, True, (255,255,255))
    screen.blit(fps_surf, (1050, 10))
    screen.blit(ms_surf, (1050, 35))

    # =========== Population Graph ===========
    # We'll draw two lines: total creatures, and predators
    graph_x, graph_y = 800, 70
    graph_w, graph_h = 380, 90
    pygame.draw.rect(screen, (50,50,50), (graph_x, graph_y, graph_w, graph_h))

    if len(population_history) > 1:
        # total population
        points_pop = []
        for i, val in enumerate(population_history):
            xx = graph_x + graph_w - (len(population_history) - i)*2
            yy = graph_y + graph_h - (val / MAX_CREATURES) * graph_h
            points_pop.append((xx, yy))
        pygame.draw.lines(screen, GRAPH_COLOR, False, points_pop, 2)

    if len(predator_history) > 1:
        # predator population
        points_pred = []
        for i, val in enumerate(predator_history):
            xx = graph_x + graph_w - (len(predator_history) - i)*2
            # scale them the same, or separate scale? We'll do the same:
            yy = graph_y + graph_h - (val / MAX_CREATURES) * graph_h
            points_pred.append((xx, yy))
        pygame.draw.lines(screen, PREDATOR_GRAPH_COLOR, False, points_pred, 2)

    pygame.display.flip()

pygame.quit()


In [ ]:
# 0.4.3 day counter, mutation tracker, daytime food spawns, toroidal wrapping, predators fear e/o 

import pygame
import numpy as np
import random
from collections import deque, Counter
from datetime import datetime

# =================== CONFIG ====================
MAX_CREATURES = 2000

# Start ratio: e.g., 200 herbivores to 1 predator
INITIAL_HERBIVORES = 200
INITIAL_PREDATORS  = 1

INITIAL_FOOD = 40

AGE_BASE = 700    # base lifespan, scaled by creature's longevity gene

ENERGY_GAIN_FROM_FOOD = 22
ENERGY_GAIN_FROM_PREY = 50
ENERGY_LOSS_PER_MOVE = 0.1

REPRODUCE_ENERGY_COST = 50
ENV_REPRO_CHANCE_BASE = 0.3   # baseline chance to attempt reproduction
ENV_REPRO_FOOD_FACTOR = 0.2   # bonus factor if there's lots of food
ENV_REPRO_POP_FACTOR  = 0.2   # penalty factor if population is high

MUTATION_RATE = 0.1
DIET_FLIP_CHANCE = 0.01
FEAR_MUTATION_CHANCE = 0.2
SIZE_MUTATION_CHANCE = 0.2
LONGEVITY_MUTATION_CHANCE = 0.2

MAX_SPEED = 8.0
MIN_SPEED = 0.5
MIN_SIZE  = 0.5
MAX_SIZE  = 2.5

# Fear
FEAR_COST = 0.02  # Extra energy drain per frame if herbivore has fear>0.3

# Predator vs. Predator
AMBUSH_BONUS = 1.1  # attacker gets a bonus if the other doesn't notice

# If a predator's own fear is > PRED_FEAR_THRESHOLD, it avoids attacking another predator if enough herbivores are around
PRED_FEAR_THRESHOLD = 0.5  
HERBIVORE_BUFFER = 5       # if >=5 herbivores in sight, skip predator fight

SIGHT_RANGE = 200.0
BLOCKING_RADIUS = 5.0

DAY_LENGTH = 600  # frames for one full day-night cycle
DAY_COLOR   = (0, 0, 50)
NIGHT_COLOR = (0, 0, 0)

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# ============== COLORS / GRAPHICS =================
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)

# ============== HELPERS ============================
def write_log(message):
    """Write to the log file every LOG_INTERVAL frames."""
    if frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return np.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    """
    Returns the shortest distance from point (px,py) to line segment A->B.
    Used for naive line-of-sight blocking.
    """
    ABx = bx - ax
    ABy = by - ay
    APx = px - ax
    APy = py - ay
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        return distance(px, py, ax, ay)
    t = (APx*ABx + APy*ABy) / AB_len_sq
    t = max(0, min(1, t))
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    """Linear interpolate between two RGB tuples, alpha in [0..1]."""
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

# ============== CLASSES ============================
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    dna = [speed, r, g, b, diet, fear, size_factor, longevity]
      - diet: 0=herbivore, 1=predator
      - fear: 0..1
      - size_factor: [0.5..2.5]
      - longevity: [0.5..2.0], multiplies base lifespan (AGE_BASE)

    Also uses a correlated random walk via self.heading.
    Wraps around screen edges (toroidal).
    """
    def __init__(self, x, y, parent_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        self.heading = random.random() * 2.0 * np.pi

        if parent_dna:
            speed = np.clip(parent_dna[0] * random.uniform(0.9, 1.1), MIN_SPEED, MAX_SPEED)
            r     = np.clip(parent_dna[1] + random.randint(-15, 15), 50, 255)
            g     = np.clip(parent_dna[2] + random.randint(-15, 15), 50, 255)
            b     = np.clip(parent_dna[3] + random.randint(-15, 15), 50, 255)

            diet  = parent_dna[4]
            if random.random() < DIET_FLIP_CHANCE:
                diet = 1 - diet

            fear  = parent_dna[5]
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            size_factor = parent_dna[6]
            if random.random() < SIZE_MUTATION_CHANCE:
                size_factor *= random.uniform(0.8, 1.2)
                size_factor = min(max(MIN_SIZE, size_factor), MAX_SIZE)

            longevity = parent_dna[7]
            if random.random() < LONGEVITY_MUTATION_CHANCE:
                longevity *= random.uniform(0.8, 1.2)
                longevity = min(max(0.5, longevity), 2.0)

            self.dna = [speed, r, g, b, diet, fear, size_factor, longevity]

            # Extra chance of big speed or color jump
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], MIN_SPEED, MAX_SPEED)

        else:
            # brand-new
            self.dna = [
                random.uniform(1.5, 3.0),        # speed
                random.randint(50, 255),        # color R
                random.randint(50, 255),        # color G
                random.randint(50, 255),        # color B
                0,                              # diet=0 by default here
                random.uniform(0.0, 0.2),       # fear
                random.uniform(MIN_SIZE, 1.2),  # size
                random.uniform(0.8, 1.2)        # longevity
            ]

        # Extract fields for convenience
        self.speed       = self.dna[0]
        self.color       = (self.dna[1], self.dna[2], self.dna[3])
        self.diet        = int(self.dna[4])   # 0 or 1
        self.fear        = self.dna[5]
        self.size_factor = self.dna[6]
        self.longevity   = self.dna[7]

    def _get_mutation_type(self, parent_dna):
        """Classify big changes for the mutation tracker."""
        if not parent_dna:
            return "initial"
        changes = []
        if abs(self.dna[0] - parent_dna[0]) > 1.0:
            changes.append("speed_shift")
        c_diff = abs(self.dna[1] - parent_dna[1]) + abs(self.dna[2] - parent_dna[2]) + abs(self.dna[3] - parent_dna[3])
        if c_diff > 45:
            changes.append("color_shift")
        if self.dna[4] != parent_dna[4]:
            changes.append("diet_flip")
        if abs(self.dna[5] - parent_dna[5]) > 0.05:
            changes.append("fear_shift")
        if abs(self.dna[6] - parent_dna[6]) > 0.3:
            changes.append("size_shift")
        if abs(self.dna[7] - parent_dna[7]) > 0.3:
            changes.append("long_shift")
        if not changes:
            return "minor"
        return "+".join(changes)

    def max_age(self):
        return AGE_BASE * self.longevity

    def strength(self):
        """
        Used when predators fight each other.
        A combination of size and partial energy:
        bigger + more energy => more likely to win.
        """
        return self.size_factor * (1.0 + 0.5*(self.energy/100.0))

    def max_energy_capacity(self):
        """Bigger = can hold more energy overall."""
        return 100 + (self.size_factor - 1.0)*50

    def update(self, all_creatures, foods):
        """Main AI logic. Returns (status, other) to indicate events like dying or eating another creature."""
        self.age += 1

        # Fear cost for herbivores
        if self.diet == 0 and self.fear > 0.3:
            self.energy -= FEAR_COST * self.fear

        self.energy -= ENERGY_LOSS_PER_MOVE
        self.energy = max(self.energy, 0)

        if self.diet == 0:
            # ========== HERBIVORE ==========
            # Possibly flee if fear is high enough
            if self.fear > 0.3:
                predator, distp = self.find_nearest(all_creatures, 
                    lambda c: c is not self and c.diet == 1)
                if predator and distp < 120:
                    self.run_away(predator.x, predator.y)
                    self.wrap_screen()
                    return self.check_survival()

            # Else look for food
            ftarget, distf = self.find_nearest(foods)
            if ftarget and distf < SIGHT_RANGE:
                if not self.is_line_blocked(ftarget.x, ftarget.y, all_creatures):
                    self.move_toward(ftarget.x, ftarget.y)
                    if distance(self.x, self.y, ftarget.x, ftarget.y) < (6 + self.size_factor*4):
                        # eat
                        self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_FOOD)
                        foods.remove(ftarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        else:
            # ========== PREDATOR ==========
            # find any creature (herb or pred)
            ctarget, distc = self.find_nearest(all_creatures, lambda c: c is not self)
            if ctarget and distc < SIGHT_RANGE:
                if not self.is_line_blocked(ctarget.x, ctarget.y, all_creatures):
                    # If ctarget is a predator and I'm cautious (fear>0.5),
                    # skip attacking if there are plenty of herbivores around
                    if ctarget.diet == 1 and self.fear > PRED_FEAR_THRESHOLD:
                        # Check if we can find enough herbivores in range
                        herb_near, dist_herb = self.find_nearest(
                            all_creatures, lambda c: c.diet==0 and c is not self)
                        # if we have at least HERBIVORE_BUFFER herbivores in the area, skip predator fight
                        if herb_near and dist_herb < SIGHT_RANGE:
                            # Let's count how many are within SIGHT_RANGE
                            count_herb = sum(1 for cr in all_creatures 
                                             if cr.diet==0 and cr is not self and 
                                                distance(self.x, self.y, cr.x, cr.y) < SIGHT_RANGE)
                            if count_herb >= HERBIVORE_BUFFER:
                                # enough herbivores -> skip predator fight
                                self.random_walk()
                                self.wrap_screen()
                                return self.check_survival()
                    # else proceed to attack
                    self.move_toward(ctarget.x, ctarget.y)
                    if distance(self.x, self.y, ctarget.x, ctarget.y) < (10 + self.size_factor*4 + ctarget.size_factor*4):
                        # if ctarget is herb => easy kill
                        if ctarget.diet == 0:
                            self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_PREY)
                            return ("ate_creature", ctarget)
                        else:
                            # predator fight
                            return self.predator_fight(ctarget)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def predator_fight(self, other):
        # 30% chance other is unaware
        if random.random() < 0.3:
            my_str = self.strength() * AMBUSH_BONUS
            their_str = other.strength()
        else:
            my_str = self.strength()
            their_str = other.strength()

        if my_str > their_str:
            # I kill other
            self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_PREY)
            return ("ate_creature", other)
        elif their_str > my_str:
            # I lose
            return ("ate_creature", self)
        else:
            # tie -> 50/50
            if random.random() < 0.5:
                return ("ate_creature", other)
            else:
                return ("ate_creature", self)

    def check_survival(self):
        if self.energy <= 0:
            return ("starved", None)
        if self.age > self.max_age():
            return ("aged", None)
        return ("alive", None)

    def reproduce(self, creatures, foods):
        """
        Environment-based reproduction chance
        and also must have enough energy.
        """
        if self.energy < 80:
            return None
        pop_size = len(creatures)
        food_count = len(foods)

        ratio_food_pop = food_count / pop_size if pop_size>0 else 1
        ratio_pop_cap  = pop_size / MAX_CREATURES

        chance = ENV_REPRO_CHANCE_BASE
        chance += ENV_REPRO_FOOD_FACTOR * ratio_food_pop
        chance -= ENV_REPRO_POP_FACTOR  * ratio_pop_cap
        chance = max(0.0, min(1.0, chance))

        if random.random() < chance:
            self.energy -= REPRODUCE_ENERGY_COST
            return Creature(self.x, self.y, self.dna)
        return None

    # ============== Movement & Utility ===============
    def random_walk(self):
        self.heading += random.uniform(-0.2, 0.2)
        dx = np.cos(self.heading) * self.speed
        dy = np.sin(self.heading) * self.speed
        self.x += dx
        self.y += dy

    def run_away(self, ox, oy):
        dx = self.x - ox
        dy = self.y - oy
        distv = distance(self.x, self.y, ox, oy)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def move_toward(self, tx, ty):
        dx = tx - self.x
        dy = ty - self.y
        distv = distance(self.x, self.y, tx, ty)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def wrap_screen(self):
        """Toroidal wrapping—going off one edge reappears on the other."""
        self.x %= 1200
        self.y %= 800

    def find_nearest(self, target_list, condition=None):
        """Return (nearest_object, distance)."""
        valids = target_list
        if condition:
            valids = [t for t in target_list if condition(t)]
        if not valids:
            return (None, float('inf'))
        best_obj, best_dist = None, float('inf')
        for obj in valids:
            d = distance(self.x, self.y, obj.x, obj.y)
            if d < best_dist:
                best_dist = d
                best_obj = obj
        return (best_obj, best_dist)

    def is_line_blocked(self, tx, ty, creatures):
        """Naive occlusion check: if any creature is near the line & closer than the target."""
        targ_dist = distance(self.x, self.y, tx, ty)
        for c in creatures:
            if c is self:
                continue
            dist_c = distance(self.x, self.y, c.x, c.y)
            if dist_c >= targ_dist:
                continue
            d_line = line_distance_point(c.x, c.y, self.x, self.y, tx, ty)
            if d_line < BLOCKING_RADIUS:
                return True
        return False

    def draw(self, surface):
        """Scale size by self.size_factor. Circle if herb, triangle if pred."""
        sz = int(8 * self.size_factor)
        if self.diet == 0:
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), sz)
        else:
            half_w = sz
            points = [
                (self.x, self.y - sz),
                (self.x - half_w, self.y + sz),
                (self.x + half_w, self.y + sz)
            ]
            pygame.draw.polygon(surface, self.color, points)

# ============== SETUP ==============================
open(LOG_FILE, "w").close()

# Initial population
creatures = []
for _ in range(INITIAL_HERBIVORES):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 0  # force herbivore
    c.diet   = 0
    creatures.append(c)
for _ in range(INITIAL_PREDATORS):
    c = Creature(random.randint(100, 1100), random.randint(100, 700))
    c.dna[4] = 1  # force predator
    c.diet   = 1
    creatures.append(c)

foods = [Food() for _ in range(INITIAL_FOOD)]

paused = False
frames = 0
population_history = deque(maxlen=300)
death_log = {'starved': 0, 'aged': 0, 'hunted': 0}

# Track how many of each mutation type occurs
mutation_tracker = Counter()

# Day counter: each time we complete a full cycle, increment day_count
day_cycle_frame = 0
day_count = 0

# Performance tracking
fps_sample_timer = 0.0
avg_frame_time = 0.0

running = True

# ============== MAIN LOOP ==========================
while running:
    # Day-night background color
    day_cycle_frame = (day_cycle_frame + 1) % DAY_LENGTH
    if day_cycle_frame == 0:
        day_count += 1  # completed a full day cycle

    alpha = day_cycle_frame / (DAY_LENGTH - 1)
    if alpha < 0.5:
        sub_a = alpha / 0.5
        bg_color = interpolate_color(NIGHT_COLOR, DAY_COLOR, sub_a)
    else:
        sub_a = (alpha - 0.5) / 0.5
        bg_color = interpolate_color(DAY_COLOR, NIGHT_COLOR, sub_a)
    screen.fill(bg_color)

    # Day or night?
    is_day = (alpha < 0.5)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused

    dt = clock.tick(30)
    fps_sample_timer += dt
    frames += 1
    avg_frame_time = 0.9 * avg_frame_time + 0.1 * dt

    if not paused:
        new_creatures = []
        dead = []

        # ----------- Update Creatures -----------
        for c in creatures:
            result, other = c.update(creatures, foods)
            if result == "starved":
                death_log["starved"] += 1
                write_log(f"Death: starved (speed={c.speed:.2f})")
                dead.append(c)
                # 30% chance to spawn new food
                if random.random() < 0.3:
                    foods.append(Food())
            elif result == "aged":
                death_log["aged"] += 1
                write_log(f"Death: aged (speed={c.speed:.2f})")
                dead.append(c)
                if random.random() < 0.3:
                    foods.append(Food())
            elif result == "ate_creature":
                death_log["hunted"] += 1
                # if c= eater, other= meal
                # if c= meal, other= eater
                if other is c:
                    # c lost the fight
                    write_log(f"Death: hunted (speed={c.speed:.2f}) [lost predator fight]")
                    dead.append(c)
                else:
                    write_log(f"Death: hunted (speed={other.speed:.2f})")
                    dead.append(other)
            else:
                # "alive" => attempt reproduction
                baby = c.reproduce(creatures, foods)
                if baby and len(creatures) < MAX_CREATURES:
                    mut_type = baby._get_mutation_type(c.dna)
                    if mut_type not in ("initial", "minor"):
                        mutation_tracker[mut_type] += 1
                    new_creatures.append(baby)
                    write_log(
                        f"Birth: speed={baby.speed:.2f}, diet={baby.diet}, size={baby.size_factor:.2f}, "
                        f"fear={baby.fear:.2f}, longevity={baby.longevity:.2f}, mutation={mut_type}"
                    )

        creatures = [cr for cr in creatures if cr not in dead]
        space_left = max(0, MAX_CREATURES - len(creatures))
        creatures += new_creatures[:space_left]

        population_history.append(len(creatures))

        # ----------- Day-based Food Spawns -----------
        # During day, let's have a random chance each frame to spawn extra food.
        if is_day:
            if random.random() < 0.02:  # 2% chance each frame
                foods.append(Food())

    # ============ DRAW ALL ===================
    for f in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(f.x), int(f.y)), 4)
    for cr in creatures:
        cr.draw(screen)

    # ============ HUD (top-left) =============
    predator_count = sum(1 for x in creatures if x.diet == 1)
    herb_count     = len(creatures) - predator_count
    # ratio of herbivores to 1 predator
    ratio_str = "∞"
    if predator_count > 0:
        ratio_val = herb_count / predator_count
        ratio_str = f"{ratio_val:.1f}"

    stats = [
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Predators: {predator_count}, ratio=1:{ratio_str}",
        f"Food: {len(foods)}",
        f"Day Count: {day_count}",
        f"Time (frames): {frames}"
    ]
    if creatures:
        avg_speed = np.mean([c.speed for c in creatures])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']}  aged:{death_log['aged']}  hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    # Show them
    for i, text in enumerate(stats):
        txt = font.render(text, True, (255, 255, 255))
        screen.blit(txt, (10, 10 + i * 25))

    # ============ MUTATION TRACKER (below that) ============
    # Let's display the top 3 mutations from our counter
    mut_top3 = mutation_tracker.most_common(3)
    if mut_top3:
        mut_strs = [f"{mt}:{cnt}" for (mt, cnt) in mut_top3]
        mut_line = "Mutations: " + ", ".join(mut_strs)
        txtm = font.render(mut_line, True, (255, 255, 255))
        screen.blit(txtm, (10, 10 + len(stats)*25))

    # ============ FPS (top-right) =============
    fps_txt = f"FPS: {int(clock.get_fps())}"
    ms_txt = f"FrameTime: {avg_frame_time:.1f}ms"
    fps_surf = font.render(fps_txt, True, (255,255,255))
    ms_surf = font.render(ms_txt, True, (255,255,255))
    screen.blit(fps_surf, (1050, 10))
    screen.blit(ms_surf, (1050, 35))

    # ============ Population Graph ============
    graph_x, graph_y = 800, 70
    graph_w, graph_h = 380, 90
    pygame.draw.rect(screen, (50,50,50), (graph_x, graph_y, graph_w, graph_h))
    if len(population_history) > 1:
        points = []
        for i, val in enumerate(population_history):
            xx = graph_x + graph_w - (len(population_history) - i)*2
            yy = graph_y + graph_h - (val / MAX_CREATURES) * graph_h
            points.append((xx, yy))
        pygame.draw.lines(screen, GRAPH_COLOR, False, points, 2)

    pygame.display.flip()

pygame.quit()


In [6]:
# 0.4.2 pvp, cost for fear, size gene, longevity, env based reproduction
import pygame
import numpy as np
import random
from collections import deque
from datetime import datetime

# =================== CONFIG ====================
MAX_CREATURES = 2000
INITIAL_HERBIVORES = 200   # for a 200:1 ratio
INITIAL_PREDATORS  = 1
INITIAL_FOOD = 40
AGE_BASE = 700            # base lifespan (modified by longevity gene)

ENERGY_GAIN_FROM_FOOD = 22
ENERGY_GAIN_FROM_PREY = 50    # if predator kills another creature
ENERGY_LOSS_PER_MOVE = 0.1

REPRODUCE_ENERGY_COST = 50
ENV_REPRO_CHANCE_BASE = 0.3   # base probability to attempt reproduction
ENV_REPRO_FOOD_FACTOR = 0.2   # bonus if food is plentiful
ENV_REPRO_POP_FACTOR  = 0.2   # penalty if population is high

MUTATION_RATE = 0.1
DIET_FLIP_CHANCE = 0.01
FEAR_MUTATION_CHANCE = 0.2
SIZE_MUTATION_CHANCE = 0.2
LONGEVITY_MUTATION_CHANCE = 0.2

MAX_SPEED = 8.0
MIN_SPEED = 0.5
MIN_SIZE  = 0.5
MAX_SIZE  = 2.5

# fear cost: e.g., each frame herbivores with fear>0.3 lose extra energy
FEAR_COST = 0.02

# For predator vs predator conflict
AMBUSH_BONUS = 1.1  # if one attacks the other unawares, multiply its strength by 1.1

SIGHT_RANGE = 200.0
BLOCKING_RADIUS = 5.0

DAY_LENGTH = 600
DAY_COLOR = (0, 0, 50)
NIGHT_COLOR = (0, 0, 0)

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# ============== COLORS / GRAPHICS =================
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)

# ============== HELPERS ============================
def write_log(message):
    """Write to the log file every LOG_INTERVAL frames."""
    if frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return np.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    """
    Returns the shortest distance from point (px,py) to line segment A->B.
    Used for naive line-of-sight blocking.
    """
    ABx = bx - ax
    ABy = by - ay
    APx = px - ax
    APy = py - ay
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        return distance(px, py, ax, ay)
    t = (APx*ABx + APy*ABy) / AB_len_sq
    t = max(0, min(1, t))
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    """Linear interpolate between two RGB tuples, alpha in [0..1]."""
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

# ============== CLASSES ============================
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    dna layout = [speed, r, g, b, diet, fear, size_factor, longevity]
      - diet: 0=herbivore, 1=predator
      - fear: 0..1
      - size_factor: in [0.5 .. 2.5]
      - longevity: in [0.5 .. 2.0], multiply with base lifespan
    We also do a correlated random walk with heading.
    """
    def __init__(self, x, y, parent_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        self.heading = random.random() * 2.0 * np.pi

        if parent_dna:
            speed       = np.clip(parent_dna[0] * random.uniform(0.9, 1.1), MIN_SPEED, MAX_SPEED)
            r           = np.clip(parent_dna[1] + random.randint(-15, 15), 50, 255)
            g           = np.clip(parent_dna[2] + random.randint(-15, 15), 50, 255)
            b           = np.clip(parent_dna[3] + random.randint(-15, 15), 50, 255)

            diet        = parent_dna[4]
            if random.random() < DIET_FLIP_CHANCE:
                diet = 1 - diet

            fear        = parent_dna[5]
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            size_factor = parent_dna[6]
            if random.random() < SIZE_MUTATION_CHANCE:
                size_factor *= random.uniform(0.8, 1.2)
                size_factor = min(max(MIN_SIZE, size_factor), MAX_SIZE)

            longevity   = parent_dna[7]
            if random.random() < LONGEVITY_MUTATION_CHANCE:
                longevity *= random.uniform(0.8, 1.2)
                longevity = min(max(0.5, longevity), 2.0)

            self.dna = [speed, r, g, b, diet, fear, size_factor, longevity]

            # Additional chance for a big speed or color jump
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], MIN_SPEED, MAX_SPEED)

        else:
            # brand-new random
            # We'll create either herbivores or predators in ratio
            self.dna = [
                random.uniform(1.5, 3.0),        # speed
                random.randint(50, 255),        # color R
                random.randint(50, 255),        # color G
                random.randint(50, 255),        # color B
                0,                              # diet (we'll set below)
                random.uniform(0.0, 0.2),       # fear
                random.uniform(MIN_SIZE, 1.2),  # size
                random.uniform(0.8, 1.2)        # longevity factor
            ]

        self.speed       = self.dna[0]
        self.color       = (self.dna[1], self.dna[2], self.dna[3])
        self.diet        = int(self.dna[4])   # 0 or 1
        self.fear        = self.dna[5]
        self.size_factor = self.dna[6]
        self.longevity   = self.dna[7]

        # clamp diet if newly created
        if parent_dna is None:
            # We manually assign diet by ratio
            # If # predators < # herbivores/200 for example, or simply
            # a random chance.
            pass

    def max_age(self):
        """Compute actual max age from base + longevity factor."""
        return AGE_BASE * self.longevity

    def strength(self):
        """
        Used in fights if a predator hunts another predator.
        E.g. size + fraction of energy => bigger size = more damage,
        high energy = more fighting power, etc.
        """
        return self.size_factor * (1.0 + (self.energy / 100.0)*0.5)

    def max_energy_capacity(self):
        """Optional example: bigger size => can hold more energy, but moves slower."""
        return 100 + (self.size_factor - 1.0)*50

    def _get_mutation_type(self, parent_dna):
        if not parent_dna:
            return "initial"
        changes = []
        if abs(self.dna[0] - parent_dna[0]) > 1.0:
            changes.append("speed_shift")
        cdiff = abs(self.dna[1] - parent_dna[1]) + abs(self.dna[2] - parent_dna[2]) + abs(self.dna[3] - parent_dna[3])
        if cdiff > 45:
            changes.append("color_shift")
        if self.dna[4] != parent_dna[4]:
            changes.append("diet_flip")
        if abs(self.dna[5] - parent_dna[5]) > 0.05:
            changes.append("fear_shift")
        if abs(self.dna[6] - parent_dna[6]) > 0.3:
            changes.append("size_shift")
        if abs(self.dna[7] - parent_dna[7]) > 0.3:
            changes.append("long_shift")
        if not changes:
            return "minor"
        return "+".join(changes)

    def update(self, all_creatures, foods):
        self.age += 1

        # extra fear cost if herbivore's fear > 0.3
        if self.diet == 0 and self.fear > 0.3:
            self.energy -= FEAR_COST * self.fear  # scale cost by fear

        self.energy -= ENERGY_LOSS_PER_MOVE
        self.energy = max(0, self.energy)

        if self.diet == 0:
            # ========== HERBIVORE LOGIC ================
            # Possibly flee if fear is high enough
            if self.fear > 0.3:
                predator, dist_pred = self.find_nearest(all_creatures, 
                    condition=lambda c: c.diet == 1 and c is not self)
                if predator and dist_pred < 120:
                    # run away
                    self.run_away(predator.x, predator.y)
                    self.wrap_screen()
                    return self.check_survival()

            # else look for nearest food, if line-of-sight
            nearest_food, distf = self.find_nearest(foods)
            if nearest_food and distf < SIGHT_RANGE:
                if not self.is_line_blocked(nearest_food.x, nearest_food.y, all_creatures):
                    self.move_toward(nearest_food.x, nearest_food.y)
                    if distance(self.x, self.y, nearest_food.x, nearest_food.y) < (6 + self.size_factor*4):
                        # eat
                        self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_FOOD)
                        foods.remove(nearest_food)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        else:
            # ========== PREDATOR LOGIC ================
            # find any creature to hunt (including other predators)
            # exclude self
            target, distt = self.find_nearest(all_creatures, condition=lambda c: c is not self)
            if target and distt < SIGHT_RANGE:
                if not self.is_line_blocked(target.x, target.y, all_creatures):
                    self.move_toward(target.x, target.y)
                    if distance(self.x, self.y, target.x, target.y) < (10 + self.size_factor*4 + target.size_factor*4):
                        # attempt to kill/eat
                        if target.diet == 0:
                            # easy kill = herbivore
                            self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_PREY)
                            return ("ate_creature", target)
                        else:
                            # fighting another predator
                            return self.predator_fight(target)
                else:
                    self.random_walk()
            else:
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def predator_fight(self, other):
        """
        If predator hunts predator. 
        Consider stealth factor:
          - if other is not aware, the attacker gets an ambush bonus
          - if both are aware, they fight on equal footing
        For simplicity, let's do a random check that the other 'noticed' in time.
        """
        # chance the other didn't notice
        if random.random() < 0.3:
            # attacker gets ambush advantage
            my_strength  = self.strength() * AMBUSH_BONUS
            other_strength = other.strength()
        else:
            # both fight equally
            my_strength  = self.strength()
            other_strength = other.strength()

        if my_strength > other_strength:
            # I kill the other
            self.energy = min(self.max_energy_capacity(), self.energy + ENERGY_GAIN_FROM_PREY)
            return ("ate_creature", other)
        elif my_strength < other_strength:
            # I lose
            return ("ate_creature", self)
        else:
            # tie: maybe both die or we do a coin flip
            if random.random() < 0.5:
                return ("ate_creature", other)
            else:
                return ("ate_creature", self)

    def check_survival(self):
        if self.energy <= 0:
            return ("starved", None)
        if self.age > self.max_age():
            return ("aged", None)
        return ("alive", None)

    def reproduce(self, creatures, foods):
        """
        Environment-based reproduction chance:
          - base chance ENV_REPRO_CHANCE_BASE
          - plus bonus if there's a lot of food relative to population
          - minus penalty if population is high
        Also must have enough energy.
        """
        if self.energy < 80:
            return None
        # gather environment metrics
        # e.g. ratio of food to creatures, or just length(foods)...
        pop_size = len(creatures)
        food_count = len(foods)

        # A simple formula:
        # chance = base + (food factor * (food_count / pop_size)) - (pop factor * (pop_size / 1000 or something))
        # Tweak as you like:
        ratio_food_pop = (food_count / pop_size) if pop_size>0 else 1
        ratio_pop_cap  = pop_size / MAX_CREATURES

        chance = ENV_REPRO_CHANCE_BASE
        chance += ENV_REPRO_FOOD_FACTOR * ratio_food_pop
        chance -= ENV_REPRO_POP_FACTOR  * ratio_pop_cap

        # clamp 0..1
        chance = min(max(chance, 0.0), 1.0)
        if random.random() < chance:
            # do it
            self.energy -= REPRODUCE_ENERGY_COST
            return Creature(self.x, self.y, self.dna)
        return None

    # ============== Movement & Utility ===============
    def random_walk(self):
        self.heading += random.uniform(-0.2, 0.2)
        dx = np.cos(self.heading) * self.speed
        dy = np.sin(self.heading) * self.speed
        self.x += dx
        self.y += dy

    def run_away(self, ox, oy):
        dx = self.x - ox
        dy = self.y - oy
        distv = distance(self.x, self.y, ox, oy)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def move_toward(self, tx, ty):
        dx = tx - self.x
        dy = ty - self.y
        distv = distance(self.x, self.y, tx, ty)
        if distv > 1:
            self.x += (dx/distv)*self.speed
            self.y += (dy/distv)*self.speed
        else:
            self.random_walk()

    def wrap_screen(self):
        self.x %= 1200
        self.y %= 800

    def find_nearest(self, target_list, condition=None):
        valids = target_list
        if condition:
            valids = [t for t in target_list if condition(t)]
        if not valids:
            return (None, float('inf'))
        best_obj, best_dist = None, float('inf')
        for obj in valids:
            d = distance(self.x, self.y, obj.x, obj.y)
            if d < best_dist:
                best_dist = d
                best_obj = obj
        return (best_obj, best_dist)

    def is_line_blocked(self, tx, ty, creatures):
        target_dist = distance(self.x, self.y, tx, ty)
        for c in creatures:
            if c is self:
                continue
            dist_c = distance(self.x, self.y, c.x, c.y)
            if dist_c >= target_dist:
                continue
            d_line = line_distance_point(c.x, c.y, self.x, self.y, tx, ty)
            if d_line < BLOCKING_RADIUS:
                return True
        return False

    def draw(self, surface):
        """Draw differently for herbivores vs predators. Also scale by size_factor."""
        sz = int(8 * self.size_factor)
        if self.diet == 0:
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), sz)
        else:
            half_w = sz
            points = [
                (self.x, self.y - sz),
                (self.x - half_w, self.y + sz),
                (self.x + half_w, self.y + sz)
            ]
            pygame.draw.polygon(surface, self.color, points)


# ============== SETUP ==============================
open(LOG_FILE, "w").close()

# Create initial population with a ratio
creatures = []
for _ in range(INITIAL_HERBIVORES):
    c = Creature(random.randint(100,1100), random.randint(100,700))
    c.dna[4] = 0  # force diet=0
    c.diet   = 0
    creatures.append(c)
for _ in range(INITIAL_PREDATORS):
    c = Creature(random.randint(100,1100), random.randint(100,700))
    c.dna[4] = 1  # force diet=1
    c.diet   = 1
    creatures.append(c)

foods = [Food() for _ in range(INITIAL_FOOD)]

paused = False
frames = 0
population_history = deque(maxlen=300)
death_log = {'starved': 0, 'aged': 0, 'hunted': 0}

day_cycle_frame = 0
running = True

fps_sample_timer = 0.0
avg_frame_time = 0.0

# ============== MAIN LOOP ==========================
while running:
    # Calculate day/night color
    day_cycle_frame = (day_cycle_frame + 1) % DAY_LENGTH
    alpha = day_cycle_frame / (DAY_LENGTH - 1)
    if alpha < 0.5:
        sub_a = alpha / 0.5
        bg_color = interpolate_color(NIGHT_COLOR, DAY_COLOR, sub_a)
    else:
        sub_a = (alpha - 0.5) / 0.5
        bg_color = interpolate_color(DAY_COLOR, NIGHT_COLOR, sub_a)
    screen.fill(bg_color)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused

    dt = clock.tick(30)
    fps_sample_timer += dt
    frames += 1
    avg_frame_time = 0.9 * avg_frame_time + 0.1 * dt

    if not paused:
        new_creatures = []
        dead = []

        for c in creatures:
            status, other = c.update(creatures, foods)
            if status == "starved":
                death_log["starved"] += 1
                write_log(f"Death: starved (speed={c.speed:.2f})")
                dead.append(c)
                if random.random() < 0.3:
                    foods.append(Food())
            elif status == "aged":
                death_log["aged"] += 1
                write_log(f"Death: aged (speed={c.speed:.2f})")
                dead.append(c)
                if random.random() < 0.3:
                    foods.append(Food())
            elif status == "ate_creature":
                # c ate 'other'
                death_log["hunted"] += 1
                # if c=the eater, we remove 'other'
                # if c=the meal, we remove c
                if other is c:
                    # means c lost the fight
                    write_log(f"Death: hunted (speed={c.speed:.2f}) [lost fight vs. predator]")
                    dead.append(c)
                else:
                    write_log(f"Death: hunted (speed={other.speed:.2f})")
                    dead.append(other)
            else:
                # alive => maybe reproduce
                child = c.reproduce(creatures, foods)
                if child and len(creatures) < MAX_CREATURES:
                    mut_type = child._get_mutation_type(c.dna)
                    new_creatures.append(child)
                    write_log(f"Birth: speed={child.speed:.2f}, diet={child.diet}, size={child.size_factor:.2f}, fear={child.fear:.2f}, longevity={child.longevity:.2f}, mutation={mut_type}")

        creatures = [cr for cr in creatures if cr not in dead]
        space_left = max(0, MAX_CREATURES - len(creatures))
        creatures += new_creatures[:space_left]
        population_history.append(len(creatures))

        # spawn extra food
        if len(foods) < INITIAL_FOOD and frames % 45 == 0:
            foods.append(Food())

    # ============ DRAW ALL ===================
    for f in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(f.x), int(f.y)), 4)
    for cr in creatures:
        cr.draw(screen)

    # HUD (top-left)
    # Count predators
    predator_count = sum(1 for x in creatures if x.diet == 1)
    herb_count     = len(creatures) - predator_count
    ratio_str = f"{predator_count}:{max(1,herb_count)}"

    stats = [
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Predators: {predator_count}, ratio={ratio_str}",
        f"Food: {len(foods)}",
        f"Time: {frames//30}s"
    ]
    if creatures:
        avg_speed = np.mean([c.speed for c in creatures])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']}  aged:{death_log['aged']}  hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    for i, text in enumerate(stats):
        txt = font.render(text, True, (255, 255, 255))
        screen.blit(txt, (10, 10 + i * 25))

    # FPS (top-right)
    fps_txt = f"FPS: {int(clock.get_fps())}"
    ms_txt = f"FrameTime: {avg_frame_time:.1f}ms"
    fps_surf = font.render(fps_txt, True, (255,255,255))
    ms_surf = font.render(ms_txt, True, (255,255,255))
    screen.blit(fps_surf, (1050, 10))
    screen.blit(ms_surf, (1050, 35))

    # Population graph
    graph_x, graph_y = 800, 70
    graph_w, graph_h = 380, 90
    pygame.draw.rect(screen, (50,50,50), (graph_x, graph_y, graph_w, graph_h))
    if len(population_history) > 1:
        points = []
        for i, val in enumerate(population_history):
            xx = graph_x + graph_w - (len(population_history) - i)*2
            yy = graph_y + graph_h - (val / MAX_CREATURES) * graph_h
            points.append((xx, yy))
        pygame.draw.lines(screen, GRAPH_COLOR, False, points, 2)

    pygame.display.flip()

pygame.quit()


In [7]:
# v0.4.1 emergent predators, meandering random walk, emergent fear response, sight range and LoS, Day/Night
import pygame
import numpy as np
import random
from collections import deque
from datetime import datetime

# =================== CONFIG ====================
MAX_CREATURES = 2000
INITIAL_CREATURES = 20
INITIAL_FOOD = 30
AGE_MAX = 700

ENERGY_GAIN_FROM_FOOD = 22
ENERGY_GAIN_FROM_PREY = 50   # If a predator (diet=1) eats an herbivore
ENERGY_LOSS_PER_MOVE = 0.1
REPRODUCE_ENERGY_COST = 50

MUTATION_RATE = 0.1      # general chance of big speed color shift
DIET_FLIP_CHANCE = 0.01   # chance that diet gene flips on child
FEAR_MUTATION_CHANCE = 0.2  # chance to tweak fear a bit
MAX_SPEED = 8.0

SIGHT_RANGE = 200.0      # maximum radius to see food/prey
BLOCKING_RADIUS = 5.0    # if something is within 5px of the line, it's blocked

DAY_LENGTH = 600         # frames for a full day-night cycle
DAY_COLOR = (0, 0, 50)   # “navy” for midday
NIGHT_COLOR = (0, 0, 0)  # black at midnight

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# ============== COLORS / GRAPHICS =================
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)

# ============== HELPERS ============================
def write_log(message):
    """Write to the log file every LOG_INTERVAL frames."""
    if frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return np.hypot(x2 - x1, y2 - y1)

def line_distance_point(px, py, ax, ay, bx, by):
    """
    Returns the shortest distance from point (px,py) to line segment A->B.
    Used for naive line-of-sight blocking.
    """
    # vector A->B
    ABx = bx - ax
    ABy = by - ay
    # vector A->P
    APx = px - ax
    APy = py - ay

    # Dot product, length squared
    AB_len_sq = ABx*ABx + ABy*ABy
    if AB_len_sq == 0:
        # A and B are the same point
        return distance(px, py, ax, ay)

    t = (APx*ABx + APy*ABy) / AB_len_sq
    # clamp between 0 and 1 to stay on segment
    t = max(0, min(1, t))

    # projection point on segment
    projx = ax + t * ABx
    projy = ay + t * ABy
    return distance(px, py, projx, projy)

def interpolate_color(c1, c2, alpha):
    """Linear interpolate between two RGB tuples, alpha in [0..1]."""
    return (
        int(c1[0] + (c2[0] - c1[0]) * alpha),
        int(c1[1] + (c2[1] - c1[1]) * alpha),
        int(c1[2] + (c2[2] - c1[2]) * alpha),
    )

# ============== CLASSES ============================
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    dna layout = [speed, r, g, b, diet, fear]
      - diet: 0=herbivore, 1=predator
      - fear: in [0..1], chance of fleeing predators if herbivore
    We do a correlated random walk by storing self.heading
    and adjusting it slightly each frame, so creatures 
    'wander' rather than 'jitter in place'.
    """
    def __init__(self, x, y, parent_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0

        # Keep a heading for random wandering
        self.heading = random.random() * 2.0 * np.pi

        if parent_dna:
            speed = np.clip(parent_dna[0] * random.uniform(0.9, 1.1), 0.5, MAX_SPEED)
            r = np.clip(parent_dna[1] + random.randint(-15, 15), 50, 255)
            g = np.clip(parent_dna[2] + random.randint(-15, 15), 50, 255)
            b = np.clip(parent_dna[3] + random.randint(-15, 15), 50, 255)

            # Possibly flip diet
            diet = parent_dna[4]
            if random.random() < DIET_FLIP_CHANCE:
                diet = 1 - diet

            fear = parent_dna[5]
            # Possibly tweak fear
            if random.random() < FEAR_MUTATION_CHANCE:
                fear += random.uniform(-0.05, 0.05)
                fear = min(max(0.0, fear), 1.0)

            self.dna = [speed, r, g, b, diet, fear]

            # Extra mutation chance
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)  # big jump in speed
                self.dna[0] = np.clip(self.dna[0], 0.5, MAX_SPEED)

        else:
            # brand-new random, mostly herbivores
            # ~5% chance predator
            diet_gene = 1 if (random.random() < 0.05) else 0
            fear_gene = random.uniform(0.0, 0.2)  # start mostly low fear?

            self.dna = [
                random.uniform(1.5, 3.0),       # speed
                random.randint(50, 255),       # color R
                random.randint(50, 255),       # color G
                random.randint(50, 255),       # color B
                diet_gene,
                fear_gene
            ]

        self.speed = self.dna[0]
        self.color = (self.dna[1], self.dna[2], self.dna[3])
        self.diet = int(self.dna[4])   # 0 or 1
        self.fear = self.dna[5]        # 0..1

    def _get_mutation_type(self, parent_dna):
        if not parent_dna:
            return "initial"
        changes = []
        # speed shift
        if abs(self.dna[0] - parent_dna[0]) > 1.0:
            changes.append("speed_shift")
        # color shift
        color_diff = abs(self.dna[1] - parent_dna[1]) + \
                     abs(self.dna[2] - parent_dna[2]) + \
                     abs(self.dna[3] - parent_dna[3])
        if color_diff > 45:
            changes.append("color_shift")
        # diet flip
        if self.dna[4] != parent_dna[4]:
            changes.append("diet_flip")
        # fear changed
        if abs(self.dna[5] - parent_dna[5]) > 0.05:
            changes.append("fear_shift")

        if not changes:
            return "minor"
        return "+".join(changes)

    def update(self, creatures, foods):
        self.age += 1
        self.energy -= ENERGY_LOSS_PER_MOVE

        if self.diet == 0:
            # ============= HERBIVORE LOGIC =============
            # 1) Possibly flee if fear > some threshold AND a predator is near
            if self.fear > 0.3:  
                # find nearest predator
                pred, dist_pred = self.find_nearest(creatures, condition=lambda c: c.diet == 1)
                if pred and dist_pred < 100:  # fear radius
                    # run away
                    self.run_away(pred.x, pred.y)
                    # skip foraging this frame
                    self.wrap_screen()
                    return self.check_survival()

            # 2) Otherwise try to eat nearest food (within sight range)
            target_food, fdist = self.find_nearest(foods)
            if target_food and fdist < SIGHT_RANGE:
                # Check if line-of-sight is blocked
                if not self.is_line_blocked(target_food.x, target_food.y, creatures):
                    # move toward it
                    self.move_toward(target_food.x, target_food.y)
                    # eat if close
                    if distance(self.x, self.y, target_food.x, target_food.y) < 10:
                        self.energy += ENERGY_GAIN_FROM_FOOD
                        foods.remove(target_food)
                else:
                    # if blocked, do a random-ish wander
                    self.random_walk()
            else:
                # if no visible food, random wander
                self.random_walk()

        else:
            # ============= PREDATOR LOGIC =============
            # find nearest herbivore in sight range
            herb, dist_herb = self.find_nearest(creatures, condition=lambda c: c.diet == 0)
            if herb and dist_herb < SIGHT_RANGE:
                # check line-of-sight
                if not self.is_line_blocked(herb.x, herb.y, creatures):
                    self.move_toward(herb.x, herb.y)
                    if distance(self.x, self.y, herb.x, herb.y) < 14:
                        self.energy += ENERGY_GAIN_FROM_PREY
                        return ("ate_creature", herb)
                else:
                    self.random_walk()
            else:
                # no target or out of range => random wander
                self.random_walk()

        self.wrap_screen()
        return self.check_survival()

    def check_survival(self):
        """Check if creature is starved or aged beyond limit."""
        if self.energy <= 0:
            return ("starved", None)
        if self.age > AGE_MAX:
            return ("aged", None)
        return ("alive", None)

    def find_nearest(self, target_list, condition=None):
        """
        Return (nearest_target, distance).
        If condition is set, only consider those targets that pass condition().
        If no valid target, return (None, large_number).
        """
        valid_targets = target_list
        if condition:
            valid_targets = [t for t in target_list if condition(t)]
        if not valid_targets:
            return (None, float('inf'))

        best_obj, best_dist = None, float('inf')
        for obj in valid_targets:
            d = distance(self.x, self.y, obj.x, obj.y)
            if d < best_dist:
                best_dist = d
                best_obj = obj
        return (best_obj, best_dist)

    def is_line_blocked(self, tx, ty, creatures):
        """
        Naive line-of-sight check:
        If any creature (including self) is within BLOCKING_RADIUS
        of the line from (self.x,self.y) to (tx,ty), 
        but is *closer* to us than the target, block it.
        """
        target_dist = distance(self.x, self.y, tx, ty)
        for c in creatures:
            if c is self:
                continue
            # if c is behind the target, ignore
            dist_c = distance(self.x, self.y, c.x, c.y)
            if dist_c >= target_dist:
                continue
            # if c is near the line, block
            d_line = line_distance_point(c.x, c.y, self.x, self.y, tx, ty)
            if d_line < BLOCKING_RADIUS:
                return True
        return False

    def random_walk(self):
        """
        Correlated random walk:
        Slightly nudge heading, then move forward.
        """
        self.heading += random.uniform(-0.2, 0.2)  # small turn
        dx = np.cos(self.heading) * self.speed
        dy = np.sin(self.heading) * self.speed
        self.x += dx
        self.y += dy

    def run_away(self, ox, oy):
        """
        Move away from (ox, oy) by heading in the opposite direction.
        """
        dx = self.x - ox
        dy = self.y - oy
        dist = distance(self.x, self.y, ox, oy)
        if dist > 1:
            self.x += (dx / dist) * self.speed
            self.y += (dy / dist) * self.speed
        else:
            # fallback
            self.random_walk()

    def move_toward(self, tx, ty):
        dx = tx - self.x
        dy = ty - self.y
        distv = distance(self.x, self.y, tx, ty)
        if distv > 1:
            self.x += (dx / distv) * self.speed
            self.y += (dy / distv) * self.speed
        else:
            # fallback
            self.random_walk()

    def wrap_screen(self):
        """Wrap around edges."""
        self.x %= 1200
        self.y %= 800

    def reproduce(self):
        if self.energy > 80:
            self.energy -= REPRODUCE_ENERGY_COST
            return Creature(self.x, self.y, self.dna)
        return None

    def draw(self, surface):
        """Draw a circle if herbivore, triangle if predator."""
        if self.diet == 0:
            # circle
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), 8)
        else:
            # predator
            points = [
                (self.x, self.y - 10),
                (self.x - 7, self.y + 7),
                (self.x + 7, self.y + 7)
            ]
            pygame.draw.polygon(surface, self.color, points)

# ============== SETUP ==============================
open(LOG_FILE, "w").close()

# Create initial creatures
creatures = [
    Creature(random.randint(100, 1100), random.randint(100, 700))
    for _ in range(INITIAL_CREATURES)
]
foods = [Food() for _ in range(INITIAL_FOOD)]

paused = False
frames = 0
population_history = deque(maxlen=300)
death_log = {'starved': 0, 'aged': 0, 'hunted': 0}

day_cycle_frame = 0  # track day/night cycle
running = True

# For measuring performance
fps_sample_timer = 0.0
avg_frame_time = 0.0

# ============== MAIN LOOP ==========================
while running:
    # Calculate day/night color
    # day_cycle_frame in [0..DAY_LENGTH], then we treat [0..0.5] as "day rising", [0.5..1.0] as "night falling"
    day_cycle_frame = (day_cycle_frame + 1) % DAY_LENGTH
    alpha = day_cycle_frame / (DAY_LENGTH - 1)  # goes 0..1
    # We'll treat alpha < 0.5 as “morning -> noon”, alpha > 0.5 as “afternoon -> midnight”.
    # For a simpler approach, just do a direct interpolation:
    #  alpha=0 => NIGHT_COLOR, alpha=0.5 => DAY_COLOR, alpha=1 => NIGHT_COLOR again
    # We'll do a triangular wave:
    if alpha < 0.5:
        # 0..0.5 => 0..1 range
        sub_a = alpha / 0.5
        bg_color = interpolate_color(NIGHT_COLOR, DAY_COLOR, sub_a)
    else:
        # 0.5..1 => 1..0 range
        sub_a = (alpha - 0.5) / 0.5
        bg_color = interpolate_color(DAY_COLOR, NIGHT_COLOR, sub_a)

    screen.fill(bg_color)

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused

    dt = clock.tick(30)  # dt in ms at ~30fps
    # track performance
    fps_sample_timer += dt
    frames += 1
    # Exponential moving average for frame time
    avg_frame_time = 0.9 * avg_frame_time + 0.1 * dt

    if not paused:
        new_creatures = []
        dead = []

        # =========== UPDATE CREATURES ===========
        for c in creatures:
            status, other = c.update(creatures, foods)
            if status == "starved":
                death_log["starved"] += 1
                write_log(f"Death: starved (speed={c.speed:.2f})")
                dead.append(c)
                # 30% chance to spawn new food
                if random.random() < 0.3:
                    foods.append(Food())

            elif status == "aged":
                death_log["aged"] += 1
                write_log(f"Death: aged (speed={c.speed:.2f})")
                dead.append(c)
                if random.random() < 0.3:
                    foods.append(Food())

            elif status == "ate_creature":
                # c ate 'other'
                death_log["hunted"] += 1
                write_log(f"Death: hunted (speed={other.speed:.2f})")
                dead.append(other)

            else:
                # "alive" => possibly reproduce
                child = c.reproduce()
                if child and len(creatures) < MAX_CREATURES:
                    mut_type = child._get_mutation_type(c.dna)
                    new_creatures.append(child)
                    write_log(f"Birth: speed={child.speed:.2f}, diet={child.diet}, fear={child.fear:.2f}, mutation={mut_type}")

        # remove the dead
        creatures = [cr for cr in creatures if cr not in dead]
        # add newborns
        space_left = max(0, MAX_CREATURES - len(creatures))
        creatures += new_creatures[:space_left]

        # track population
        population_history.append(len(creatures))

        # spawn more food if needed (same logic as older version)
        if len(foods) < INITIAL_FOOD and frames % 45 == 0:
            foods.append(Food())

    # ============= DRAW OBJECTS =================
    for f in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(f.x), int(f.y)), 4)

    for c in creatures:
        c.draw(screen)

    # ============= HUD ==========================
    # top-left
    stats = [
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Food: {len(foods)}",
        f"Time: {frames//30}s",
    ]
    if creatures:
        avg_speed = np.mean([c.speed for c in creatures])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']}  aged:{death_log['aged']}  hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    for i, text in enumerate(stats):
        txt = font.render(text, True, (255, 255, 255))
        screen.blit(txt, (10, 10 + i * 25))

    # FPS + performance on top-right
    fps_txt = f"FPS: {int(clock.get_fps())}"
    ms_txt = f"FrameTime: {avg_frame_time:.1f}ms"
    fps_surf = font.render(fps_txt, True, (255,255,255))
    ms_surf = font.render(ms_txt, True, (255,255,255))
    screen.blit(fps_surf, (1050, 10))
    screen.blit(ms_surf, (1050, 35))

    # Simple population graph in top-right corner (below FPS)
    graph_x = 800
    graph_y = 70
    graph_w = 380
    graph_h = 90
    pygame.draw.rect(screen, (50, 50, 50), (graph_x, graph_y, graph_w, graph_h))
    if len(population_history) > 1:
        points = []
        max_val = MAX_CREATURES
        for i, val in enumerate(population_history):
            xx = graph_x + graph_w - (len(population_history) - i)*2
            yy = graph_y + graph_h - (val / max_val) * graph_h
            points.append((xx, yy))
        pygame.draw.lines(screen, GRAPH_COLOR, False, points, 2)

    pygame.display.flip()

pygame.quit()


2025-01-28 00:04:32.109 python[12491:16260095] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-28 00:04:32.109 python[12491:16260095] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [ ]:
# v0.4 gpt pro, adding predators, enhanced dna, mutation logging
import pygame
import numpy as np
import random
from collections import deque
from datetime import datetime

# ========== CONFIGURATION ==========
MAX_CREATURES = 2000
INITIAL_CREATURES = 10
INITIAL_FOOD = 30
AGE_MAX = 700

ENERGY_GAIN_FROM_FOOD = 22
ENERGY_LOSS_PER_MOVE = 0.1
REPRODUCE_ENERGY_COST = 50
MUTATION_RATE = 0.15
MAX_SPEED = 8.0

LOG_INTERVAL = 30
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = f"sim_log_{timestamp}.txt"

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Colors
BACKGROUND = (30, 30, 30)
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)

# ========== HELPER FUNCTIONS ==========
def write_log(message):
    """Write to the log file every LOG_INTERVAL frames."""
    if frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {frames}] {message}\n")

def distance(x1, y1, x2, y2):
    return np.hypot(x2 - x1, y2 - y1)

# ========== CLASSES ==========
class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    """
    Single species, but each has a diet gene:
      0 => herbivore (eats food)
      1 => carnivore (hunts herbivores)
    """
    def __init__(self, x, y, parent_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0

        if parent_dna:
            # Slight variation from parent
            speed = np.clip(parent_dna[0] * random.uniform(0.9, 1.1), 0.5, MAX_SPEED)
            r = np.clip(parent_dna[1] + random.randint(-15, 15), 50, 255)
            g = np.clip(parent_dna[2] + random.randint(-15, 15), 50, 255)
            b = np.clip(parent_dna[3] + random.randint(-15, 15), 50, 255)
            diet = parent_dna[4]

            # Random chance to flip diet from 0->1 or 1->0
            if random.random() < 0.05:  # 5% chance to mutate diet
                diet = 1 - diet  # flip

            self.dna = [speed, r, g, b, diet]

            # Extra big mutation chance
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)  # big jump in speed
                self.dna[0] = np.clip(self.dna[0], 0.5, MAX_SPEED)
        else:
            # Brand new random
            # diet is randomly 0 or 1 (most likely 0 to start, you can tweak)
            diet_gene = 0 if random.random() < 0.95 else 1
            self.dna = [
                random.uniform(1.5, 3.0),      # speed
                random.randint(50, 255),      # color R
                random.randint(50, 255),      # color G
                random.randint(50, 255),      # color B
                diet_gene                      # 0=herb, 1=carn
            ]

        self.speed = self.dna[0]
        self.color = (self.dna[1], self.dna[2], self.dna[3])
        self.diet = int(self.dna[4])  # 0 or 1

    def _get_mutation_type(self, parent_dna):
        """Check for big changes vs. parent's DNA."""
        if not parent_dna:
            return "initial"

        changes = []
        # big speed shift
        if abs(self.dna[0] - parent_dna[0]) > 1.0:
            changes.append("speed_shift")
        # color shift
        color_diff = abs(self.dna[1] - parent_dna[1]) + \
                     abs(self.dna[2] - parent_dna[2]) + \
                     abs(self.dna[3] - parent_dna[3])
        if color_diff > 45:
            changes.append("color_shift")
        # diet flip
        if self.dna[4] != parent_dna[4]:
            changes.append("diet_flip")

        if not changes:
            return "minor"
        return "+".join(changes)

    def update(self, creatures, foods):
        """ 
        - If herbivore (diet=0), look for nearest food.
        - If carnivore (diet=1), look for nearest herbivore.
        """
        self.age += 1
        self.energy -= ENERGY_LOSS_PER_MOVE

        if self.diet == 0:
            # Herbivore logic
            if random.random() < 0.3 or not foods:
                self.move_random()
            else:
                # find nearest food
                food, dist = min(
                    [(f, distance(self.x, self.y, f.x, f.y)) for f in foods],
                    key=lambda x: x[1],
                    default=(None, float('inf'))
                )
                if food:
                    self.move_toward(food.x, food.y)
                    # Eat if close enough
                    if distance(self.x, self.y, food.x, food.y) < 10:
                        self.energy += ENERGY_GAIN_FROM_FOOD
                        foods.remove(food)
        else:
            # Carnivore logic (diet=1)
            # tries to hunt the nearest herbivore
            # (i.e. any creature with diet=0)
            possible_targets = [c for c in creatures if c is not self and c.diet == 0]
            if not possible_targets or random.random() < 0.3:
                self.move_random()
            else:
                target, dist = min(
                    [(c, distance(self.x, self.y, c.x, c.y)) for c in possible_targets],
                    key=lambda x: x[1],
                    default=(None, float('inf'))
                )
                if target:
                    self.move_toward(target.x, target.y)
                    # If close enough, "eat" that creature
                    if distance(self.x, self.y, target.x, target.y) < 14:
                        self.energy += 50  # or some other carnivorous gain
                        return ("ate_creature", target)

        # wrap around
        self.x %= 1200
        self.y %= 800

        # check survival
        if self.energy <= 0:
            return ("starved", None)
        if self.age > AGE_MAX:
            return ("aged", None)
        return ("alive", None)

    def move_random(self):
        self.x += random.uniform(-self.speed, self.speed)
        self.y += random.uniform(-self.speed, self.speed)

    def move_toward(self, tx, ty):
        dx = tx - self.x
        dy = ty - self.y
        dist = max(1, distance(self.x, self.y, tx, ty))
        self.x += (dx / dist) * self.speed
        self.y += (dy / dist) * self.speed

    def reproduce(self):
        """If energy is high, spawn a child. (Same as original logic)"""
        if self.energy > 80:
            self.energy -= REPRODUCE_ENERGY_COST
            return Creature(self.x, self.y, self.dna)
        return None

    def draw(self, surface):
        """Draw differently if herbivore or carnivore."""
        if self.diet == 0:
            # circle if herbivore
            pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), 8)
        else:
            # small triangle if carnivore
            points = [
                (self.x, self.y - 10),
                (self.x - 7, self.y + 7),
                (self.x + 7, self.y + 7)
            ]
            pygame.draw.polygon(surface, self.color, points)

# ========== SETUP ==========
open(LOG_FILE, "w").close()

creatures = [
    Creature(random.randint(100, 1100), random.randint(100, 700))
    for _ in range(INITIAL_CREATURES)
]
foods = [Food() for _ in range(INITIAL_FOOD)]

paused = False
frames = 0
population_history = deque(maxlen=300)
death_log = {
    'starved': 0,
    'aged': 0,
    'hunted': 0
}

running = True

# ========== MAIN LOOP ==========
while running:
    screen.fill(BACKGROUND)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused

    if not paused:
        frames += 1

        new_creatures = []
        dead = []

        # Update creatures
        for creature in creatures:
            result, other = creature.update(creatures, foods)
            if result == "starved":
                death_log['starved'] += 1
                write_log(f"Death: starved (speed={creature.speed:.2f})")
                dead.append(creature)
                # ~30% chance to spawn food
                if random.random() < 0.3:
                    foods.append(Food())

            elif result == "aged":
                death_log['aged'] += 1
                write_log(f"Death: aged (speed={creature.speed:.2f})")
                dead.append(creature)
                if random.random() < 0.3:
                    foods.append(Food())

            elif result == "ate_creature":
                # The creature has eaten `other`
                # We'll remove that ‘other’ from the world
                death_log['hunted'] += 1
                write_log(f"Death: hunted (speed={other.speed:.2f})")
                dead.append(other)
                # note: the eater is still “alive” so do not remove it

            else:
                # alive => maybe reproduce
                child = creature.reproduce()
                if child and len(creatures) < MAX_CREATURES:
                    mut_type = child._get_mutation_type(creature.dna)
                    new_creatures.append(child)
                    write_log(f"Birth: speed={child.speed:.2f}, diet={child.diet}, mutation={mut_type}")

        # Remove all that died
        creatures = [c for c in creatures if c not in dead]
        # Add children (bounded by max creatures)
        creatures += new_creatures[:max(0, MAX_CREATURES - len(creatures))]

        population_history.append(len(creatures))

        # Restore cyc-based food spawning
        # (like original: if below initial, add a new one every 45 frames)
        if len(foods) < INITIAL_FOOD and frames % 45 == 0:
            foods.append(Food())

    # ========== DRAWING ==========
    for f in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(f.x), int(f.y)), 4)
    for c in creatures:
        c.draw(screen)

    # HUD
    stats = [
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Food: {len(foods)}",
        f"Time: {frames//30}s",
    ]
    if creatures:
        avg_speed = np.mean([c.speed for c in creatures])
        stats.append(f"Avg Speed: {avg_speed:.2f}")
    stats.append(f"Deaths => starved:{death_log['starved']}  aged:{death_log['aged']}  hunted:{death_log['hunted']}")
    stats.append("[SPACE] Pause/Resume")

    for i, text in enumerate(stats):
        txt = font.render(text, True, (255, 255, 255))
        screen.blit(txt, (10, 10 + i * 25))

    # Simple population graph in top-right corner
    pygame.draw.rect(screen, (50, 50, 50), (800, 10, 380, 150))
    if len(population_history) > 1:
        points = []
        max_val = MAX_CREATURES
        for i, val in enumerate(population_history):
            x = 800 + 380 - (len(population_history) - i)*2
            y = 10 + 150 - (val / max_val) * 150
            points.append((x, y))
        pygame.draw.lines(screen, GRAPH_COLOR, False, points, 2)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()


In [ ]:
# v0.3.1 optimised and increased to 2k

import pygame
import numpy as np
import random
from collections import deque
from datetime import datetime

# Generate timestamp with format: YYYYMMDD_HHMMSS
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# ==== CONFIGURATION ====
MAX_CREATURES = 2000
INITIAL_CREATURES = 10
INITIAL_FOOD = 30     
AGE_MAX = 700
ENERGY_GAIN_FROM_FOOD = 22
ENERGY_LOSS_PER_MOVE = 0.1
REPRODUCE_ENERGY_COST = 50
MUTATION_RATE = 0.15
MAX_SPEED = 8.0
LOG_INTERVAL = 30
LOG_FILE = f"sim_log_{timestamp}.txt"  # Fixed f-string
# =======================

pygame.init()
screen = pygame.display.set_mode((1200, 800))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Colors
BACKGROUND = (30, 30, 30)
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)

class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    def __init__(self, x, y, parent_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        
        if parent_dna:
            self.dna = [
                np.clip(parent_dna[0] * random.uniform(0.9, 1.1), 0.5, MAX_SPEED),
                np.clip(parent_dna[1] + random.randint(-15, 15), 50, 255),
                np.clip(parent_dna[2] + random.randint(-15, 15), 50, 255),
                np.clip(parent_dna[3] + random.randint(-15, 15), 50, 255)
            ]
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
                self.dna[0] = np.clip(self.dna[0], 0.5, MAX_SPEED)
        else:
            self.dna = [
                random.uniform(1.5, 3.0),
                random.randint(50, 255),
                random.randint(50, 255),
                random.randint(50, 255)
            ]
        
        self.speed = self.dna[0]
        self.color = (self.dna[1], self.dna[2], self.dna[3])

    def _get_mutation_type(self, parent_dna):
        """Analyze mutation differences"""
        if not parent_dna:
            return "initial"
        speed_ratio = self.dna[0] / parent_dna[0]
        if speed_ratio > 1.5 or speed_ratio < 0.6:
            return "megamutation"
        return "normal"

    def update(self, foods):
        self.age += 1
        self.energy -= ENERGY_LOSS_PER_MOVE
        
        # Find food with random exploration
        if random.random() < 0.3:  # 30% chance to pick random direction
            self.x += random.uniform(-self.speed, self.speed)
            self.y += random.uniform(-self.speed, self.speed)
        else:
            food = min(
                [(f, np.hypot(self.x-f.x, self.y-f.y)) for f in foods],
                key=lambda x: x[1],
                default=(None, float('inf'))
            )[0]
            if food:
                dx = food.x - self.x
                dy = food.y - self.y
                dist = max(1, np.hypot(dx, dy))
                self.x += dx/dist * self.speed
                self.y += dy/dist * self.speed

                if np.hypot(self.x-food.x, self.y-food.y) < 10:
                    self.energy += ENERGY_GAIN_FROM_FOOD
                    foods.remove(food)

        # Wrap around screen edges
        self.x = self.x % 1200
        self.y = self.y % 800

        if self.energy <= 0:
            return 'starved'
        if self.age > AGE_MAX:
            return 'aged'
        return 'alive'

    def reproduce(self):
        if self.energy > 80 and len(creatures) < MAX_CREATURES:
            self.energy -= REPRODUCE_ENERGY_COST
            return Creature(self.x, self.y, self.dna)
        return None

def draw_graph(surface, history, max_value, pos, size):
    if len(history) < 2: return
    points = []
    for i, val in enumerate(history):
        x = pos[0] + size[0] - (len(history) - i) * 2
        y = pos[1] + size[1] - (val / max_value) * size[1]
        points.append((x, y))
    pygame.draw.lines(surface, GRAPH_COLOR, False, points, 2)

def write_log(message):
    if frames % LOG_INTERVAL == 0:
        with open(LOG_FILE, "a") as f:
            f.write(f"[Frame {frames}] {message}\n")

# Initialize
open(LOG_FILE, "w").close()
creatures = [Creature(random.randint(100, 1100), random.randint(100, 700)) 
             for _ in range(INITIAL_CREATURES)]
foods = [Food() for _ in range(INITIAL_FOOD)]
paused = False
frames = 0
population_history = deque(maxlen=300)
death_log = {'starved': 0, 'aged': 0}

# Main loop
running = True
while running:
    screen.fill(BACKGROUND)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
            paused = not paused
    
    if not paused:
        frames += 1
        new_creatures = []
        dead = []
        
        # Update creatures
        for creature in creatures:
            status = creature.update(foods)
            if status != 'alive':
                write_log(f"Death: {status} | Speed: {creature.speed:.2f}")
                dead.append(creature)
                death_log[status] += 1
                if random.random() < 0.3:
                    foods.append(Food())
            elif (child := creature.reproduce()):
                write_log(f"Birth: Speed: {child.speed:.2f}")
                new_creatures.append(child)
        
        # Update populations
        creatures = [c for c in creatures if c not in dead]
        creatures += new_creatures[:MAX_CREATURES - len(creatures)]
        population_history.append(len(creatures))
        
        # Spawn food
        if len(foods) < INITIAL_FOOD and frames % 45 == 0:
            foods.append(Food())

    # Draw everything
    for food in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(food.x), int(food.y)), 4)
    for creature in creatures:
        pygame.draw.circle(screen, creature.color, (int(creature.x), int(creature.y)), 8)
    
    # Dashboard
    stats = [
        f"Creatures: {len(creatures)}/{MAX_CREATURES}",
        f"Food: {len(foods)}",
        f"Time: {frames//30}s",
        f"Avg Speed: {np.mean([c.speed for c in creatures]):.2f}" if creatures else "",
        f"Deaths: ★{death_log['starved']} ⏳{death_log['aged']}",
        "[SPACE] Pause/Resume"
    ]
    for i, text in enumerate(stats):
        txt = font.render(text, True, (255, 255, 255))
        screen.blit(txt, (10, 10 + i*25))
    
    pygame.draw.rect(screen, (50, 50, 50), (800, 10, 380, 150))
    if population_history:
        draw_graph(screen, population_history, MAX_CREATURES, (800, 10), (380, 150))
    
    pygame.display.flip()
    clock.tick(30)

pygame.quit()

In [ ]:
# v0.3 first sustainable at 1k

import pygame
import numpy as np
import random
from collections import deque

# ==== CONFIGURATION ====
MAX_CREATURES = 1000  # Increased from 200
INITIAL_CREATURES = 10  # Reduced from 20
INITIAL_FOOD = 30       # Reduced from 50
AGE_MAX = 700           # Longer lifespan
ENERGY_GAIN_FROM_FOOD = 22  # Reduced from 25
ENERGY_LOSS_PER_MOVE = 0.1  # Increased from 0.08
REPRODUCE_ENERGY_COST = 50  # Increased from 40
MUTATION_RATE = 0.15    # Chance for drastic mutation
MAX_SPEED = 8.0         # Absolute speed cap
LOG_FILE = "sim_log.txt"
# =======================

pygame.init()
screen = pygame.display.set_mode((1200, 800))  # Wider for graphs
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Colors
BACKGROUND = (30, 30, 30)
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)

class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    def __init__(self, x, y, parent_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        
        # DNA with mutation chance
        if parent_dna:
            self.dna = [
                np.clip(parent_dna[0] * np.random.normal(1.0, 0.1), 0.5, MAX_SPEED), #parent_dna[0] * np.random.normal(1.0, 0.1),  # Speed
                np.clip(parent_dna[1] + random.randint(-15, 15), 50, 255),
                np.clip(parent_dna[2] + random.randint(-15, 15), 50, 255),
                np.clip(parent_dna[3] + random.randint(-15, 15), 50, 255)
            ]
            # Apply occasional big mutation
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
        else:
            self.dna = [
                random.uniform(1.0, 3.0),
                random.randint(50, 255),
                random.randint(50, 255),
                random.randint(50, 255)
            ]
        
        self.speed = max(0.5, self.dna[0])  # Prevent negative speed
        self.color = (self.dna[1], self.dna[2], self.dna[3])

    def update(self, foods):
        self.age += 1
        self.energy -= ENERGY_LOSS_PER_MOVE
        
        # Find food
        food = min(
            [(f, distance(self.x, self.y, f.x, f.y)) for f in foods],
            key=lambda x: x[1],
            default=(None, float('inf'))
        )[0]
        
        if food and food in foods:
            angle = np.arctan2(food.y - self.y, food.x - self.x)
            self.x += np.cos(angle) * self.speed
            self.y += np.sin(angle) * self.speed
            
            if distance(self.x, self.y, food.x, food.y) < 10:
                self.energy += ENERGY_GAIN_FROM_FOOD
                foods.remove(food)
        else:
            # Wander
            self.x += random.uniform(-self.speed, self.speed)
            self.y += random.uniform(-self.speed, self.speed)
        
        # Boundaries
        self.x = np.clip(self.x, 0, 1200)
        self.y = np.clip(self.y, 0, 800)
        
        # Death checks
        if self.energy <= 0:
            return 'starved'
        if self.age > AGE_MAX:
            return 'aged'
        return 'alive'

    def reproduce(self):
        if self.energy > 80:  # Higher threshold
            self.energy -= REPRODUCE_ENERGY_COST
            return Creature(self.x, self.y, parent_dna=self.dna)
        return None


def distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def draw_graph(surface, history, max_value, pos, size):
    """Draw a scrolling population graph"""
    if len(history) < 2: return
    points = []
    for i, val in enumerate(history):
        x = pos[0] + size[0] - (len(history) - i) * 2
        y = pos[1] + size[1] - (val / max_value) * size[1]
        points.append((x, y))
    pygame.draw.lines(surface, GRAPH_COLOR, False, points, 2)

def write_log(message):
    with open(LOG_FILE, "a") as f:
        f.write(message + "\n")

# Initialize with empty log
open(LOG_FILE, "w").close()    

# Initialize
creatures = [Creature(random.randint(100, 1100), random.randint(100, 700)) 
             for _ in range(INITIAL_CREATURES)]
foods = [Food() for _ in range(INITIAL_FOOD)]
paused = False
frames = 0
population_history = deque(maxlen=300)  # Track last 300 frames
death_log = {'starved': 0, 'aged': 0}

# Main loop
running = True
while running:
    frames += 1
    screen.fill(BACKGROUND)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused
    
    # Spawn food gradually (slower than before)
    if len(foods) < INITIAL_FOOD and frames % 45 == 0:
        foods.append(Food())
    
    if not paused:
        new_creatures = []
        dead = []
        for creature in creatures:
            status = creature.update(foods)
            if status != 'alive':
                write_log(f"Death: {status} | Speed: {creature.speed:.2f} | Age: {creature.age}")
                dead.append(creature)
                death_log[status] += 1
                # Spawn food on death
                if random.random() < 0.3:
                    foods.append(Food())
            elif (child := creature.reproduce()) and len(creatures) < MAX_CREATURES:
                write_log(f"Birth: Speed: {child.speed:.2f} | Color: {child.color}")
                new_creatures.append(child)
        
        creatures = [c for c in creatures if c not in dead][:MAX_CREATURES] + new_creatures[:MAX_CREATURES - len(creatures)]
        population_history.appends(len(creatures))
    
    # Draw food
    for food in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(food.x), int(food.y)), 4)
    
    # Draw creatures
    for creature in creatures:
        pygame.draw.circle(screen, creature.color, (int(creature.x), int(creature.y)), 8)
    
    # ==== Dashboard ====
    # Text stats
    stats = [
        f"Creatures: {len(creatures)} / {MAX_CREATURES}",
        f"Food: {len(foods)}",
        f"Time: {frames//30}s",
        f"Avg Speed: {np.mean([c.speed for c in creatures]):.2f}" if creatures else "",
        f"Deaths: Starved {death_log['starved']} | Aged {death_log['aged']}",
        f"[SPACE] {'Resume' if paused else 'Pause'}"
    ]
    for i, text in enumerate(stats):
        txt = font.render(text, True, (255, 255, 255))
        screen.blit(txt, (10, 10 + i*25))
    
    # Population graph
    pygame.draw.rect(screen, (50, 50, 50), (800, 10, 380, 150))
    if population_history:
        draw_graph(screen, population_history, MAX_CREATURES, (800, 10), (380, 150))
    
    pygame.display.flip()
    clock.tick(30)

pygame.quit()

In [ ]:
# v0.2 premature explosions
import pygame
import numpy as np
import random
from collections import deque

# ==== CONFIGURATION ====
MAX_CREATURES = 1000  # Increased from 200
INITIAL_CREATURES = 10  # Reduced from 20
INITIAL_FOOD = 30       # Reduced from 50
AGE_MAX = 700           # Longer lifespan
ENERGY_GAIN_FROM_FOOD = 22  # Reduced from 25
ENERGY_LOSS_PER_MOVE = 0.1  # Increased from 0.08
REPRODUCE_ENERGY_COST = 50  # Increased from 40
MUTATION_RATE = 0.15    # Chance for drastic mutation
# =======================

pygame.init()
screen = pygame.display.set_mode((1200, 800))  # Wider for graphs
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Colors
BACKGROUND = (30, 30, 30)
FOOD_COLOR = (100, 200, 100)
GRAPH_COLOR = (200, 100, 150)

class Food:
    def __init__(self):
        self.x = random.randint(50, 1150)
        self.y = random.randint(50, 750)

class Creature:
    def __init__(self, x, y, parent_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        
        # DNA with mutation chance
        if parent_dna:
            self.dna = [
                parent_dna[0] * np.random.normal(1.0, 0.1),  # Speed
                np.clip(parent_dna[1] + random.randint(-15, 15), 50, 255),
                np.clip(parent_dna[2] + random.randint(-15, 15), 50, 255),
                np.clip(parent_dna[3] + random.randint(-15, 15), 50, 255)
            ]
            # Apply occasional big mutation
            if random.random() < MUTATION_RATE:
                self.dna[0] *= random.uniform(0.5, 2.0)
        else:
            self.dna = [
                random.uniform(1.0, 3.0),
                random.randint(50, 255),
                random.randint(50, 255),
                random.randint(50, 255)
            ]
        
        self.speed = max(0.5, self.dna[0])  # Prevent negative speed
        self.color = (self.dna[1], self.dna[2], self.dna[3])

    def update(self, foods):
        self.age += 1
        self.energy -= ENERGY_LOSS_PER_MOVE
        
        # Find food
        food = min(
            [(f, distance(self.x, self.y, f.x, f.y)) for f in foods],
            key=lambda x: x[1],
            default=(None, float('inf'))
        )[0]
        
        if food and food in foods:
            angle = np.arctan2(food.y - self.y, food.x - self.x)
            self.x += np.cos(angle) * self.speed
            self.y += np.sin(angle) * self.speed
            
            if distance(self.x, self.y, food.x, food.y) < 10:
                self.energy += ENERGY_GAIN_FROM_FOOD
                foods.remove(food)
        else:
            # Wander
            self.x += random.uniform(-self.speed, self.speed)
            self.y += random.uniform(-self.speed, self.speed)
        
        # Boundaries
        self.x = np.clip(self.x, 0, 1200)
        self.y = np.clip(self.y, 0, 800)
        
        # Death checks
        if self.energy <= 0:
            return 'starved'
        if self.age > AGE_MAX:
            return 'aged'
        return 'alive'

    def reproduce(self):
        if self.energy > 80:  # Higher threshold
            self.energy -= REPRODUCE_ENERGY_COST
            return Creature(self.x, self.y, parent_dna=self.dna)
        return None

def distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def draw_graph(surface, history, max_value, pos, size):
    """Draw a scrolling population graph"""
    if len(history) < 2: return
    points = []
    for i, val in enumerate(history):
        x = pos[0] + size[0] - (len(history) - i) * 2
        y = pos[1] + size[1] - (val / max_value) * size[1]
        points.append((x, y))
    pygame.draw.lines(surface, GRAPH_COLOR, False, points, 2)

# Initialize
creatures = [Creature(random.randint(100, 1100), random.randint(100, 700)) 
             for _ in range(INITIAL_CREATURES)]
foods = [Food() for _ in range(INITIAL_FOOD)]
paused = False
frames = 0
population_history = deque(maxlen=300)  # Track last 300 frames
death_log = {'starved': 0, 'aged': 0}

# Main loop
running = True
while running:
    frames += 1
    screen.fill(BACKGROUND)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                paused = not paused
    
    # Spawn food gradually (slower than before)
    if len(foods) < INITIAL_FOOD and frames % 45 == 0:
        foods.append(Food())
    
    if not paused:
        # Update creatures
        new_creatures = []
        dead = []
        for creature in creatures:
            status = creature.update(foods)
            if status != 'alive':
                dead.append(creature)
                death_log[status] += 1
                # Spawn food on death
                if random.random() < 0.3:
                    foods.append(Food())
            elif child := creature.reproduce():
                new_creatures.append(child)
        
        creatures = [c for c in creatures if c not in dead] + new_creatures
        population_history.append(len(creatures))
    
    # Draw food
    for food in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(food.x), int(food.y)), 4)
    
    # Draw creatures
    for creature in creatures:
        pygame.draw.circle(screen, creature.color, (int(creature.x), int(creature.y)), 8)
    
    # ==== Dashboard ====
    # Text stats
    stats = [
        f"Creatures: {len(creatures)} / {MAX_CREATURES}",
        f"Food: {len(foods)}",
        f"Time: {frames//30}s",
        f"Avg Speed: {np.mean([c.speed for c in creatures]):.2f}" if creatures else "",
        f"Deaths: Starved {death_log['starved']} | Aged {death_log['aged']}",
        f"[SPACE] {'Resume' if paused else 'Pause'}"
    ]
    for i, text in enumerate(stats):
        txt = font.render(text, True, (255, 255, 255))
        screen.blit(txt, (10, 10 + i*25))
    
    # Population graph
    pygame.draw.rect(screen, (50, 50, 50), (800, 10, 380, 150))
    if population_history:
        draw_graph(screen, population_history, MAX_CREATURES, (800, 10), (380, 150))
    
    pygame.display.flip()
    clock.tick(30)

pygame.quit()

In [ ]:
# v0.1 stops too fast
import pygame
import numpy as np
import random
from pygame.locals import *

# ==== CONFIGURATION (Tweak these!) ====
MAX_CREATURES = 200  # Pause sim when exceeded
INITIAL_CREATURES = 20
INITIAL_FOOD = 50
AGE_MAX = 500  # Frames until death by aging
ENERGY_GAIN_FROM_FOOD = 25
ENERGY_LOSS_PER_MOVE = 0.08
# ======================================

pygame.init()
screen = pygame.display.set_mode((1000, 700))
font = pygame.font.Font(None, 24)
clock = pygame.time.Clock()

# Colors
BACKGROUND = (30, 30, 30)
FOOD_COLOR = (100, 200, 100)

class Food:
    def __init__(self):
        self.x = random.randint(50, 950)
        self.y = random.randint(50, 650)
        self.energy = ENERGY_GAIN_FROM_FOOD

class Creature:
    def __init__(self, x, y, parent_dna=None):
        self.x = x
        self.y = y
        self.energy = 100
        self.age = 0
        
        # DNA: [speed, color_r, color_g, color_b]
        if parent_dna:
            self.dna = [
                parent_dna[0] + random.uniform(-0.2, 0.2),
                np.clip(parent_dna[1] + random.randint(-10, 10), 50, 255),
                np.clip(parent_dna[2] + random.randint(-10, 10), 50, 255),
                np.clip(parent_dna[3] + random.randint(-10, 10), 50, 255)
            ]
        else:
            self.dna = [
                random.uniform(1.5, 3.0),  # Speed
                random.randint(50, 255),   # R
                random.randint(50, 255),   # G
                random.randint(50, 255)    # B
            ]
        
        self.speed = self.dna[0]
        self.color = (self.dna[1], self.dna[2], self.dna[3])

    def move_toward(self, target_x, target_y):
        angle = np.arctan2(target_y - self.y, target_x - self.x)
        self.x += np.cos(angle) * self.speed
        self.y += np.sin(angle) * self.speed
        self.energy -= ENERGY_LOSS_PER_MOVE
        self.age += 1

    def find_closest_food(self, foods):
        closest = None
        min_dist = float('inf')
        for food in foods:
            d = (self.x - food.x)**2 + (self.y - food.y)**2
            if d < min_dist and d < 2500:  # 50px radius
                closest = food
                min_dist = d
        return closest

    def update(self, foods):
        # Aging death
        if self.age > AGE_MAX:
            return 'age_death'
        
        # Energy death
        if self.energy <= 0:
            return 'energy_death'
        
        # Find food
        food = self.find_closest_food(foods)
        if food:
            self.move_toward(food.x, food.y)
            if distance(self.x, self.y, food.x, food.y) < 10:
                self.energy += food.energy
                foods.remove(food)
                return 'ate_food'
        else:
            # Wander
            self.x += random.uniform(-self.speed, self.speed)
            self.y += random.uniform(-self.speed, self.speed)
        
        # Boundaries
        self.x = np.clip(self.x, 0, 1000)
        self.y = np.clip(self.y, 0, 700)
        return 'alive'

    def reproduce(self):
        if self.energy > 60:
            self.energy -= 40
            return Creature(self.x, self.y, parent_dna=self.dna)
        return None

def distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def draw_text(surface, text, pos):
    txt = font.render(text, True, (255, 255, 255))
    surface.blit(txt, pos)

# Initialize
creatures = [Creature(random.randint(100, 900), random.randint(100, 600)) 
             for _ in range(INITIAL_CREATURES)]
foods = [Food() for _ in range(INITIAL_FOOD)]
paused = False
frames = 0

# Main loop
running = True
while running:
    frames += 1
    screen.fill(BACKGROUND)
    
    for event in pygame.event.get():
        if event.type == QUIT:
            running = False
        if event.type == KEYDOWN:
            if event.key == K_SPACE:
                paused = not paused
    
    # Spawn food gradually
    if len(foods) < INITIAL_FOOD and frames % 30 == 0:
        foods.append(Food())
    
    if not paused and len(creatures) < MAX_CREATURES:
        # Update creatures
        new_creatures = []
        dead_creatures = []
        food_spawned = []
        
        for creature in creatures:
            status = creature.update(foods)
            
            if status == 'ate_food':
                food_spawned.append(Food())  # Dead food becomes new food after delay
            
            if status in ['age_death', 'energy_death']:
                dead_creatures.append(creature)
                # 50% chance to spawn food on death
                if random.random() < 0.5:
                    foods.append(Food())
            else:
                if child := creature.reproduce():
                    new_creatures.append(child)
        
        creatures = [c for c in creatures if c not in dead_creatures] + new_creatures
        foods += food_spawned
    
    # Draw food
    for food in foods:
        pygame.draw.circle(screen, FOOD_COLOR, (int(food.x), int(food.y)), 4)
    
    # Draw creatures
    for creature in creatures:
        pygame.draw.circle(screen, creature.color, (int(creature.x), int(creature.y)), 8)
    
    # Dashboard
    draw_text(screen, f"Creatures: {len(creatures)}", (10, 10))
    draw_text(screen, f"Food: {len(foods)}", (10, 35))
    draw_text(screen, f"Time: {frames//30}s", (10, 60))
    draw_text(screen, f"Avg Speed: {np.mean([c.speed for c in creatures]):.2f}" 
              if creatures else "Avg Speed: 0", (10, 85))
    draw_text(screen, "[SPACE] Pause/Resume", (10, 660))
    
    if len(creatures) >= MAX_CREATURES:
        draw_text(screen, "MAX POPULATION REACHED! (Press SPACE to resume)", (300, 10))
    
    pygame.display.flip()
    clock.tick(30)

pygame.quit()

In [ ]:
# version that fills screen. v0 
import pygame
import numpy as np
import random
import sys

# ==== Configuration Constants (Tweak These!) ====
INITIAL_CREATURES = 10
INITIAL_FOOD = 30
CREATURE_SPEED = 2.5       # Increased from 1.5
ENERGY_LOSS_PER_MOVE = 0.05  # Reduced from 0.1
FOOD_ENERGY = 30           # Increased from 20
REPRODUCTION_THRESHOLD = 60  # Reduced from 80
FOOD_DETECTION_RADIUS = 25  # Increased from 10
# ================================================

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((800, 600))
clock = pygame.time.Clock()
running = True

# Colors
GREEN = (0, 255, 0)
RED = (255, 0, 0)

def distance(x1, y1, x2, y2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

class Food:
    def __init__(self):
        self.respawn()
        self.value = FOOD_ENERGY

    def respawn(self):
        self.x = random.uniform(50, 750)  # Avoid edges
        self.y = random.uniform(50, 550)

class Creature:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.energy = 100
        self.speed = CREATURE_SPEED + random.uniform(-0.3, 0.3)  # Initial variation
        print(f"New creature: speed={self.speed:.2f}")

    def move_toward(self, target_x, target_y):
        """Smart movement toward a target instead of random wandering"""
        angle = np.arctan2(target_y - self.y, target_x - self.x)
        self.x += np.cos(angle) * self.speed
        self.y += np.sin(angle) * self.speed
        self.energy -= ENERGY_LOSS_PER_MOVE

    def find_closest_food(self, food_list):
        """Locate nearest food within detection radius"""
        closest = None
        min_dist = float('inf')
        for food in food_list:
            d = distance(self.x, self.y, food.x, food.y)
            if d < min_dist and d < FOOD_DETECTION_RADIUS:
                closest = food
                min_dist = d
        return closest

    def update(self, food_list):
        # Find and move toward food if hungry
        if self.energy < 80:
            target_food = self.find_closest_food(food_list)
            if target_food:
                self.move_toward(target_food.x, target_food.y)
                self.eat(target_food)
            else:
                # Wander slightly if no food nearby
                self.x += random.uniform(-self.speed, self.speed)
                self.y += random.uniform(-self.speed, self.speed)
        else:
            # Explore when energized
            self.x += random.uniform(-self.speed, self.speed)
            self.y += random.uniform(-self.speed, self.speed)
        
        # Boundary checks
        self.x = np.clip(self.x, 0, 800)
        self.y = np.clip(self.y, 0, 600)

    def eat(self, food):
        if distance(self.x, self.y, food.x, food.y) < 10:
            self.energy += food.value
            food.respawn()
            print(f"Creature ate! Energy: {self.energy:.1f}")

    def reproduce(self):
        if self.energy > REPRODUCTION_THRESHOLD:
            self.energy -= 40  # Fixed cost instead of halving
            child = Creature(self.x, self.y)
            child.speed = self.speed + random.uniform(-0.2, 0.2)
            print(f"Reproduced! Child speed: {child.speed:.2f}")
            return child
        return None

# Initialize entities
creatures = [Creature(random.randint(100, 700), random.randint(100, 500)) 
             for _ in range(INITIAL_CREATURES)]
foods = [Food() for _ in range(INITIAL_FOOD)]

# Main loop
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((0, 0, 0))
    
    # Update creatures
    new_creatures = []
    dead = []
    for creature in creatures:
        creature.update(foods)
        if child := creature.reproduce():
            new_creatures.append(child)
        if creature.energy <= 0:
            dead.append(creature)
    
    creatures = [c for c in creatures if c not in dead] + new_creatures
    
    # Spawn new food periodically
    if len(foods) < INITIAL_FOOD:
        foods.append(Food())
    
    # Draw
    for food in foods:
        pygame.draw.circle(screen, GREEN, (int(food.x), int(food.y)), 5)
    for creature in creatures:
        pygame.draw.circle(screen, RED, (int(creature.x), int(creature.y)), 8)
    
    pygame.display.flip()
    clock.tick(30)

pygame.quit()